In [ ]:
## This file implements neural networks before and after lasso selection for p0040kpresabsSTCC_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p0040kpresabsSTCC_qual.csv')
df.shape

(253, 184)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      2
1      0
2      2
3      2
4      2
      ..
248    2
249    1
250    1
251    2
252    2
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCGCTGTTGCAACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACT,...,AGCAGATAAACTTT,ACTTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTT,ACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTGATTGT,ACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTT,ACAAGTCGCTGAAATATT,AACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTT,AACTGCTCATACAGTTAAAACAGCACAAACTGCTCATACAGTTAAAACAGCACAAACTGCTCAAGAACAAAATAAAGTTCAAACACCTGTTAAAGATGTT,ST,CC,pheno
0,107,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,5,5,2
1,109,1,0,1,0,0,0,1,0,0,...,1,1,0,0,1,0,0,8,8,0
2,115,0,0,1,0,0,0,1,0,0,...,1,0,0,0,1,0,0,5,5,2
3,120335,0,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,5,5,2
4,120337,0,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,5,5,2


In [6]:
df['pheno'].value_counts()

2    95
1    94
0    64
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 183)

In [9]:
df_clean.head()

,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCGCTGTTGCAACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACT,TTTCAGCGACTT,...,AGCAGATAAACTTT,ACTTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTT,ACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTGATTGT,ACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTT,ACAAGTCGCTGAAATATT,AACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTT,AACTGCTCATACAGTTAAAACAGCACAAACTGCTCATACAGTTAAAACAGCACAAACTGCTCAAGAACAAAATAAAGTTCAAACACCTGTTAAAGATGTT,ST,CC,pheno
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,5,5,2
1,1,0,1,0,0,0,1,0,0,1,...,1,1,0,0,1,0,0,8,8,0
2,0,0,1,0,0,0,1,0,0,1,...,1,0,0,0,1,0,0,5,5,2
3,0,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,5,5,2
4,0,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,5,5,2


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 183) (253,)


In [11]:
############# Fully-Connected Neural Network ################

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

Using TensorFlow backend.


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [14]:
dat = pd.DataFrame(X_test.iloc[:,0])
dat['test'] = y_test

In [15]:
dat

,id,test
221,NRS266,2
177,NRS209,1
223,NRS272,2
49,CFBREBSa114,2
53,CFBREBSa119,1
...,...,...
162,NRS187,1
237,SR1287,0
84,CFBRSa50,0
168,NRS196,2


In [16]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [17]:
#### neural network on over-sampling data
model1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [18]:
model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model1.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 712us/step - loss: 9.8521 - accuracy: 0.3446 - val_loss: 2.8994 - val_accuracy: 0.4605
Epoch 2/1000
177/177 [==============================] - 0s 101us/step - loss: 8.3208 - accuracy: 0.3559 - val_loss: 2.4455 - val_accuracy: 0.4342
Epoch 3/1000
177/177 [==============================] - 0s 106us/step - loss: 7.2296 - accuracy: 0.3785 - val_loss: 2.0524 - val_accuracy: 0.4211
Epoch 4/1000
177/177 [==============================] - 0s 105us/step - loss: 6.2552 - accuracy: 0.3446 - val_loss: 1.7020 - val_accuracy: 0.4474
Epoch 5/1000
177/177 [==============================] - 0s 98us/step - loss: 5.0992 - accuracy: 0.3616 - val_loss: 1.4294 - val_accuracy: 0.4737
Epoch 6/1000
177/177 [==============================] - 0s 115us/step - loss: 4.1722 - accuracy: 0.3955 - val_loss: 1.2764 - val_accuracy: 0.4737
Epoch 7/1000
177/177 [==============================] - 0s 111us/step - loss: 3.

177/177 [==============================] - 0s 89us/step - loss: 0.7151 - accuracy: 0.7062 - val_loss: 0.8973 - val_accuracy: 0.5395
Epoch 113/1000
177/177 [==============================] - 0s 124us/step - loss: 0.7329 - accuracy: 0.6723 - val_loss: 0.8878 - val_accuracy: 0.5395
Epoch 114/1000
177/177 [==============================] - 0s 87us/step - loss: 0.7326 - accuracy: 0.6610 - val_loss: 0.9027 - val_accuracy: 0.5526
Epoch 115/1000
177/177 [==============================] - 0s 139us/step - loss: 0.7139 - accuracy: 0.7062 - val_loss: 0.9146 - val_accuracy: 0.5263
Epoch 116/1000
177/177 [==============================] - 0s 115us/step - loss: 0.7096 - accuracy: 0.7006 - val_loss: 0.9163 - val_accuracy: 0.5395
Epoch 117/1000
177/177 [==============================] - 0s 134us/step - loss: 0.7022 - accuracy: 0.7006 - val_loss: 0.8962 - val_accuracy: 0.5263
Epoch 118/1000
177/177 [==============================] - 0s 95us/step - loss: 0.7132 - accuracy: 0.6836 - val_loss: 0.9017 - val

Epoch 168/1000
177/177 [==============================] - 0s 121us/step - loss: 0.6563 - accuracy: 0.7288 - val_loss: 0.9321 - val_accuracy: 0.5263
Epoch 169/1000
177/177 [==============================] - 0s 88us/step - loss: 0.6911 - accuracy: 0.7175 - val_loss: 0.9564 - val_accuracy: 0.5132
Epoch 170/1000
177/177 [==============================] - 0s 76us/step - loss: 0.6412 - accuracy: 0.7232 - val_loss: 0.9550 - val_accuracy: 0.5263
Epoch 171/1000
177/177 [==============================] - 0s 69us/step - loss: 0.6496 - accuracy: 0.7175 - val_loss: 0.9440 - val_accuracy: 0.5263
Epoch 172/1000
177/177 [==============================] - 0s 69us/step - loss: 0.6416 - accuracy: 0.7232 - val_loss: 0.9390 - val_accuracy: 0.5132
Epoch 173/1000
177/177 [==============================] - 0s 66us/step - loss: 0.6498 - accuracy: 0.7401 - val_loss: 0.9430 - val_accuracy: 0.5132
Epoch 174/1000
177/177 [==============================] - 0s 65us/step - loss: 0.6533 - accuracy: 0.7288 - val_loss: 

Epoch 224/1000
177/177 [==============================] - 0s 102us/step - loss: 0.5789 - accuracy: 0.7797 - val_loss: 0.9833 - val_accuracy: 0.5526
Epoch 225/1000
177/177 [==============================] - 0s 102us/step - loss: 0.5856 - accuracy: 0.7740 - val_loss: 0.9701 - val_accuracy: 0.5526
Epoch 226/1000
177/177 [==============================] - 0s 96us/step - loss: 0.5871 - accuracy: 0.7514 - val_loss: 0.9688 - val_accuracy: 0.5395
Epoch 227/1000
177/177 [==============================] - 0s 121us/step - loss: 0.6100 - accuracy: 0.7458 - val_loss: 0.9871 - val_accuracy: 0.5395
Epoch 228/1000
177/177 [==============================] - 0s 107us/step - loss: 0.5817 - accuracy: 0.7684 - val_loss: 0.9802 - val_accuracy: 0.5395
Epoch 229/1000
177/177 [==============================] - 0s 76us/step - loss: 0.5797 - accuracy: 0.7458 - val_loss: 0.9686 - val_accuracy: 0.5395
Epoch 230/1000
177/177 [==============================] - 0s 105us/step - loss: 0.5758 - accuracy: 0.7514 - val_lo

Epoch 279/1000
177/177 [==============================] - 0s 78us/step - loss: 0.5493 - accuracy: 0.7853 - val_loss: 1.0167 - val_accuracy: 0.5395
Epoch 280/1000
177/177 [==============================] - 0s 77us/step - loss: 0.5307 - accuracy: 0.7966 - val_loss: 1.0040 - val_accuracy: 0.5395
Epoch 281/1000
177/177 [==============================] - 0s 84us/step - loss: 0.5571 - accuracy: 0.7966 - val_loss: 0.9901 - val_accuracy: 0.5263
Epoch 282/1000
177/177 [==============================] - 0s 80us/step - loss: 0.5384 - accuracy: 0.7966 - val_loss: 1.0273 - val_accuracy: 0.5263
Epoch 283/1000
177/177 [==============================] - 0s 61us/step - loss: 0.5399 - accuracy: 0.7797 - val_loss: 1.0504 - val_accuracy: 0.5395
Epoch 284/1000
177/177 [==============================] - 0s 85us/step - loss: 0.5598 - accuracy: 0.7797 - val_loss: 1.0247 - val_accuracy: 0.5263
Epoch 285/1000
177/177 [==============================] - 0s 100us/step - loss: 0.5376 - accuracy: 0.7853 - val_loss: 

...

Epoch 725/1000
177/177 [==============================] - 0s 52us/step - loss: 0.3652 - accuracy: 0.8983 - val_loss: 1.3368 - val_accuracy: 0.5263
Epoch 726/1000
177/177 [==============================] - 0s 62us/step - loss: 0.3413 - accuracy: 0.8983 - val_loss: 1.3641 - val_accuracy: 0.5526
Epoch 727/1000
177/177 [==============================] - 0s 55us/step - loss: 0.3608 - accuracy: 0.8814 - val_loss: 1.3859 - val_accuracy: 0.5526
Epoch 728/1000
177/177 [==============================] - 0s 52us/step - loss: 0.3553 - accuracy: 0.8757 - val_loss: 1.3594 - val_accuracy: 0.5395
Epoch 729/1000
177/177 [==============================] - 0s 53us/step - loss: 0.3479 - accuracy: 0.8814 - val_loss: 1.3325 - val_accuracy: 0.5395
Epoch 730/1000
177/177 [==============================] - 0s 56us/step - loss: 0.3445 - accuracy: 0.8927 - val_loss: 1.3489 - val_accuracy: 0.5395
Epoch 731/1000
177/177 [==============================] - 0s 58us/step - loss: 0.3390 - accuracy: 0.8870 - val_loss: 1

Epoch 781/1000
177/177 [==============================] - 0s 42us/step - loss: 0.4156 - accuracy: 0.8757 - val_loss: 1.4143 - val_accuracy: 0.5789
Epoch 782/1000
177/177 [==============================] - 0s 42us/step - loss: 0.3672 - accuracy: 0.8757 - val_loss: 1.4513 - val_accuracy: 0.5789
Epoch 783/1000
177/177 [==============================] - 0s 41us/step - loss: 0.3999 - accuracy: 0.8588 - val_loss: 1.4105 - val_accuracy: 0.5526
Epoch 784/1000
177/177 [==============================] - 0s 53us/step - loss: 0.3495 - accuracy: 0.8814 - val_loss: 1.4226 - val_accuracy: 0.5395
Epoch 785/1000
177/177 [==============================] - 0s 46us/step - loss: 0.3638 - accuracy: 0.8814 - val_loss: 1.4354 - val_accuracy: 0.5395
Epoch 786/1000
177/177 [==============================] - 0s 42us/step - loss: 0.3708 - accuracy: 0.8757 - val_loss: 1.4558 - val_accuracy: 0.5658
Epoch 787/1000
177/177 [==============================] - 0s 49us/step - loss: 0.3478 - accuracy: 0.8757 - val_loss: 1

Epoch 837/1000
177/177 [==============================] - 0s 73us/step - loss: 0.3321 - accuracy: 0.8757 - val_loss: 1.4576 - val_accuracy: 0.5658
Epoch 838/1000
177/177 [==============================] - 0s 78us/step - loss: 0.3421 - accuracy: 0.8701 - val_loss: 1.4580 - val_accuracy: 0.5658
Epoch 839/1000
177/177 [==============================] - 0s 87us/step - loss: 0.3338 - accuracy: 0.8757 - val_loss: 1.4544 - val_accuracy: 0.5658
Epoch 840/1000
177/177 [==============================] - 0s 67us/step - loss: 0.3392 - accuracy: 0.8870 - val_loss: 1.4667 - val_accuracy: 0.5526
Epoch 841/1000
177/177 [==============================] - 0s 75us/step - loss: 0.3179 - accuracy: 0.8983 - val_loss: 1.4973 - val_accuracy: 0.5658
Epoch 842/1000
177/177 [==============================] - 0s 77us/step - loss: 0.3601 - accuracy: 0.8870 - val_loss: 1.4969 - val_accuracy: 0.5658
Epoch 843/1000
177/177 [==============================] - 0s 71us/step - loss: 0.3278 - accuracy: 0.8927 - val_loss: 1

Epoch 893/1000
177/177 [==============================] - 0s 59us/step - loss: 0.3729 - accuracy: 0.8814 - val_loss: 1.4982 - val_accuracy: 0.5658
Epoch 894/1000
177/177 [==============================] - 0s 62us/step - loss: 0.3437 - accuracy: 0.8814 - val_loss: 1.5606 - val_accuracy: 0.5658
Epoch 895/1000
177/177 [==============================] - 0s 54us/step - loss: 0.3530 - accuracy: 0.8757 - val_loss: 1.5765 - val_accuracy: 0.5395
Epoch 896/1000
177/177 [==============================] - 0s 60us/step - loss: 0.4024 - accuracy: 0.8588 - val_loss: 1.5528 - val_accuracy: 0.5263
Epoch 897/1000
177/177 [==============================] - 0s 59us/step - loss: 0.3501 - accuracy: 0.8870 - val_loss: 1.5306 - val_accuracy: 0.5526
Epoch 898/1000
177/177 [==============================] - 0s 54us/step - loss: 0.3433 - accuracy: 0.8701 - val_loss: 1.5154 - val_accuracy: 0.5526
Epoch 899/1000
177/177 [==============================] - 0s 62us/step - loss: 0.3614 - accuracy: 0.8870 - val_loss: 1

Epoch 949/1000
177/177 [==============================] - 0s 89us/step - loss: 0.3158 - accuracy: 0.8814 - val_loss: 1.6050 - val_accuracy: 0.5526
Epoch 950/1000
177/177 [==============================] - 0s 73us/step - loss: 0.3434 - accuracy: 0.8870 - val_loss: 1.5981 - val_accuracy: 0.5395
Epoch 951/1000
177/177 [==============================] - 0s 62us/step - loss: 0.3285 - accuracy: 0.8814 - val_loss: 1.5449 - val_accuracy: 0.5395
Epoch 952/1000
177/177 [==============================] - 0s 72us/step - loss: 0.3513 - accuracy: 0.8870 - val_loss: 1.5337 - val_accuracy: 0.5526
Epoch 953/1000
177/177 [==============================] - 0s 72us/step - loss: 0.3462 - accuracy: 0.8814 - val_loss: 1.5921 - val_accuracy: 0.5526
Epoch 954/1000
177/177 [==============================] - 0s 65us/step - loss: 0.3399 - accuracy: 0.8814 - val_loss: 1.5770 - val_accuracy: 0.5526
Epoch 955/1000
177/177 [==============================] - 0s 55us/step - loss: 0.3177 - accuracy: 0.8870 - val_loss: 1

In [46]:
acc_test1 = model1.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test1*100))

76/76 [==============================] - 0s 121us/step
test accuracy: 56.58%


In [20]:
pred = model1.predict_classes(X_test)
pred

array([2, 0, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 1, 2, 2, 1, 2, 0, 2, 2,
       2, 1, 1, 1, 2, 2, 2, 1, 0, 1, 2, 2, 1, 0, 1, 2, 1, 2, 2, 2, 1, 1,
       2, 2, 1, 1, 0, 2, 0, 1, 2, 2, 1, 0, 2, 2, 2, 2, 2, 1, 0, 1, 2, 0,
       0, 0, 1, 2, 1, 2, 0, 0, 2, 0])

In [21]:
dat['pred'] = pred
dat

,id,test,pred
221,NRS266,2,2
177,NRS209,1,0
223,NRS272,2,1
49,CFBREBSa114,2,2
53,CFBREBSa119,1,2
...,...,...,...
162,NRS187,1,2
237,SR1287,0,0
84,CFBRSa50,0,0
168,NRS196,2,2


In [22]:
proba1 = model1.predict_proba(X_test)
dat_proba1 = pd.DataFrame(proba1)

In [23]:
dat_proba1

,0,1,2
0,0.002816,0.159452,0.837731
1,0.569819,0.411174,0.019007
2,0.301769,0.591542,0.106688
3,0.259023,0.019398,0.721579
4,0.049656,0.004542,0.945802
...,...,...,...
71,0.052303,0.132281,0.815416
72,0.988652,0.000037,0.011311
73,0.721587,0.119755,0.158658
74,0.000036,0.015767,0.984197


In [24]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [25]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p40ST.csv", index = False,
         header=None)

In [50]:
hist1 = model1.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 98us/step - loss: 0.3848 - accuracy: 0.8644 - val_loss: 1.4621 - val_accuracy: 0.5658
Epoch 2/1000
177/177 [==============================] - 0s 77us/step - loss: 0.3924 - accuracy: 0.8588 - val_loss: 1.5177 - val_accuracy: 0.5526
Epoch 3/1000
177/177 [==============================] - 0s 94us/step - loss: 0.3542 - accuracy: 0.8644 - val_loss: 1.4907 - val_accuracy: 0.5658
Epoch 4/1000
177/177 [==============================] - 0s 110us/step - loss: 0.3648 - accuracy: 0.8644 - val_loss: 1.5074 - val_accuracy: 0.5658
Epoch 5/1000
177/177 [==============================] - 0s 90us/step - loss: 0.3496 - accuracy: 0.8644 - val_loss: 1.5171 - val_accuracy: 0.5526
Epoch 6/1000
177/177 [==============================] - 0s 103us/step - loss: 0.3548 - accuracy: 0.8475 - val_loss: 1.5107 - val_accuracy: 0.5395
Epoch 7/1000
177/177 [==============================] - 0s 78us/step - loss: 0.3499

Epoch 112/1000
177/177 [==============================] - 0s 92us/step - loss: 0.3643 - accuracy: 0.8644 - val_loss: 1.6337 - val_accuracy: 0.5526
Epoch 113/1000
177/177 [==============================] - 0s 148us/step - loss: 0.3669 - accuracy: 0.8531 - val_loss: 1.6882 - val_accuracy: 0.5395
Epoch 114/1000
177/177 [==============================] - 0s 120us/step - loss: 0.3719 - accuracy: 0.8588 - val_loss: 1.6341 - val_accuracy: 0.5395
Epoch 115/1000
177/177 [==============================] - 0s 150us/step - loss: 0.4073 - accuracy: 0.8531 - val_loss: 1.6183 - val_accuracy: 0.5526
Epoch 116/1000
177/177 [==============================] - 0s 132us/step - loss: 0.3907 - accuracy: 0.8531 - val_loss: 1.6668 - val_accuracy: 0.5526
Epoch 117/1000
177/177 [==============================] - 0s 133us/step - loss: 0.3550 - accuracy: 0.8531 - val_loss: 1.5989 - val_accuracy: 0.5658
Epoch 118/1000
177/177 [==============================] - 0s 150us/step - loss: 0.3881 - accuracy: 0.8757 - val_l

Epoch 223/1000
177/177 [==============================] - 0s 109us/step - loss: 0.3174 - accuracy: 0.8701 - val_loss: 1.6974 - val_accuracy: 0.5395
Epoch 224/1000
177/177 [==============================] - 0s 97us/step - loss: 0.3503 - accuracy: 0.8644 - val_loss: 1.7117 - val_accuracy: 0.5395
Epoch 225/1000
177/177 [==============================] - 0s 114us/step - loss: 0.3449 - accuracy: 0.8644 - val_loss: 1.7146 - val_accuracy: 0.5263
Epoch 226/1000
177/177 [==============================] - 0s 113us/step - loss: 0.3373 - accuracy: 0.8644 - val_loss: 1.6423 - val_accuracy: 0.5526
Epoch 227/1000
177/177 [==============================] - 0s 84us/step - loss: 0.3960 - accuracy: 0.8531 - val_loss: 1.7266 - val_accuracy: 0.5395
Epoch 228/1000
177/177 [==============================] - 0s 94us/step - loss: 0.3438 - accuracy: 0.8701 - val_loss: 1.8458 - val_accuracy: 0.5395
Epoch 229/1000
177/177 [==============================] - 0s 86us/step - loss: 0.4317 - accuracy: 0.8588 - val_loss

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.102878). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


177/177 [==============================] - 0s 154us/step - loss: 0.3398 - accuracy: 0.8588 - val_loss: 1.7304 - val_accuracy: 0.5263
Epoch 289/1000
177/177 [==============================] - 0s 146us/step - loss: 0.3783 - accuracy: 0.8644 - val_loss: 1.6719 - val_accuracy: 0.5263
Epoch 290/1000
177/177 [==============================] - 0s 123us/step - loss: 0.4184 - accuracy: 0.8588 - val_loss: 1.7060 - val_accuracy: 0.5395
Epoch 291/1000
177/177 [==============================] - 0s 134us/step - loss: 0.3177 - accuracy: 0.8588 - val_loss: 1.8364 - val_accuracy: 0.5395
Epoch 292/1000
177/177 [==============================] - 0s 165us/step - loss: 0.3796 - accuracy: 0.8588 - val_loss: 1.8143 - val_accuracy: 0.5395
Epoch 293/1000
177/177 [==============================] - 0s 155us/step - loss: 0.3480 - accuracy: 0.8644 - val_loss: 1.7118 - val_accuracy: 0.5263
Epoch 294/1000
177/177 [==============================] - 0s 236us/step - loss: 0.3496 - accuracy: 0.8644 - val_loss: 1.7328 - 

...

177/177 [==============================] - 0s 292us/step - loss: 0.3527 - accuracy: 0.8701 - val_loss: 1.9043 - val_accuracy: 0.5263
Epoch 509/1000
177/177 [==============================] - 0s 181us/step - loss: 0.3716 - accuracy: 0.8701 - val_loss: 1.9998 - val_accuracy: 0.5000
Epoch 510/1000
177/177 [==============================] - 0s 145us/step - loss: 0.3783 - accuracy: 0.8531 - val_loss: 1.8769 - val_accuracy: 0.5263
Epoch 511/1000
177/177 [==============================] - 0s 124us/step - loss: 0.3433 - accuracy: 0.8644 - val_loss: 1.8938 - val_accuracy: 0.5263
Epoch 512/1000
177/177 [==============================] - 0s 146us/step - loss: 0.3552 - accuracy: 0.8588 - val_loss: 1.9744 - val_accuracy: 0.5263
Epoch 513/1000
177/177 [==============================] - 0s 97us/step - loss: 0.3666 - accuracy: 0.8531 - val_loss: 2.0035 - val_accuracy: 0.4737
Epoch 514/1000
177/177 [==============================] - 0s 81us/step - loss: 0.3927 - accuracy: 0.8475 - val_loss: 1.9202 - va

Epoch 619/1000
177/177 [==============================] - 0s 61us/step - loss: 0.3491 - accuracy: 0.8588 - val_loss: 2.0411 - val_accuracy: 0.5263
Epoch 620/1000
177/177 [==============================] - 0s 76us/step - loss: 0.3187 - accuracy: 0.8701 - val_loss: 1.9546 - val_accuracy: 0.5263
Epoch 621/1000
177/177 [==============================] - 0s 78us/step - loss: 0.3013 - accuracy: 0.8870 - val_loss: 1.9105 - val_accuracy: 0.5263
Epoch 622/1000
177/177 [==============================] - 0s 68us/step - loss: 0.3591 - accuracy: 0.8814 - val_loss: 1.9461 - val_accuracy: 0.5263
Epoch 623/1000
177/177 [==============================] - 0s 167us/step - loss: 0.3489 - accuracy: 0.8531 - val_loss: 2.0839 - val_accuracy: 0.5263
Epoch 624/1000
177/177 [==============================] - 0s 234us/step - loss: 0.4042 - accuracy: 0.8701 - val_loss: 2.0712 - val_accuracy: 0.5263
Epoch 625/1000
177/177 [==============================] - 0s 145us/step - loss: 0.3411 - accuracy: 0.8757 - val_loss

177/177 [==============================] - 0s 82us/step - loss: 0.2947 - accuracy: 0.8870 - val_loss: 2.0037 - val_accuracy: 0.5263
Epoch 730/1000
177/177 [==============================] - 0s 161us/step - loss: 0.2854 - accuracy: 0.8870 - val_loss: 2.0062 - val_accuracy: 0.5263
Epoch 731/1000
177/177 [==============================] - 0s 151us/step - loss: 0.2829 - accuracy: 0.8814 - val_loss: 2.0030 - val_accuracy: 0.5263
Epoch 732/1000
177/177 [==============================] - 0s 174us/step - loss: 0.2847 - accuracy: 0.8701 - val_loss: 2.0337 - val_accuracy: 0.5132
Epoch 733/1000
177/177 [==============================] - 0s 236us/step - loss: 0.2902 - accuracy: 0.8644 - val_loss: 1.9902 - val_accuracy: 0.5132
Epoch 734/1000
177/177 [==============================] - 0s 514us/step - loss: 0.3147 - accuracy: 0.8814 - val_loss: 2.0031 - val_accuracy: 0.5132
Epoch 735/1000
177/177 [==============================] - 0s 168us/step - loss: 0.3155 - accuracy: 0.8644 - val_loss: 2.0497 - v

177/177 [==============================] - 0s 96us/step - loss: 0.3016 - accuracy: 0.8757 - val_loss: 2.0020 - val_accuracy: 0.5263
Epoch 840/1000
177/177 [==============================] - 0s 141us/step - loss: 0.3750 - accuracy: 0.8644 - val_loss: 2.0230 - val_accuracy: 0.5395
Epoch 841/1000
177/177 [==============================] - 0s 255us/step - loss: 0.3038 - accuracy: 0.8814 - val_loss: 2.1207 - val_accuracy: 0.5395
Epoch 842/1000
177/177 [==============================] - 0s 141us/step - loss: 0.3866 - accuracy: 0.8701 - val_loss: 2.1623 - val_accuracy: 0.5395
Epoch 843/1000
177/177 [==============================] - 0s 144us/step - loss: 0.3807 - accuracy: 0.8531 - val_loss: 2.0410 - val_accuracy: 0.5395
Epoch 844/1000
177/177 [==============================] - 0s 251us/step - loss: 0.3118 - accuracy: 0.8757 - val_loss: 2.0733 - val_accuracy: 0.5395
Epoch 845/1000
177/177 [==============================] - 0s 228us/step - loss: 0.2785 - accuracy: 0.8814 - val_loss: 2.1019 - v

177/177 [==============================] - 0s 176us/step - loss: 0.3090 - accuracy: 0.8870 - val_loss: 2.1543 - val_accuracy: 0.5395
Epoch 950/1000
177/177 [==============================] - 0s 128us/step - loss: 0.2950 - accuracy: 0.8757 - val_loss: 2.1587 - val_accuracy: 0.5395
Epoch 951/1000
177/177 [==============================] - 0s 96us/step - loss: 0.2983 - accuracy: 0.8814 - val_loss: 2.1289 - val_accuracy: 0.5263
Epoch 952/1000
177/177 [==============================] - 0s 181us/step - loss: 0.2997 - accuracy: 0.8701 - val_loss: 2.1764 - val_accuracy: 0.5263
Epoch 953/1000
177/177 [==============================] - 0s 143us/step - loss: 0.3089 - accuracy: 0.8814 - val_loss: 2.1070 - val_accuracy: 0.5395
Epoch 954/1000
177/177 [==============================] - 0s 85us/step - loss: 0.3096 - accuracy: 0.8927 - val_loss: 2.1128 - val_accuracy: 0.5395
Epoch 955/1000
177/177 [==============================] - 0s 70us/step - loss: 0.2758 - accuracy: 0.8870 - val_loss: 2.1994 - val

In [51]:
print('train accuracy: %.2f%%' % (np.mean(hist1.history['accuracy'])*100))

train accuracy: 86.68%


In [17]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [18]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS027,1,0,0.759813,0.001304,0.238883
1,p0017kpresabs_qual,NRS246,1,0,0.947819,0.009591,0.042591
2,p0017kpresabs_qual,NRS218,2,1,0.008693,0.989390,0.001916
3,p0017kpresabs_qual,CFBRSa70,2,0,0.813774,0.001482,0.184744
4,p0017kpresabs_qual,NRS177,1,1,0.000916,0.998926,0.000157
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS187,1,1,0.216843,0.568123,0.215034
604,p0040presabsSTCC_qual,SR1287,0,1,0.418802,0.553160,0.028038
605,p0040presabsSTCC_qual,CFBRSa50,0,0,0.948100,0.035031,0.016869
606,p0040presabsSTCC_qual,NRS196,2,2,0.000964,0.039095,0.959940


In [19]:
y_prob = df_proba[df_proba['phage']=='p0040kpresabsSTCC_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[2.81645240e-03, 1.59452070e-01, 8.37731500e-01],
       [5.69818740e-01, 4.11174000e-01, 1.90072710e-02],
       [3.01769170e-01, 5.91542300e-01, 1.06688500e-01],
       [2.59023250e-01, 1.93975870e-02, 7.21579130e-01],
       [4.96556300e-02, 4.54200100e-03, 9.45802400e-01],
       [9.38257900e-02, 1.07720910e-02, 8.95402130e-01],
       [9.11001100e-02, 1.41726480e-01, 7.67173350e-01],
       [9.02826900e-01, 8.34461400e-02, 1.37269330e-02],
       [1.25755790e-02, 1.00481110e-01, 8.86943340e-01],
       [2.37851020e-01, 1.74015370e-01, 5.88133630e-01],
       [3.17368570e-01, 1.79021540e-01, 5.03609900e-01],
       [7.00317980e-03, 3.82765430e-04, 9.92614030e-01],
       [8.84013530e-01, 9.66054600e-03, 1.06325895e-01],
       [4.92152870e-01, 2.59962920e-02, 4.81850860e-01],
       [6.61622640e-03, 9.82560800e-01, 1.08229240e-02],
       [6.61455300e-02, 2.03917530e-02, 9.13462760e-01],
       [3.49417070e-03, 9.15959700e-03, 9.87346230e-01],
       [6.05872130e-03, 9.37172

In [20]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [21]:
ovo1 = rocauc_ovo(y_test, y_prob, average="macro", multi_class="ovo")
ovo1

0.659555648248932

In [22]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [23]:
ovr1 = rocauc_ovr(y_test, y_prob, average="macro", multi_class="ovr")
ovr1

0.659555648248932

In [24]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y)

In [25]:
dat2 = pd.DataFrame(X_test.iloc[:,0])
dat2['test'] = y_test

In [26]:
dat2

,id,test
165,NRS191,2
237,SR1287,0
243,SR3569,2
128,NRS100,0
107,NRS001,1
...,...,...
220,NRS265,1
233,NY439,2
68,CFBRSa05,0
175,NRS205,2


In [27]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [28]:
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [31]:
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model2.fit(X_train, y_train,
          batch_size=16, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 910us/step - loss: 8.2168 - accuracy: 0.4350 - val_loss: 8.8452 - val_accuracy: 0.4079
Epoch 2/1000
177/177 [==============================] - 0s 224us/step - loss: 4.2416 - accuracy: 0.4294 - val_loss: 4.7402 - val_accuracy: 0.3947
Epoch 3/1000
177/177 [==============================] - 0s 255us/step - loss: 3.3104 - accuracy: 0.4011 - val_loss: 3.7277 - val_accuracy: 0.3816
Epoch 4/1000
177/177 [==============================] - 0s 320us/step - loss: 1.5602 - accuracy: 0.4294 - val_loss: 1.2560 - val_accuracy: 0.4342
Epoch 5/1000
177/177 [==============================] - 0s 272us/step - loss: 1.3900 - accuracy: 0.4520 - val_loss: 2.2655 - val_accuracy: 0.4342
Epoch 6/1000
177/177 [==============================] - 0s 233us/step - loss: 1.6282 - accuracy: 0.4350 - val_loss: 1.6731 - val_accuracy: 0.3816
Epoch 7/1000
177/177 [==============================] - 0s 239us/step - loss: 1

Epoch 112/1000
177/177 [==============================] - 0s 235us/step - loss: 0.6912 - accuracy: 0.6836 - val_loss: 1.1607 - val_accuracy: 0.4605
Epoch 113/1000
177/177 [==============================] - 0s 194us/step - loss: 0.6681 - accuracy: 0.7006 - val_loss: 1.1400 - val_accuracy: 0.4342
Epoch 114/1000
177/177 [==============================] - 0s 168us/step - loss: 0.7166 - accuracy: 0.6723 - val_loss: 1.2941 - val_accuracy: 0.4605
Epoch 115/1000
177/177 [==============================] - 0s 197us/step - loss: 0.7646 - accuracy: 0.6780 - val_loss: 1.8694 - val_accuracy: 0.4737
Epoch 116/1000
177/177 [==============================] - 0s 147us/step - loss: 2.3317 - accuracy: 0.6554 - val_loss: 3.6423 - val_accuracy: 0.4737
Epoch 117/1000
177/177 [==============================] - 0s 164us/step - loss: 1.6389 - accuracy: 0.6610 - val_loss: 1.7531 - val_accuracy: 0.5000
Epoch 118/1000
177/177 [==============================] - 0s 153us/step - loss: 0.9023 - accuracy: 0.6610 - val_

177/177 [==============================] - 0s 162us/step - loss: 0.6040 - accuracy: 0.7401 - val_loss: 1.3907 - val_accuracy: 0.5000
Epoch 223/1000
177/177 [==============================] - 0s 154us/step - loss: 0.5601 - accuracy: 0.7571 - val_loss: 1.4077 - val_accuracy: 0.5000
Epoch 224/1000
177/177 [==============================] - 0s 138us/step - loss: 0.5608 - accuracy: 0.7514 - val_loss: 1.3739 - val_accuracy: 0.4737
Epoch 225/1000
177/177 [==============================] - 0s 188us/step - loss: 0.5832 - accuracy: 0.7458 - val_loss: 1.4854 - val_accuracy: 0.4737
Epoch 226/1000
177/177 [==============================] - 0s 185us/step - loss: 0.5684 - accuracy: 0.7514 - val_loss: 1.3823 - val_accuracy: 0.5263
Epoch 227/1000
177/177 [==============================] - 0s 162us/step - loss: 0.5492 - accuracy: 0.7514 - val_loss: 1.3292 - val_accuracy: 0.4868
Epoch 228/1000
177/177 [==============================] - 0s 159us/step - loss: 0.5452 - accuracy: 0.7514 - val_loss: 1.3833 - 

177/177 [==============================] - 0s 109us/step - loss: 1.0872 - accuracy: 0.7458 - val_loss: 2.0729 - val_accuracy: 0.4868
Epoch 333/1000
177/177 [==============================] - 0s 106us/step - loss: 0.7752 - accuracy: 0.7062 - val_loss: 2.0429 - val_accuracy: 0.4737
Epoch 334/1000
177/177 [==============================] - 0s 104us/step - loss: 0.5177 - accuracy: 0.7797 - val_loss: 1.6394 - val_accuracy: 0.4868
Epoch 335/1000
177/177 [==============================] - 0s 120us/step - loss: 0.5373 - accuracy: 0.7740 - val_loss: 1.7923 - val_accuracy: 0.5132
Epoch 336/1000
177/177 [==============================] - 0s 175us/step - loss: 0.5167 - accuracy: 0.7797 - val_loss: 1.7146 - val_accuracy: 0.4868
Epoch 337/1000
177/177 [==============================] - 0s 221us/step - loss: 0.5142 - accuracy: 0.7910 - val_loss: 1.7575 - val_accuracy: 0.5000
Epoch 338/1000
177/177 [==============================] - 0s 214us/step - loss: 0.5036 - accuracy: 0.7740 - val_loss: 1.8447 - 

177/177 [==============================] - 0s 164us/step - loss: 0.4285 - accuracy: 0.8362 - val_loss: 1.7169 - val_accuracy: 0.4737
Epoch 443/1000
177/177 [==============================] - 0s 153us/step - loss: 0.4253 - accuracy: 0.8192 - val_loss: 1.7934 - val_accuracy: 0.5132
Epoch 444/1000
177/177 [==============================] - 0s 161us/step - loss: 0.4330 - accuracy: 0.8362 - val_loss: 1.7433 - val_accuracy: 0.4868
Epoch 445/1000
177/177 [==============================] - 0s 174us/step - loss: 0.4175 - accuracy: 0.8362 - val_loss: 1.7685 - val_accuracy: 0.5526
Epoch 446/1000
177/177 [==============================] - 0s 146us/step - loss: 0.4331 - accuracy: 0.8362 - val_loss: 1.7818 - val_accuracy: 0.5263
Epoch 447/1000
177/177 [==============================] - 0s 151us/step - loss: 0.6690 - accuracy: 0.7910 - val_loss: 1.9031 - val_accuracy: 0.5132
Epoch 448/1000
177/177 [==============================] - 0s 163us/step - loss: 0.5422 - accuracy: 0.8305 - val_loss: 1.8262 - 

177/177 [==============================] - 0s 150us/step - loss: 0.4384 - accuracy: 0.8362 - val_loss: 1.8366 - val_accuracy: 0.5526
Epoch 553/1000
177/177 [==============================] - 0s 156us/step - loss: 0.4375 - accuracy: 0.8249 - val_loss: 1.8787 - val_accuracy: 0.5263
Epoch 554/1000
177/177 [==============================] - 0s 153us/step - loss: 0.4842 - accuracy: 0.8023 - val_loss: 1.7969 - val_accuracy: 0.5132
Epoch 555/1000
177/177 [==============================] - 0s 154us/step - loss: 0.4682 - accuracy: 0.8192 - val_loss: 2.2540 - val_accuracy: 0.5000
Epoch 556/1000
177/177 [==============================] - 0s 207us/step - loss: 0.5572 - accuracy: 0.7966 - val_loss: 1.8583 - val_accuracy: 0.5132
Epoch 557/1000
177/177 [==============================] - 0s 151us/step - loss: 0.4755 - accuracy: 0.8136 - val_loss: 2.2613 - val_accuracy: 0.5000
Epoch 558/1000
177/177 [==============================] - 0s 156us/step - loss: 0.4464 - accuracy: 0.7966 - val_loss: 1.8638 - 

177/177 [==============================] - 0s 148us/step - loss: 0.4061 - accuracy: 0.8136 - val_loss: 2.1643 - val_accuracy: 0.4868
Epoch 663/1000
177/177 [==============================] - 0s 174us/step - loss: 0.3888 - accuracy: 0.8362 - val_loss: 2.0645 - val_accuracy: 0.4868
Epoch 664/1000
177/177 [==============================] - 0s 162us/step - loss: 0.3715 - accuracy: 0.8362 - val_loss: 2.0012 - val_accuracy: 0.4868
Epoch 665/1000
177/177 [==============================] - 0s 250us/step - loss: 0.3658 - accuracy: 0.8305 - val_loss: 2.1388 - val_accuracy: 0.5132
Epoch 666/1000
177/177 [==============================] - 0s 173us/step - loss: 0.4113 - accuracy: 0.8418 - val_loss: 2.2584 - val_accuracy: 0.5000
Epoch 667/1000
177/177 [==============================] - 0s 177us/step - loss: 0.4266 - accuracy: 0.8249 - val_loss: 2.1254 - val_accuracy: 0.5132
Epoch 668/1000
177/177 [==============================] - 0s 156us/step - loss: 0.4566 - accuracy: 0.8192 - val_loss: 2.5520 - 

177/177 [==============================] - 0s 156us/step - loss: 0.3616 - accuracy: 0.8475 - val_loss: 2.2657 - val_accuracy: 0.5132
Epoch 773/1000
177/177 [==============================] - 0s 157us/step - loss: 0.3477 - accuracy: 0.8475 - val_loss: 2.3133 - val_accuracy: 0.5132
Epoch 774/1000
177/177 [==============================] - 0s 151us/step - loss: 0.3641 - accuracy: 0.8588 - val_loss: 2.3058 - val_accuracy: 0.5132
Epoch 775/1000
177/177 [==============================] - 0s 158us/step - loss: 0.3771 - accuracy: 0.8418 - val_loss: 2.4121 - val_accuracy: 0.5000
Epoch 776/1000
177/177 [==============================] - 0s 146us/step - loss: 0.3980 - accuracy: 0.8644 - val_loss: 2.2870 - val_accuracy: 0.5000
Epoch 777/1000
177/177 [==============================] - 0s 154us/step - loss: 0.3457 - accuracy: 0.8644 - val_loss: 2.1811 - val_accuracy: 0.4868
Epoch 778/1000
177/177 [==============================] - 0s 149us/step - loss: 0.3418 - accuracy: 0.8475 - val_loss: 2.1849 - 

177/177 [==============================] - 0s 166us/step - loss: 0.3518 - accuracy: 0.8588 - val_loss: 2.3485 - val_accuracy: 0.5263
Epoch 883/1000
177/177 [==============================] - 0s 172us/step - loss: 0.3576 - accuracy: 0.8644 - val_loss: 2.2596 - val_accuracy: 0.5000
Epoch 884/1000
177/177 [==============================] - 0s 152us/step - loss: 0.3471 - accuracy: 0.8418 - val_loss: 2.4579 - val_accuracy: 0.4868
Epoch 885/1000
177/177 [==============================] - 0s 151us/step - loss: 0.3539 - accuracy: 0.8362 - val_loss: 2.4815 - val_accuracy: 0.4868
Epoch 886/1000
177/177 [==============================] - 0s 157us/step - loss: 0.3486 - accuracy: 0.8362 - val_loss: 2.4419 - val_accuracy: 0.5395
Epoch 887/1000
177/177 [==============================] - 0s 210us/step - loss: 0.3702 - accuracy: 0.8531 - val_loss: 3.0084 - val_accuracy: 0.5263
Epoch 888/1000
177/177 [==============================] - 0s 156us/step - loss: 0.5689 - accuracy: 0.8079 - val_loss: 2.3922 - 

177/177 [==============================] - 0s 159us/step - loss: 0.3444 - accuracy: 0.8305 - val_loss: 2.4124 - val_accuracy: 0.5000
Epoch 993/1000
177/177 [==============================] - 0s 169us/step - loss: 0.3192 - accuracy: 0.8475 - val_loss: 2.7351 - val_accuracy: 0.4868
Epoch 994/1000
177/177 [==============================] - 0s 162us/step - loss: 0.3557 - accuracy: 0.8362 - val_loss: 2.3562 - val_accuracy: 0.4868
Epoch 995/1000
177/177 [==============================] - 0s 159us/step - loss: 0.3557 - accuracy: 0.8305 - val_loss: 2.6239 - val_accuracy: 0.4605
Epoch 996/1000
177/177 [==============================] - 0s 152us/step - loss: 0.3494 - accuracy: 0.8362 - val_loss: 2.6020 - val_accuracy: 0.5132
Epoch 997/1000
177/177 [==============================] - 0s 165us/step - loss: 0.3166 - accuracy: 0.8644 - val_loss: 2.5471 - val_accuracy: 0.5263
Epoch 998/1000
177/177 [==============================] - 0s 186us/step - loss: 0.3073 - accuracy: 0.8531 - val_loss: 2.6935 - 

In [67]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

76/76 [==============================] - 0s 175us/step
test accuracy: 52.63%


In [33]:
pred2 = model2.predict_classes(X_test)
pred2

array([2, 0, 2, 1, 0, 1, 2, 2, 2, 1, 2, 1, 2, 2, 0, 1, 2, 2, 2, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 2, 0, 2, 2, 2, 2, 1, 1, 2, 0, 0, 2, 0, 2, 2, 1,
       1, 1, 2, 1, 2, 2, 0, 2, 1, 1, 1, 1, 1, 0, 1, 1, 2, 2, 2, 0, 1, 0,
       1, 0, 2, 0, 0, 2, 0, 0, 2, 2])

In [34]:
dat2['pred'] = pred2
dat2

,id,test,pred
165,NRS191,2,2
237,SR1287,0,0
243,SR3569,2,2
128,NRS100,0,1
107,NRS001,1,0
...,...,...,...
220,NRS265,1,2
233,NY439,2,0
68,CFBRSa05,0,0
175,NRS205,2,2


In [35]:
proba2 = model2.predict_proba(X_test)
dat_proba2 = pd.DataFrame(proba2)

In [36]:
dat_proba2

,0,1,2
0,0.018425,0.007282,0.974294
1,0.954937,0.040994,0.004069
2,0.135369,0.005344,0.859287
3,0.287932,0.665677,0.046391
4,0.964073,0.034310,0.001617
...,...,...,...
71,0.000494,0.342701,0.656805
72,0.856553,0.010285,0.133162
73,0.474452,0.433111,0.092436
74,0.001079,0.007076,0.991845


In [37]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [38]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p40ST.csv", index = False,
         header=None)

In [71]:
hist2 = model2.fit(X_train, y_train,
          batch_size=16, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 198us/step - loss: 0.2873 - accuracy: 0.8814 - val_loss: 2.4048 - val_accuracy: 0.5263
Epoch 2/1000
177/177 [==============================] - 0s 295us/step - loss: 0.2660 - accuracy: 0.8870 - val_loss: 2.3171 - val_accuracy: 0.5263
Epoch 3/1000
177/177 [==============================] - 0s 184us/step - loss: 0.2608 - accuracy: 0.8870 - val_loss: 2.3364 - val_accuracy: 0.5395
Epoch 4/1000
177/177 [==============================] - 0s 175us/step - loss: 0.2633 - accuracy: 0.8870 - val_loss: 2.3257 - val_accuracy: 0.5263
Epoch 5/1000
177/177 [==============================] - 0s 186us/step - loss: 0.2722 - accuracy: 0.8870 - val_loss: 2.2985 - val_accuracy: 0.5132
Epoch 6/1000
177/177 [==============================] - 0s 182us/step - loss: 0.8067 - accuracy: 0.8475 - val_loss: 2.7937 - val_accuracy: 0.5132
Epoch 7/1000
177/177 [==============================] - 0s 200us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 251us/step - loss: 0.2564 - accuracy: 0.8983 - val_loss: 2.4807 - val_accuracy: 0.5132
Epoch 113/1000
177/177 [==============================] - 0s 233us/step - loss: 0.2536 - accuracy: 0.9040 - val_loss: 2.4450 - val_accuracy: 0.5132
Epoch 114/1000
177/177 [==============================] - 0s 340us/step - loss: 0.2559 - accuracy: 0.8927 - val_loss: 2.4705 - val_accuracy: 0.5132
Epoch 115/1000
177/177 [==============================] - 0s 278us/step - loss: 0.2576 - accuracy: 0.9040 - val_loss: 2.3970 - val_accuracy: 0.5132
Epoch 116/1000
177/177 [==============================] - 0s 361us/step - loss: 0.2541 - accuracy: 0.8927 - val_loss: 2.6020 - val_accuracy: 0.5263
Epoch 117/1000
177/177 [==============================] - 0s 208us/step - loss: 0.2617 - accuracy: 0.8814 - val_loss: 2.3473 - val_accuracy: 0.5395
Epoch 118/1000
177/177 [==============================] - 0s 323us/step - loss: 0.2643 - accuracy: 0.8814 - val_

177/177 [==============================] - 0s 223us/step - loss: 0.2527 - accuracy: 0.8983 - val_loss: 2.6646 - val_accuracy: 0.5526
Epoch 223/1000
177/177 [==============================] - 0s 266us/step - loss: 0.2471 - accuracy: 0.8814 - val_loss: 2.5973 - val_accuracy: 0.5263
Epoch 224/1000
177/177 [==============================] - 0s 209us/step - loss: 0.2485 - accuracy: 0.8983 - val_loss: 2.7662 - val_accuracy: 0.5263
Epoch 225/1000
177/177 [==============================] - 0s 227us/step - loss: 0.2477 - accuracy: 0.9096 - val_loss: 2.6242 - val_accuracy: 0.5526
Epoch 226/1000
177/177 [==============================] - 0s 224us/step - loss: 0.2929 - accuracy: 0.8757 - val_loss: 3.3057 - val_accuracy: 0.5000
Epoch 227/1000
177/177 [==============================] - 0s 244us/step - loss: 0.4308 - accuracy: 0.8757 - val_loss: 2.5848 - val_accuracy: 0.5000
Epoch 228/1000
177/177 [==============================] - 0s 271us/step - loss: 0.3298 - accuracy: 0.8870 - val_loss: 2.9426 - 

177/177 [==============================] - 0s 325us/step - loss: 0.2388 - accuracy: 0.8983 - val_loss: 2.8157 - val_accuracy: 0.5395
Epoch 333/1000
177/177 [==============================] - 0s 245us/step - loss: 0.2337 - accuracy: 0.8927 - val_loss: 2.8910 - val_accuracy: 0.5526
Epoch 334/1000
177/177 [==============================] - 0s 256us/step - loss: 0.2281 - accuracy: 0.8927 - val_loss: 2.8692 - val_accuracy: 0.5263
Epoch 335/1000
177/177 [==============================] - 0s 217us/step - loss: 0.2481 - accuracy: 0.8927 - val_loss: 2.8590 - val_accuracy: 0.5132
Epoch 336/1000
177/177 [==============================] - 0s 243us/step - loss: 0.2793 - accuracy: 0.8927 - val_loss: 2.8307 - val_accuracy: 0.5263
Epoch 337/1000
177/177 [==============================] - 0s 202us/step - loss: 0.2926 - accuracy: 0.8588 - val_loss: 3.3232 - val_accuracy: 0.4868
Epoch 338/1000
177/177 [==============================] - 0s 202us/step - loss: 0.2460 - accuracy: 0.8983 - val_loss: 3.0558 - 

177/177 [==============================] - 0s 177us/step - loss: 0.2283 - accuracy: 0.8983 - val_loss: 3.0422 - val_accuracy: 0.5789
Epoch 443/1000
177/177 [==============================] - 0s 173us/step - loss: 0.2186 - accuracy: 0.9040 - val_loss: 3.0052 - val_accuracy: 0.5526
Epoch 444/1000
177/177 [==============================] - 0s 176us/step - loss: 0.2215 - accuracy: 0.9040 - val_loss: 3.0515 - val_accuracy: 0.5395
Epoch 445/1000
177/177 [==============================] - 0s 153us/step - loss: 0.2296 - accuracy: 0.9096 - val_loss: 3.0735 - val_accuracy: 0.5526
Epoch 446/1000
177/177 [==============================] - 0s 184us/step - loss: 0.2258 - accuracy: 0.9096 - val_loss: 3.0211 - val_accuracy: 0.5658
Epoch 447/1000
177/177 [==============================] - 0s 164us/step - loss: 0.2292 - accuracy: 0.9096 - val_loss: 3.0426 - val_accuracy: 0.5658
Epoch 448/1000
177/177 [==============================] - 0s 172us/step - loss: 0.2263 - accuracy: 0.9096 - val_loss: 3.0217 - 

177/177 [==============================] - 0s 302us/step - loss: 0.4148 - accuracy: 0.8588 - val_loss: 2.9222 - val_accuracy: 0.5263
Epoch 553/1000
177/177 [==============================] - 0s 308us/step - loss: 0.2408 - accuracy: 0.8870 - val_loss: 3.8092 - val_accuracy: 0.4737
Epoch 554/1000
177/177 [==============================] - 0s 308us/step - loss: 0.3612 - accuracy: 0.8870 - val_loss: 3.1248 - val_accuracy: 0.5263
Epoch 555/1000
177/177 [==============================] - 0s 293us/step - loss: 0.2655 - accuracy: 0.8814 - val_loss: 3.2883 - val_accuracy: 0.5395
Epoch 556/1000
177/177 [==============================] - 0s 339us/step - loss: 0.2196 - accuracy: 0.8927 - val_loss: 3.4826 - val_accuracy: 0.5658
Epoch 557/1000
177/177 [==============================] - 0s 306us/step - loss: 0.2323 - accuracy: 0.8870 - val_loss: 3.1950 - val_accuracy: 0.5395
Epoch 558/1000
177/177 [==============================] - 0s 493us/step - loss: 0.2182 - accuracy: 0.8983 - val_loss: 3.2551 - 

Epoch 662/1000
177/177 [==============================] - 0s 296us/step - loss: 0.2048 - accuracy: 0.9040 - val_loss: 3.3327 - val_accuracy: 0.5395
Epoch 663/1000
177/177 [==============================] - 0s 249us/step - loss: 0.2044 - accuracy: 0.9040 - val_loss: 3.3184 - val_accuracy: 0.5658
Epoch 664/1000
177/177 [==============================] - 0s 308us/step - loss: 0.2052 - accuracy: 0.9040 - val_loss: 3.3564 - val_accuracy: 0.5132
Epoch 665/1000
177/177 [==============================] - 0s 370us/step - loss: 0.2032 - accuracy: 0.9096 - val_loss: 3.3550 - val_accuracy: 0.5132
Epoch 666/1000
177/177 [==============================] - 0s 494us/step - loss: 0.2059 - accuracy: 0.8927 - val_loss: 3.3034 - val_accuracy: 0.5263
Epoch 667/1000
177/177 [==============================] - 0s 395us/step - loss: 0.2074 - accuracy: 0.9096 - val_loss: 3.3527 - val_accuracy: 0.5263
Epoch 668/1000
177/177 [==============================] - 0s 512us/step - loss: 0.2033 - accuracy: 0.8927 - val_

177/177 [==============================] - 0s 143us/step - loss: 0.2075 - accuracy: 0.8927 - val_loss: 3.4425 - val_accuracy: 0.5132
Epoch 773/1000
177/177 [==============================] - 0s 162us/step - loss: 0.2083 - accuracy: 0.8983 - val_loss: 3.4582 - val_accuracy: 0.5132
Epoch 774/1000
177/177 [==============================] - 0s 284us/step - loss: 0.2036 - accuracy: 0.9096 - val_loss: 3.5200 - val_accuracy: 0.5132
Epoch 775/1000
177/177 [==============================] - 0s 142us/step - loss: 0.2042 - accuracy: 0.9040 - val_loss: 3.4770 - val_accuracy: 0.5263
Epoch 776/1000
177/177 [==============================] - 0s 134us/step - loss: 0.2252 - accuracy: 0.8870 - val_loss: 3.3132 - val_accuracy: 0.5395
Epoch 777/1000
177/177 [==============================] - 0s 139us/step - loss: 0.2314 - accuracy: 0.9096 - val_loss: 3.4008 - val_accuracy: 0.5395
Epoch 778/1000
177/177 [==============================] - 0s 132us/step - loss: 0.2260 - accuracy: 0.8983 - val_loss: 3.4586 - 

177/177 [==============================] - 0s 152us/step - loss: 0.2063 - accuracy: 0.9040 - val_loss: 3.5940 - val_accuracy: 0.5263
Epoch 883/1000
177/177 [==============================] - 0s 153us/step - loss: 0.2095 - accuracy: 0.8927 - val_loss: 3.6016 - val_accuracy: 0.5000
Epoch 884/1000
177/177 [==============================] - 0s 159us/step - loss: 0.2202 - accuracy: 0.8927 - val_loss: 3.6074 - val_accuracy: 0.5000
Epoch 885/1000
177/177 [==============================] - 0s 152us/step - loss: 0.2051 - accuracy: 0.9040 - val_loss: 3.5236 - val_accuracy: 0.4868
Epoch 886/1000
177/177 [==============================] - 0s 144us/step - loss: 0.2057 - accuracy: 0.8870 - val_loss: 3.7583 - val_accuracy: 0.5000
Epoch 887/1000
177/177 [==============================] - 0s 149us/step - loss: 0.2232 - accuracy: 0.8983 - val_loss: 3.9540 - val_accuracy: 0.5000
Epoch 888/1000
177/177 [==============================] - 0s 151us/step - loss: 0.2978 - accuracy: 0.8870 - val_loss: 3.9345 - 

177/177 [==============================] - 0s 167us/step - loss: 0.2579 - accuracy: 0.8983 - val_loss: 3.6699 - val_accuracy: 0.5395
Epoch 993/1000
177/177 [==============================] - 0s 186us/step - loss: 0.2284 - accuracy: 0.9040 - val_loss: 3.7568 - val_accuracy: 0.5526
Epoch 994/1000
177/177 [==============================] - 0s 170us/step - loss: 0.2034 - accuracy: 0.8983 - val_loss: 3.6668 - val_accuracy: 0.5395
Epoch 995/1000
177/177 [==============================] - 0s 152us/step - loss: 0.1968 - accuracy: 0.9096 - val_loss: 3.6807 - val_accuracy: 0.5263
Epoch 996/1000
177/177 [==============================] - 0s 153us/step - loss: 0.2088 - accuracy: 0.9040 - val_loss: 3.5267 - val_accuracy: 0.5263
Epoch 997/1000
177/177 [==============================] - 0s 158us/step - loss: 0.2232 - accuracy: 0.8983 - val_loss: 3.6515 - val_accuracy: 0.5263
Epoch 998/1000
177/177 [==============================] - 0s 163us/step - loss: 0.2157 - accuracy: 0.8814 - val_loss: 3.6587 - 

In [72]:
print('train accuracy: %.2f%%' % (np.mean(hist2.history['accuracy'])*100))

train accuracy: 89.11%


In [29]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [30]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS210,2,0,0.999887,0.000112,8.851192e-07
1,p0017kpresabs_qual,Grady1,0,0,0.625329,0.369782,4.889404e-03
2,p0017kpresabs_qual,CFBRSa29,2,0,0.999098,0.000269,6.335156e-04
3,p0017kpresabs_qual,CFBRSa03,0,0,0.647338,0.331796,2.086646e-02
4,p0017kpresabs_qual,217,1,0,0.613342,0.381903,4.754707e-03
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS265,1,1,0.025601,0.687962,2.864372e-01
604,p0040presabsSTCC_qual,NY439,2,2,0.161947,0.266501,5.715521e-01
605,p0040presabsSTCC_qual,CFBRSa05,0,0,0.652983,0.254172,9.284494e-02
606,p0040presabsSTCC_qual,NRS205,2,2,0.000927,0.033760,9.653131e-01


In [31]:
y_prob2 = df_proba2[df_proba2['phage']=='p0040kpresabsSTCC_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[1.84247550e-02, 7.28163700e-03, 9.74293650e-01],
       [9.54936560e-01, 4.09944730e-02, 4.06888730e-03],
       [1.35368900e-01, 5.34389870e-03, 8.59287200e-01],
       [2.87932280e-01, 6.65677130e-01, 4.63906500e-02],
       [9.64073300e-01, 3.43099000e-02, 1.61676230e-03],
       [1.63950320e-01, 8.10355370e-01, 2.56943220e-02],
       [2.28987890e-01, 1.20588890e-10, 7.71012100e-01],
       [7.78398150e-03, 4.26918900e-05, 9.92173300e-01],
       [2.10642430e-01, 1.46998180e-01, 6.42359400e-01],
       [1.92961700e-01, 7.87709900e-01, 1.93283450e-02],
       [3.85104130e-04, 8.82549300e-03, 9.90789500e-01],
       [1.13126980e-07, 9.99999900e-01, 1.20730700e-08],
       [1.28378690e-01, 6.53651060e-02, 8.06256200e-01],
       [2.73000270e-01, 1.35853960e-03, 7.25641200e-01],
       [4.83185800e-01, 1.17745650e-01, 3.99068470e-01],
       [2.02772650e-01, 4.67352480e-01, 3.29874870e-01],
       [7.74612600e-02, 3.51442840e-04, 9.22187300e-01],
       [1.21983460e-04, 2.27353

In [32]:
ovo2 = rocauc_ovo(y_test, y_prob2, average="macro", multi_class="ovo")
ovo2

0.6755149739159299

In [33]:
ovr2 = rocauc_ovr(y_test, y_prob2, average="macro", multi_class="ovr")
ovr2

0.6755149739159299

In [34]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y)

In [35]:
dat3 = pd.DataFrame(X_test.iloc[:,0])
dat3['test'] = y_test

In [36]:
dat3

,id,test
165,NRS191,2
64,CFBREBSa135,0
121,NRS064,1
228,NY224,1
114,NRS035,1
...,...,...
16,BCH-SA-01,0
13,504,0
96,GA27,2
177,NRS209,1


In [37]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [38]:
model3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [44]:
model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [45]:
model3.fit(X_train, y_train,
          batch_size=16, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 954us/step - loss: 6.2706 - accuracy: 0.3503 - val_loss: 2.0976 - val_accuracy: 0.3684
Epoch 2/1000
177/177 [==============================] - 0s 246us/step - loss: 3.1092 - accuracy: 0.3220 - val_loss: 1.3125 - val_accuracy: 0.3816
Epoch 3/1000
177/177 [==============================] - 0s 291us/step - loss: 1.3309 - accuracy: 0.3955 - val_loss: 1.6854 - val_accuracy: 0.4079
Epoch 4/1000
177/177 [==============================] - 0s 294us/step - loss: 1.3364 - accuracy: 0.4520 - val_loss: 1.4134 - val_accuracy: 0.4211
Epoch 5/1000
177/177 [==============================] - 0s 207us/step - loss: 1.3581 - accuracy: 0.4068 - val_loss: 1.1997 - val_accuracy: 0.3684
Epoch 6/1000
177/177 [==============================] - 0s 203us/step - loss: 1.2104 - accuracy: 0.4689 - val_loss: 1.3653 - val_accuracy: 0.4211
Epoch 7/1000
177/177 [==============================] - 0s 193us/step - loss: 1

Epoch 112/1000
177/177 [==============================] - 0s 176us/step - loss: 0.9653 - accuracy: 0.6328 - val_loss: 1.0171 - val_accuracy: 0.4868
Epoch 113/1000
177/177 [==============================] - 0s 174us/step - loss: 0.7939 - accuracy: 0.6780 - val_loss: 1.1239 - val_accuracy: 0.5000
Epoch 114/1000
177/177 [==============================] - 0s 210us/step - loss: 0.8222 - accuracy: 0.6780 - val_loss: 0.9728 - val_accuracy: 0.5526
Epoch 115/1000
177/177 [==============================] - 0s 180us/step - loss: 0.8058 - accuracy: 0.6554 - val_loss: 1.4205 - val_accuracy: 0.4211
Epoch 116/1000
177/177 [==============================] - 0s 201us/step - loss: 3.9665 - accuracy: 0.5876 - val_loss: 4.0452 - val_accuracy: 0.3947
Epoch 117/1000
177/177 [==============================] - 0s 135us/step - loss: 4.6041 - accuracy: 0.5989 - val_loss: 2.3754 - val_accuracy: 0.4342
Epoch 118/1000
177/177 [==============================] - 0s 125us/step - loss: 2.8757 - accuracy: 0.5876 - val_

177/177 [==============================] - 0s 158us/step - loss: 0.7497 - accuracy: 0.6780 - val_loss: 1.0527 - val_accuracy: 0.5658
Epoch 223/1000
177/177 [==============================] - 0s 166us/step - loss: 0.7409 - accuracy: 0.7232 - val_loss: 1.0703 - val_accuracy: 0.5395
Epoch 224/1000
177/177 [==============================] - 0s 264us/step - loss: 0.7846 - accuracy: 0.6949 - val_loss: 1.1049 - val_accuracy: 0.5132
Epoch 225/1000
177/177 [==============================] - 0s 151us/step - loss: 0.8014 - accuracy: 0.7119 - val_loss: 1.0701 - val_accuracy: 0.5263
Epoch 226/1000
177/177 [==============================] - 0s 281us/step - loss: 0.8792 - accuracy: 0.6836 - val_loss: 1.0732 - val_accuracy: 0.5263
Epoch 227/1000
177/177 [==============================] - 0s 375us/step - loss: 0.7278 - accuracy: 0.6836 - val_loss: 1.0683 - val_accuracy: 0.5263
Epoch 228/1000
177/177 [==============================] - 0s 302us/step - loss: 0.8048 - accuracy: 0.7006 - val_loss: 1.3153 - 

177/177 [==============================] - 0s 147us/step - loss: 1.3740 - accuracy: 0.7119 - val_loss: 1.3539 - val_accuracy: 0.5000
Epoch 333/1000
177/177 [==============================] - 0s 156us/step - loss: 0.8978 - accuracy: 0.7006 - val_loss: 1.3295 - val_accuracy: 0.5263
Epoch 334/1000
177/177 [==============================] - 0s 150us/step - loss: 0.6739 - accuracy: 0.7458 - val_loss: 1.2190 - val_accuracy: 0.5658
Epoch 335/1000
177/177 [==============================] - 0s 144us/step - loss: 1.0616 - accuracy: 0.7232 - val_loss: 1.3665 - val_accuracy: 0.4605
Epoch 336/1000
177/177 [==============================] - 0s 160us/step - loss: 0.9865 - accuracy: 0.7175 - val_loss: 1.1291 - val_accuracy: 0.5789
Epoch 337/1000
177/177 [==============================] - 0s 166us/step - loss: 0.6747 - accuracy: 0.7853 - val_loss: 1.1651 - val_accuracy: 0.5658
Epoch 338/1000
177/177 [==============================] - 0s 151us/step - loss: 0.6883 - accuracy: 0.7571 - val_loss: 1.1503 - 

177/177 [==============================] - 0s 148us/step - loss: 0.5717 - accuracy: 0.7853 - val_loss: 1.3062 - val_accuracy: 0.5132
Epoch 443/1000
177/177 [==============================] - 0s 151us/step - loss: 0.5507 - accuracy: 0.8023 - val_loss: 1.3080 - val_accuracy: 0.5263
Epoch 444/1000
177/177 [==============================] - 0s 152us/step - loss: 0.5525 - accuracy: 0.8079 - val_loss: 1.3134 - val_accuracy: 0.5395
Epoch 445/1000
177/177 [==============================] - 0s 142us/step - loss: 0.5403 - accuracy: 0.7966 - val_loss: 1.3387 - val_accuracy: 0.5526
Epoch 446/1000
177/177 [==============================] - 0s 133us/step - loss: 0.6881 - accuracy: 0.8192 - val_loss: 1.4069 - val_accuracy: 0.5263
Epoch 447/1000
177/177 [==============================] - 0s 149us/step - loss: 0.5311 - accuracy: 0.8249 - val_loss: 1.3430 - val_accuracy: 0.5395
Epoch 448/1000
177/177 [==============================] - 0s 172us/step - loss: 0.6328 - accuracy: 0.7966 - val_loss: 1.2976 - 

177/177 [==============================] - 0s 135us/step - loss: 0.6456 - accuracy: 0.7345 - val_loss: 1.5385 - val_accuracy: 0.5132
Epoch 553/1000
177/177 [==============================] - 0s 126us/step - loss: 0.6641 - accuracy: 0.7797 - val_loss: 1.5815 - val_accuracy: 0.5000
Epoch 554/1000
177/177 [==============================] - 0s 146us/step - loss: 0.5132 - accuracy: 0.8136 - val_loss: 1.4404 - val_accuracy: 0.5395
Epoch 555/1000
177/177 [==============================] - 0s 172us/step - loss: 0.5437 - accuracy: 0.8079 - val_loss: 1.3746 - val_accuracy: 0.5263
Epoch 556/1000
177/177 [==============================] - 0s 161us/step - loss: 0.5514 - accuracy: 0.8136 - val_loss: 1.4719 - val_accuracy: 0.5263
Epoch 557/1000
177/177 [==============================] - 0s 170us/step - loss: 0.5915 - accuracy: 0.8023 - val_loss: 1.4279 - val_accuracy: 0.5395
Epoch 558/1000
177/177 [==============================] - 0s 173us/step - loss: 0.4782 - accuracy: 0.8249 - val_loss: 1.4413 - 

177/177 [==============================] - 0s 151us/step - loss: 0.4829 - accuracy: 0.8023 - val_loss: 1.5482 - val_accuracy: 0.5132
Epoch 663/1000
177/177 [==============================] - 0s 155us/step - loss: 0.5062 - accuracy: 0.7966 - val_loss: 1.5705 - val_accuracy: 0.4868
Epoch 664/1000
177/177 [==============================] - 0s 146us/step - loss: 0.5610 - accuracy: 0.8079 - val_loss: 1.6095 - val_accuracy: 0.5000
Epoch 665/1000
177/177 [==============================] - 0s 139us/step - loss: 0.4923 - accuracy: 0.8023 - val_loss: 1.6399 - val_accuracy: 0.4737
Epoch 666/1000
177/177 [==============================] - 0s 146us/step - loss: 0.6280 - accuracy: 0.8136 - val_loss: 1.6319 - val_accuracy: 0.5000
Epoch 667/1000
177/177 [==============================] - 0s 155us/step - loss: 0.5028 - accuracy: 0.8305 - val_loss: 1.6159 - val_accuracy: 0.4868
Epoch 668/1000
177/177 [==============================] - 0s 160us/step - loss: 0.6501 - accuracy: 0.8249 - val_loss: 1.5426 - 

177/177 [==============================] - 0s 157us/step - loss: 0.5120 - accuracy: 0.8249 - val_loss: 1.7008 - val_accuracy: 0.4868
Epoch 773/1000
177/177 [==============================] - 0s 153us/step - loss: 0.4496 - accuracy: 0.8531 - val_loss: 1.6885 - val_accuracy: 0.4868
Epoch 774/1000
177/177 [==============================] - 0s 161us/step - loss: 0.3957 - accuracy: 0.8475 - val_loss: 1.7363 - val_accuracy: 0.4868
Epoch 775/1000
177/177 [==============================] - 0s 170us/step - loss: 0.4148 - accuracy: 0.8418 - val_loss: 1.7864 - val_accuracy: 0.4737
Epoch 776/1000
177/177 [==============================] - 0s 180us/step - loss: 0.4202 - accuracy: 0.8531 - val_loss: 1.7492 - val_accuracy: 0.5000
Epoch 777/1000
177/177 [==============================] - 0s 183us/step - loss: 0.5012 - accuracy: 0.8418 - val_loss: 1.7818 - val_accuracy: 0.4868
Epoch 778/1000
177/177 [==============================] - 0s 165us/step - loss: 0.4821 - accuracy: 0.8305 - val_loss: 1.7305 - 

177/177 [==============================] - 0s 155us/step - loss: 0.4031 - accuracy: 0.8588 - val_loss: 1.9798 - val_accuracy: 0.5000
Epoch 883/1000
177/177 [==============================] - 0s 154us/step - loss: 0.4603 - accuracy: 0.8418 - val_loss: 1.8252 - val_accuracy: 0.4474
Epoch 884/1000
177/177 [==============================] - 0s 150us/step - loss: 0.4124 - accuracy: 0.8531 - val_loss: 1.8864 - val_accuracy: 0.4737
Epoch 885/1000
177/177 [==============================] - 0s 156us/step - loss: 0.5365 - accuracy: 0.8192 - val_loss: 1.8068 - val_accuracy: 0.4737
Epoch 886/1000
177/177 [==============================] - 0s 154us/step - loss: 0.5545 - accuracy: 0.7966 - val_loss: 1.8168 - val_accuracy: 0.5132
Epoch 887/1000
177/177 [==============================] - 0s 156us/step - loss: 0.6025 - accuracy: 0.8023 - val_loss: 2.0612 - val_accuracy: 0.4737
Epoch 888/1000
177/177 [==============================] - 0s 164us/step - loss: 2.8964 - accuracy: 0.7232 - val_loss: 4.8264 - 

177/177 [==============================] - 0s 149us/step - loss: 0.4808 - accuracy: 0.8362 - val_loss: 1.9581 - val_accuracy: 0.4737
Epoch 993/1000
177/177 [==============================] - 0s 153us/step - loss: 0.4617 - accuracy: 0.8588 - val_loss: 1.9337 - val_accuracy: 0.4605
Epoch 994/1000
177/177 [==============================] - 0s 154us/step - loss: 0.4643 - accuracy: 0.8418 - val_loss: 1.8996 - val_accuracy: 0.4737
Epoch 995/1000
177/177 [==============================] - 0s 151us/step - loss: 0.5942 - accuracy: 0.7853 - val_loss: 1.9065 - val_accuracy: 0.4605
Epoch 996/1000
177/177 [==============================] - 0s 157us/step - loss: 0.3999 - accuracy: 0.8362 - val_loss: 1.9026 - val_accuracy: 0.5395
Epoch 997/1000
177/177 [==============================] - 0s 146us/step - loss: 0.4349 - accuracy: 0.8475 - val_loss: 1.8894 - val_accuracy: 0.4868
Epoch 998/1000
177/177 [==============================] - 0s 161us/step - loss: 0.6255 - accuracy: 0.8475 - val_loss: 1.9165 - 

In [88]:
acc_test3 = model3.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test3*100))

76/76 [==============================] - 0s 89us/step
test accuracy: 52.63%


In [46]:
pred3 = model3.predict_classes(X_test)
pred3

array([2, 2, 1, 1, 2, 1, 0, 0, 1, 1, 1, 0, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2,
       0, 1, 2, 1, 2, 1, 0, 1, 2, 2, 1, 2, 2, 1, 0, 2, 0, 2, 0, 2, 1, 2,
       2, 1, 2, 2, 1, 0, 1, 1, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2, 1, 1, 2,
       2, 2, 2, 1, 1, 0, 2, 0, 0, 1])

In [47]:
dat3['pred'] = pred3
dat3

,id,test,pred
165,NRS191,2,2
64,CFBREBSa135,0,2
121,NRS064,1,1
228,NY224,1,1
114,NRS035,1,2
...,...,...,...
16,BCH-SA-01,0,0
13,504,0,2
96,GA27,2,0
177,NRS209,1,0


In [48]:
proba3 = model3.predict_proba(X_test)
dat_proba3 = pd.DataFrame(proba3)

In [49]:
dat_proba3

,0,1,2
0,0.018405,0.123371,0.858224
1,0.030174,0.031852,0.937974
2,0.024678,0.974793,0.000530
3,0.088177,0.483004,0.428819
4,0.236390,0.057490,0.706120
...,...,...,...
71,0.998154,0.000131,0.001715
72,0.007284,0.274291,0.718424
73,0.872361,0.056775,0.070865
74,0.605828,0.339342,0.054829


In [50]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [51]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p40ST.csv", index = False,
         header=None)

In [92]:
hist3 = model3.fit(X_train, y_train,
          batch_size=16, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 213us/step - loss: 0.3291 - accuracy: 0.8701 - val_loss: 2.2165 - val_accuracy: 0.5395
Epoch 2/1000
177/177 [==============================] - 0s 278us/step - loss: 0.6771 - accuracy: 0.8588 - val_loss: 3.2464 - val_accuracy: 0.5395
Epoch 3/1000
177/177 [==============================] - 0s 318us/step - loss: 0.8308 - accuracy: 0.8531 - val_loss: 2.7899 - val_accuracy: 0.5132
Epoch 4/1000
177/177 [==============================] - 0s 239us/step - loss: 0.5983 - accuracy: 0.8701 - val_loss: 2.2542 - val_accuracy: 0.5132
Epoch 5/1000
177/177 [==============================] - 0s 246us/step - loss: 0.4758 - accuracy: 0.8644 - val_loss: 2.2256 - val_accuracy: 0.5000
Epoch 6/1000
177/177 [==============================] - 0s 213us/step - loss: 0.4030 - accuracy: 0.8644 - val_loss: 2.4340 - val_accuracy: 0.5263
Epoch 7/1000
177/177 [==============================] - 0s 264us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 274us/step - loss: 0.7868 - accuracy: 0.8136 - val_loss: 2.3825 - val_accuracy: 0.5000
Epoch 113/1000
177/177 [==============================] - 0s 266us/step - loss: 0.5034 - accuracy: 0.8023 - val_loss: 2.3951 - val_accuracy: 0.5132
Epoch 114/1000
177/177 [==============================] - 0s 284us/step - loss: 0.4173 - accuracy: 0.8362 - val_loss: 2.3382 - val_accuracy: 0.5000
Epoch 115/1000
177/177 [==============================] - 0s 278us/step - loss: 0.4489 - accuracy: 0.8531 - val_loss: 2.3169 - val_accuracy: 0.5132
Epoch 116/1000
177/177 [==============================] - 0s 256us/step - loss: 0.4429 - accuracy: 0.8588 - val_loss: 2.3397 - val_accuracy: 0.5395
Epoch 117/1000
177/177 [==============================] - 0s 250us/step - loss: 0.4111 - accuracy: 0.8644 - val_loss: 2.3552 - val_accuracy: 0.5395
Epoch 118/1000
177/177 [==============================] - 0s 277us/step - loss: 0.4626 - accuracy: 0.8475 - val_

177/177 [==============================] - 0s 259us/step - loss: 0.3520 - accuracy: 0.8814 - val_loss: 2.5287 - val_accuracy: 0.4737
Epoch 223/1000
177/177 [==============================] - 0s 188us/step - loss: 0.4063 - accuracy: 0.8644 - val_loss: 2.5648 - val_accuracy: 0.4868
Epoch 224/1000
177/177 [==============================] - 0s 194us/step - loss: 0.4335 - accuracy: 0.8757 - val_loss: 2.6070 - val_accuracy: 0.4474
Epoch 225/1000
177/177 [==============================] - 0s 350us/step - loss: 1.8069 - accuracy: 0.7910 - val_loss: 4.1777 - val_accuracy: 0.3947
Epoch 226/1000
177/177 [==============================] - 0s 295us/step - loss: 3.9049 - accuracy: 0.7910 - val_loss: 3.5383 - val_accuracy: 0.4342
Epoch 227/1000
177/177 [==============================] - 0s 288us/step - loss: 3.0499 - accuracy: 0.7853 - val_loss: 2.8777 - val_accuracy: 0.4737
Epoch 228/1000
177/177 [==============================] - 0s 207us/step - loss: 1.9489 - accuracy: 0.7966 - val_loss: 2.6833 - 

177/177 [==============================] - 0s 414us/step - loss: 0.3059 - accuracy: 0.8870 - val_loss: 2.7225 - val_accuracy: 0.5000
Epoch 333/1000
177/177 [==============================] - 0s 247us/step - loss: 0.3081 - accuracy: 0.8983 - val_loss: 2.7195 - val_accuracy: 0.5000
Epoch 334/1000
177/177 [==============================] - 0s 279us/step - loss: 0.3187 - accuracy: 0.8983 - val_loss: 2.7117 - val_accuracy: 0.5000
Epoch 335/1000
177/177 [==============================] - 0s 237us/step - loss: 0.3409 - accuracy: 0.8927 - val_loss: 2.7113 - val_accuracy: 0.5000
Epoch 336/1000
177/177 [==============================] - 0s 323us/step - loss: 0.3762 - accuracy: 0.8927 - val_loss: 2.7706 - val_accuracy: 0.4868
Epoch 337/1000
177/177 [==============================] - 0s 237us/step - loss: 0.3188 - accuracy: 0.8814 - val_loss: 2.7242 - val_accuracy: 0.5000
Epoch 338/1000
177/177 [==============================] - 0s 236us/step - loss: 0.3176 - accuracy: 0.8927 - val_loss: 2.7378 - 

177/177 [==============================] - 0s 239us/step - loss: 0.3415 - accuracy: 0.8701 - val_loss: 2.9861 - val_accuracy: 0.4868
Epoch 443/1000
177/177 [==============================] - 0s 463us/step - loss: 0.3236 - accuracy: 0.8927 - val_loss: 2.8964 - val_accuracy: 0.4868
Epoch 444/1000
177/177 [==============================] - 0s 318us/step - loss: 0.3096 - accuracy: 0.8983 - val_loss: 2.8887 - val_accuracy: 0.5000
Epoch 445/1000
177/177 [==============================] - 0s 291us/step - loss: 0.3538 - accuracy: 0.8814 - val_loss: 2.9734 - val_accuracy: 0.5132
Epoch 446/1000
177/177 [==============================] - 0s 252us/step - loss: 0.3295 - accuracy: 0.8814 - val_loss: 3.0681 - val_accuracy: 0.4737
Epoch 447/1000
177/177 [==============================] - 0s 185us/step - loss: 0.3374 - accuracy: 0.8814 - val_loss: 3.0542 - val_accuracy: 0.4868
Epoch 448/1000
177/177 [==============================] - 0s 168us/step - loss: 0.3151 - accuracy: 0.8814 - val_loss: 3.0114 - 

177/177 [==============================] - 0s 188us/step - loss: 0.3725 - accuracy: 0.8701 - val_loss: 3.0571 - val_accuracy: 0.5000
Epoch 553/1000
177/177 [==============================] - 0s 173us/step - loss: 0.3220 - accuracy: 0.8870 - val_loss: 3.0261 - val_accuracy: 0.4868
Epoch 554/1000
177/177 [==============================] - 0s 226us/step - loss: 0.3463 - accuracy: 0.8757 - val_loss: 3.0766 - val_accuracy: 0.5000
Epoch 555/1000
177/177 [==============================] - 0s 255us/step - loss: 0.3035 - accuracy: 0.8870 - val_loss: 3.0300 - val_accuracy: 0.5000
Epoch 556/1000
177/177 [==============================] - 0s 278us/step - loss: 0.3005 - accuracy: 0.8983 - val_loss: 3.0444 - val_accuracy: 0.4868
Epoch 557/1000
177/177 [==============================] - 0s 160us/step - loss: 0.3426 - accuracy: 0.8814 - val_loss: 3.0215 - val_accuracy: 0.5000
Epoch 558/1000
177/177 [==============================] - 0s 145us/step - loss: 0.3081 - accuracy: 0.9040 - val_loss: 3.0384 - 

177/177 [==============================] - 0s 192us/step - loss: 0.3529 - accuracy: 0.8814 - val_loss: 3.2228 - val_accuracy: 0.4605
Epoch 663/1000
177/177 [==============================] - 0s 195us/step - loss: 0.3459 - accuracy: 0.8983 - val_loss: 3.2170 - val_accuracy: 0.4474
Epoch 664/1000
177/177 [==============================] - 0s 199us/step - loss: 0.5072 - accuracy: 0.8644 - val_loss: 3.1265 - val_accuracy: 0.4868
Epoch 665/1000
177/177 [==============================] - 0s 192us/step - loss: 0.4324 - accuracy: 0.8418 - val_loss: 3.3473 - val_accuracy: 0.4605
Epoch 666/1000
177/177 [==============================] - 0s 163us/step - loss: 0.3897 - accuracy: 0.8588 - val_loss: 3.1633 - val_accuracy: 0.4605
Epoch 667/1000
177/177 [==============================] - 0s 166us/step - loss: 0.4380 - accuracy: 0.8588 - val_loss: 3.1229 - val_accuracy: 0.4737
Epoch 668/1000
177/177 [==============================] - 0s 174us/step - loss: 0.3124 - accuracy: 0.8927 - val_loss: 3.1412 - 

177/177 [==============================] - 0s 173us/step - loss: 0.2942 - accuracy: 0.8870 - val_loss: 3.3079 - val_accuracy: 0.5000
Epoch 773/1000
177/177 [==============================] - 0s 180us/step - loss: 0.3022 - accuracy: 0.9040 - val_loss: 3.2824 - val_accuracy: 0.5000
Epoch 774/1000
177/177 [==============================] - 0s 164us/step - loss: 0.2840 - accuracy: 0.8983 - val_loss: 3.2856 - val_accuracy: 0.5000
Epoch 775/1000
177/177 [==============================] - 0s 146us/step - loss: 0.3170 - accuracy: 0.8983 - val_loss: 3.2728 - val_accuracy: 0.5000
Epoch 776/1000
177/177 [==============================] - 0s 161us/step - loss: 0.3938 - accuracy: 0.8814 - val_loss: 3.2794 - val_accuracy: 0.5000
Epoch 777/1000
177/177 [==============================] - 0s 184us/step - loss: 0.6435 - accuracy: 0.8588 - val_loss: 3.3280 - val_accuracy: 0.4737
Epoch 778/1000
177/177 [==============================] - 0s 172us/step - loss: 0.3872 - accuracy: 0.8757 - val_loss: 3.2937 - 

177/177 [==============================] - 0s 164us/step - loss: 0.2501 - accuracy: 0.9040 - val_loss: 3.4719 - val_accuracy: 0.4868
Epoch 883/1000
177/177 [==============================] - 0s 176us/step - loss: 0.3664 - accuracy: 0.8927 - val_loss: 3.5527 - val_accuracy: 0.4474
Epoch 884/1000
177/177 [==============================] - 0s 184us/step - loss: 2.1986 - accuracy: 0.8079 - val_loss: 5.6860 - val_accuracy: 0.3816
Epoch 885/1000
177/177 [==============================] - 0s 202us/step - loss: 3.7817 - accuracy: 0.7966 - val_loss: 4.8948 - val_accuracy: 0.4079
Epoch 886/1000
177/177 [==============================] - 0s 181us/step - loss: 2.8133 - accuracy: 0.7966 - val_loss: 4.1284 - val_accuracy: 0.4474
Epoch 887/1000
177/177 [==============================] - 0s 168us/step - loss: 1.4344 - accuracy: 0.8475 - val_loss: 3.6535 - val_accuracy: 0.4605
Epoch 888/1000
177/177 [==============================] - 0s 191us/step - loss: 0.4630 - accuracy: 0.8588 - val_loss: 3.5878 - 

177/177 [==============================] - 0s 176us/step - loss: 0.9309 - accuracy: 0.8418 - val_loss: 3.8084 - val_accuracy: 0.4737
Epoch 993/1000
177/177 [==============================] - 0s 208us/step - loss: 0.5806 - accuracy: 0.8701 - val_loss: 3.7207 - val_accuracy: 0.5263
Epoch 994/1000
177/177 [==============================] - 0s 163us/step - loss: 0.4027 - accuracy: 0.8870 - val_loss: 3.6841 - val_accuracy: 0.5132
Epoch 995/1000
177/177 [==============================] - 0s 168us/step - loss: 0.3030 - accuracy: 0.8927 - val_loss: 3.7372 - val_accuracy: 0.5000
Epoch 996/1000
177/177 [==============================] - 0s 168us/step - loss: 0.3120 - accuracy: 0.8983 - val_loss: 3.7000 - val_accuracy: 0.5000
Epoch 997/1000
177/177 [==============================] - 0s 172us/step - loss: 0.2949 - accuracy: 0.8983 - val_loss: 3.6585 - val_accuracy: 0.5000
Epoch 998/1000
177/177 [==============================] - 0s 278us/step - loss: 0.2743 - accuracy: 0.8983 - val_loss: 3.6673 - 

In [93]:
print('train accuracy: %.2f%%' % (np.mean(hist3.history['accuracy'])*100))

train accuracy: 87.66%


In [39]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [40]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NY360,2,2,2.165526e-02,4.848140e-01,0.493531
1,p0017kpresabs_qual,EUH25,2,0,9.986388e-01,1.245148e-03,0.000116
2,p0017kpresabs_qual,EUH15,2,2,9.227520e-04,1.424882e-02,0.984828
3,p0017kpresabs_qual,NRS241,0,0,8.374333e-01,1.614128e-01,0.001154
4,p0017kpresabs_qual,SR2852,2,2,3.976981e-09,5.145955e-10,1.000000
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,BCH-SA-01,0,0,9.305407e-01,6.356251e-02,0.005897
604,p0040presabsSTCC_qual,504,0,1,4.476389e-02,8.577548e-01,0.097481
605,p0040presabsSTCC_qual,GA27,2,1,2.779456e-01,5.384378e-01,0.183617
606,p0040presabsSTCC_qual,NRS209,1,0,4.210180e-01,3.559393e-01,0.223043


In [41]:
y_prob3 = df_proba3[df_proba3['phage']=='p0040kpresabsSTCC_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[1.84047240e-02, 1.23370910e-01, 8.58224400e-01],
       [3.01738250e-02, 3.18521100e-02, 9.37974100e-01],
       [2.46778340e-02, 9.74792600e-01, 5.29571960e-04],
       [8.81773900e-02, 4.83004060e-01, 4.28818550e-01],
       [2.36389710e-01, 5.74901630e-02, 7.06120130e-01],
       [1.34070330e-02, 5.34539400e-01, 4.52053500e-01],
       [9.91652200e-01, 1.74775600e-07, 8.34764200e-03],
       [9.13512470e-01, 8.62743200e-02, 2.13249630e-04],
       [1.65458500e-03, 9.30250300e-01, 6.80951500e-02],
       [3.26524230e-03, 8.63844400e-01, 1.32890340e-01],
       [1.41579100e-02, 9.84664260e-01, 1.17783030e-03],
       [5.14899130e-01, 1.56066660e-01, 3.29034240e-01],
       [2.46587500e-03, 7.07946350e-02, 9.26739500e-01],
       [2.35644470e-02, 1.22822740e-01, 8.53612800e-01],
       [4.16206330e-01, 1.18312260e-02, 5.71962400e-01],
       [5.29317100e-02, 9.11809400e-01, 3.52589080e-02],
       [1.77910940e-02, 9.76023700e-01, 6.18527400e-03],
       [1.97817140e-02, 9.79714

In [42]:
ovo3 = rocauc_ovo(y_test, y_prob3, average="macro", multi_class="ovo")
ovo3

0.6678516701493923

In [43]:
ovr3 = rocauc_ovr(y_test, y_prob3, average="macro", multi_class="ovr")
ovr3

0.6678516701493923

In [44]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y)

In [45]:
dat4 = pd.DataFrame(X_test.iloc[:,0])
dat4['test'] = y_test

In [46]:
dat4

,id,test
54,CFBREBSa121,1
28,BCH-SA-13,1
143,NRS148,2
73,CFBRSa23,2
57,CFBREBSa125,0
...,...,...
208,NRS247,1
183,NRS215,1
248,SR4152,2
114,NRS035,1


In [47]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [48]:
model4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [57]:
model4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [58]:
model4.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 824us/step - loss: 18.5000 - accuracy: 0.2542 - val_loss: 22.1666 - val_accuracy: 0.3289
Epoch 2/1000
177/177 [==============================] - 0s 71us/step - loss: 15.7549 - accuracy: 0.2768 - val_loss: 18.3164 - val_accuracy: 0.3947
Epoch 3/1000
177/177 [==============================] - 0s 106us/step - loss: 13.2167 - accuracy: 0.2655 - val_loss: 14.6581 - val_accuracy: 0.3289
Epoch 4/1000
177/177 [==============================] - 0s 106us/step - loss: 10.3279 - accuracy: 0.2655 - val_loss: 11.1065 - val_accuracy: 0.2895
Epoch 5/1000
177/177 [==============================] - 0s 169us/step - loss: 7.8435 - accuracy: 0.3164 - val_loss: 7.8195 - val_accuracy: 0.3158
Epoch 6/1000
177/177 [==============================] - 0s 95us/step - loss: 5.9247 - accuracy: 0.3842 - val_loss: 7.3262 - val_accuracy: 0.3816
Epoch 7/1000
177/177 [==============================] - 0s 139us/step - l

177/177 [==============================] - 0s 76us/step - loss: 0.7445 - accuracy: 0.7006 - val_loss: 1.2025 - val_accuracy: 0.3816
Epoch 112/1000
177/177 [==============================] - 0s 91us/step - loss: 0.7525 - accuracy: 0.7006 - val_loss: 1.1624 - val_accuracy: 0.3947
Epoch 113/1000
177/177 [==============================] - 0s 135us/step - loss: 0.7823 - accuracy: 0.7006 - val_loss: 1.2288 - val_accuracy: 0.3684
Epoch 114/1000
177/177 [==============================] - 0s 117us/step - loss: 0.7653 - accuracy: 0.6836 - val_loss: 1.3038 - val_accuracy: 0.3421
Epoch 115/1000
177/177 [==============================] - 0s 169us/step - loss: 0.7944 - accuracy: 0.6836 - val_loss: 1.2546 - val_accuracy: 0.3947
Epoch 116/1000
177/177 [==============================] - 0s 201us/step - loss: 0.7374 - accuracy: 0.7119 - val_loss: 1.2822 - val_accuracy: 0.3816
Epoch 117/1000
177/177 [==============================] - 0s 124us/step - loss: 0.8105 - accuracy: 0.6949 - val_loss: 1.2980 - va

Epoch 167/1000
177/177 [==============================] - 0s 64us/step - loss: 0.6943 - accuracy: 0.7232 - val_loss: 1.3720 - val_accuracy: 0.4605
Epoch 168/1000
177/177 [==============================] - 0s 63us/step - loss: 0.6895 - accuracy: 0.7232 - val_loss: 1.3293 - val_accuracy: 0.4211
Epoch 169/1000
177/177 [==============================] - 0s 77us/step - loss: 0.7008 - accuracy: 0.7119 - val_loss: 1.3245 - val_accuracy: 0.4342
Epoch 170/1000
177/177 [==============================] - 0s 90us/step - loss: 0.6703 - accuracy: 0.7119 - val_loss: 1.2410 - val_accuracy: 0.4474
Epoch 171/1000
177/177 [==============================] - 0s 73us/step - loss: 0.6666 - accuracy: 0.7345 - val_loss: 1.2523 - val_accuracy: 0.4737
Epoch 172/1000
177/177 [==============================] - 0s 73us/step - loss: 0.6504 - accuracy: 0.7401 - val_loss: 1.2297 - val_accuracy: 0.4474
Epoch 173/1000
177/177 [==============================] - 0s 74us/step - loss: 0.6519 - accuracy: 0.7345 - val_loss: 1

Epoch 223/1000
177/177 [==============================] - 0s 63us/step - loss: 0.6026 - accuracy: 0.7740 - val_loss: 1.2648 - val_accuracy: 0.4605
Epoch 224/1000
177/177 [==============================] - 0s 63us/step - loss: 0.5994 - accuracy: 0.7740 - val_loss: 1.3198 - val_accuracy: 0.4342
Epoch 225/1000
177/177 [==============================] - 0s 79us/step - loss: 0.6045 - accuracy: 0.7627 - val_loss: 1.2759 - val_accuracy: 0.4474
Epoch 226/1000
177/177 [==============================] - 0s 61us/step - loss: 0.6160 - accuracy: 0.7458 - val_loss: 1.2538 - val_accuracy: 0.4605
Epoch 227/1000
177/177 [==============================] - 0s 55us/step - loss: 0.6121 - accuracy: 0.7627 - val_loss: 1.2743 - val_accuracy: 0.4474
Epoch 228/1000
177/177 [==============================] - 0s 55us/step - loss: 0.6127 - accuracy: 0.7571 - val_loss: 1.2471 - val_accuracy: 0.4605
Epoch 229/1000
177/177 [==============================] - 0s 60us/step - loss: 0.6022 - accuracy: 0.7514 - val_loss: 1

Epoch 279/1000
177/177 [==============================] - 0s 65us/step - loss: 0.5723 - accuracy: 0.7627 - val_loss: 1.3553 - val_accuracy: 0.4605
Epoch 280/1000
177/177 [==============================] - 0s 67us/step - loss: 0.5679 - accuracy: 0.7740 - val_loss: 1.3054 - val_accuracy: 0.4605
Epoch 281/1000
177/177 [==============================] - 0s 81us/step - loss: 0.5683 - accuracy: 0.7684 - val_loss: 1.2676 - val_accuracy: 0.4737
Epoch 282/1000
177/177 [==============================] - 0s 217us/step - loss: 0.5625 - accuracy: 0.7684 - val_loss: 1.2850 - val_accuracy: 0.4737
Epoch 283/1000
177/177 [==============================] - 0s 79us/step - loss: 0.5556 - accuracy: 0.7627 - val_loss: 1.3404 - val_accuracy: 0.4605
Epoch 284/1000
177/177 [==============================] - 0s 84us/step - loss: 0.5875 - accuracy: 0.7627 - val_loss: 1.2744 - val_accuracy: 0.4737
Epoch 285/1000
177/177 [==============================] - 0s 72us/step - loss: 0.5636 - accuracy: 0.7797 - val_loss: 

...

Epoch 667/1000
177/177 [==============================] - 0s 77us/step - loss: 0.4100 - accuracy: 0.8249 - val_loss: 1.5183 - val_accuracy: 0.4605
Epoch 668/1000
177/177 [==============================] - 0s 80us/step - loss: 0.4163 - accuracy: 0.8249 - val_loss: 1.5569 - val_accuracy: 0.4211
Epoch 669/1000
177/177 [==============================] - 0s 65us/step - loss: 0.4064 - accuracy: 0.8249 - val_loss: 1.5589 - val_accuracy: 0.4474
Epoch 670/1000
177/177 [==============================] - 0s 74us/step - loss: 0.4043 - accuracy: 0.8249 - val_loss: 1.5924 - val_accuracy: 0.4474
Epoch 671/1000
177/177 [==============================] - 0s 88us/step - loss: 0.4100 - accuracy: 0.8249 - val_loss: 1.5274 - val_accuracy: 0.4605
Epoch 672/1000
177/177 [==============================] - 0s 81us/step - loss: 0.4374 - accuracy: 0.8192 - val_loss: 1.5215 - val_accuracy: 0.4342
Epoch 673/1000
177/177 [==============================] - 0s 78us/step - loss: 0.4166 - accuracy: 0.8192 - val_loss: 1

Epoch 723/1000
177/177 [==============================] - 0s 52us/step - loss: 0.4123 - accuracy: 0.8362 - val_loss: 1.5616 - val_accuracy: 0.4605
Epoch 724/1000
177/177 [==============================] - 0s 55us/step - loss: 0.4001 - accuracy: 0.8249 - val_loss: 1.6156 - val_accuracy: 0.4605
Epoch 725/1000
177/177 [==============================] - 0s 52us/step - loss: 0.4088 - accuracy: 0.8079 - val_loss: 1.6331 - val_accuracy: 0.4868
Epoch 726/1000
177/177 [==============================] - 0s 51us/step - loss: 0.4003 - accuracy: 0.8192 - val_loss: 1.6173 - val_accuracy: 0.4605
Epoch 727/1000
177/177 [==============================] - 0s 53us/step - loss: 0.3916 - accuracy: 0.8305 - val_loss: 1.5809 - val_accuracy: 0.4737
Epoch 728/1000
177/177 [==============================] - 0s 58us/step - loss: 0.3900 - accuracy: 0.8418 - val_loss: 1.5928 - val_accuracy: 0.4605
Epoch 729/1000
177/177 [==============================] - 0s 50us/step - loss: 0.3931 - accuracy: 0.8192 - val_loss: 1

Epoch 779/1000
177/177 [==============================] - 0s 75us/step - loss: 0.4069 - accuracy: 0.8192 - val_loss: 1.7724 - val_accuracy: 0.4474
Epoch 780/1000
177/177 [==============================] - 0s 89us/step - loss: 0.4178 - accuracy: 0.8079 - val_loss: 1.6610 - val_accuracy: 0.4605
Epoch 781/1000
177/177 [==============================] - 0s 82us/step - loss: 0.3917 - accuracy: 0.8362 - val_loss: 1.6057 - val_accuracy: 0.4605
Epoch 782/1000
177/177 [==============================] - 0s 63us/step - loss: 0.3839 - accuracy: 0.8305 - val_loss: 1.6633 - val_accuracy: 0.4474
Epoch 783/1000
177/177 [==============================] - 0s 75us/step - loss: 0.3961 - accuracy: 0.8362 - val_loss: 1.6345 - val_accuracy: 0.4605
Epoch 784/1000
177/177 [==============================] - 0s 85us/step - loss: 0.3805 - accuracy: 0.8305 - val_loss: 1.6558 - val_accuracy: 0.4079
Epoch 785/1000
177/177 [==============================] - 0s 83us/step - loss: 0.4146 - accuracy: 0.8136 - val_loss: 1

Epoch 835/1000
177/177 [==============================] - 0s 63us/step - loss: 0.3977 - accuracy: 0.8475 - val_loss: 1.6597 - val_accuracy: 0.4605
Epoch 836/1000
177/177 [==============================] - 0s 74us/step - loss: 0.3797 - accuracy: 0.8418 - val_loss: 1.7607 - val_accuracy: 0.4474
Epoch 837/1000
177/177 [==============================] - 0s 71us/step - loss: 0.3981 - accuracy: 0.8362 - val_loss: 1.6632 - val_accuracy: 0.4605
Epoch 838/1000
177/177 [==============================] - 0s 76us/step - loss: 0.3803 - accuracy: 0.8531 - val_loss: 1.7441 - val_accuracy: 0.4079
Epoch 839/1000
177/177 [==============================] - 0s 74us/step - loss: 0.4262 - accuracy: 0.8249 - val_loss: 1.6585 - val_accuracy: 0.4605
Epoch 840/1000
177/177 [==============================] - 0s 63us/step - loss: 0.3772 - accuracy: 0.8475 - val_loss: 1.6933 - val_accuracy: 0.4474
Epoch 841/1000
177/177 [==============================] - 0s 64us/step - loss: 0.3739 - accuracy: 0.8531 - val_loss: 1

Epoch 891/1000
177/177 [==============================] - 0s 62us/step - loss: 0.3561 - accuracy: 0.8531 - val_loss: 1.7453 - val_accuracy: 0.4605
Epoch 892/1000
177/177 [==============================] - 0s 55us/step - loss: 0.3541 - accuracy: 0.8475 - val_loss: 1.7421 - val_accuracy: 0.4474
Epoch 893/1000
177/177 [==============================] - 0s 78us/step - loss: 0.3572 - accuracy: 0.8531 - val_loss: 1.7276 - val_accuracy: 0.4605
Epoch 894/1000
177/177 [==============================] - 0s 76us/step - loss: 0.3583 - accuracy: 0.8475 - val_loss: 1.7376 - val_accuracy: 0.4474
Epoch 895/1000
177/177 [==============================] - 0s 65us/step - loss: 0.3563 - accuracy: 0.8475 - val_loss: 1.7607 - val_accuracy: 0.4605
Epoch 896/1000
177/177 [==============================] - 0s 57us/step - loss: 0.3571 - accuracy: 0.8531 - val_loss: 1.7451 - val_accuracy: 0.4605
Epoch 897/1000
177/177 [==============================] - 0s 70us/step - loss: 0.3563 - accuracy: 0.8418 - val_loss: 1

In [113]:
acc_test4 = model4.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test4*100))

76/76 [==============================] - 0s 103us/step
test accuracy: 50.00%


In [59]:
pred4 = model4.predict_classes(X_test)
pred4

array([1, 1, 2, 2, 2, 0, 2, 0, 2, 1, 1, 2, 0, 2, 0, 1, 0, 0, 1, 2, 1, 1,
       2, 1, 1, 1, 1, 0, 2, 2, 1, 1, 2, 2, 0, 1, 2, 0, 2, 1, 1, 2, 2, 2,
       0, 0, 0, 2, 1, 0, 0, 2, 0, 0, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 2, 1,
       1, 1, 2, 1, 1, 2, 1, 2, 2, 2])

In [60]:
dat4['pred'] = pred4
dat4

,id,test,pred
54,CFBREBSa121,1,1
28,BCH-SA-13,1,1
143,NRS148,2,2
73,CFBRSa23,2,2
57,CFBREBSa125,0,2
...,...,...,...
208,NRS247,1,2
183,NRS215,1,1
248,SR4152,2,2
114,NRS035,1,2


In [61]:
proba4 = model4.predict_proba(X_test)
dat_proba4 = pd.DataFrame(proba4)

In [62]:
dat_proba4

,0,1,2
0,1.680595e-01,0.716124,0.115816
1,5.092228e-02,0.947701,0.001376
2,6.816542e-04,0.006345,0.992973
3,1.525053e-01,0.091207,0.756288
4,4.154682e-01,0.036786,0.547746
...,...,...,...
71,6.374773e-02,0.023217,0.913036
72,9.414636e-03,0.586090,0.404495
73,8.972445e-02,0.104419,0.805856
74,6.757236e-02,0.049859,0.882568


In [63]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [64]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p40ST.csv", index = False,
         header=None)

In [117]:
hist4 = model4.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 77us/step - loss: 0.3030 - accuracy: 0.8870 - val_loss: 1.8970 - val_accuracy: 0.5000
Epoch 2/1000
177/177 [==============================] - 0s 81us/step - loss: 0.3050 - accuracy: 0.8701 - val_loss: 1.9318 - val_accuracy: 0.4868
Epoch 3/1000
177/177 [==============================] - 0s 104us/step - loss: 0.3086 - accuracy: 0.8814 - val_loss: 1.8865 - val_accuracy: 0.5000
Epoch 4/1000
177/177 [==============================] - 0s 119us/step - loss: 0.3006 - accuracy: 0.8814 - val_loss: 1.8642 - val_accuracy: 0.5132
Epoch 5/1000
177/177 [==============================] - 0s 103us/step - loss: 0.3052 - accuracy: 0.8757 - val_loss: 1.8514 - val_accuracy: 0.5000
Epoch 6/1000
177/177 [==============================] - 0s 109us/step - loss: 0.3018 - accuracy: 0.8757 - val_loss: 1.8830 - val_accuracy: 0.5132
Epoch 7/1000
177/177 [==============================] - 0s 125us/step - loss: 0.3

Epoch 57/1000
177/177 [==============================] - 0s 103us/step - loss: 0.3304 - accuracy: 0.8644 - val_loss: 1.8828 - val_accuracy: 0.5000
Epoch 58/1000
177/177 [==============================] - 0s 114us/step - loss: 0.2963 - accuracy: 0.8757 - val_loss: 2.0528 - val_accuracy: 0.5000
Epoch 59/1000
177/177 [==============================] - 0s 126us/step - loss: 0.3108 - accuracy: 0.8814 - val_loss: 2.0587 - val_accuracy: 0.5000
Epoch 60/1000
177/177 [==============================] - 0s 101us/step - loss: 0.3022 - accuracy: 0.8757 - val_loss: 1.8902 - val_accuracy: 0.5000
Epoch 61/1000
177/177 [==============================] - 0s 121us/step - loss: 0.2961 - accuracy: 0.8757 - val_loss: 1.9002 - val_accuracy: 0.5000
Epoch 62/1000
177/177 [==============================] - 0s 101us/step - loss: 0.3046 - accuracy: 0.8757 - val_loss: 1.9246 - val_accuracy: 0.5132
Epoch 63/1000
177/177 [==============================] - 0s 112us/step - loss: 0.3268 - accuracy: 0.8757 - val_loss: 2

Epoch 113/1000
177/177 [==============================] - 0s 62us/step - loss: 0.2896 - accuracy: 0.8814 - val_loss: 1.9973 - val_accuracy: 0.5132
Epoch 114/1000
177/177 [==============================] - 0s 88us/step - loss: 0.2925 - accuracy: 0.8927 - val_loss: 1.9709 - val_accuracy: 0.5263
Epoch 115/1000
177/177 [==============================] - 0s 76us/step - loss: 0.2953 - accuracy: 0.8814 - val_loss: 1.9614 - val_accuracy: 0.5395
Epoch 116/1000
177/177 [==============================] - 0s 65us/step - loss: 0.2891 - accuracy: 0.8814 - val_loss: 1.9904 - val_accuracy: 0.5132
Epoch 117/1000
177/177 [==============================] - 0s 72us/step - loss: 0.2902 - accuracy: 0.8814 - val_loss: 1.9392 - val_accuracy: 0.5000
Epoch 118/1000
177/177 [==============================] - 0s 70us/step - loss: 0.2879 - accuracy: 0.8870 - val_loss: 1.9257 - val_accuracy: 0.5263
Epoch 119/1000
177/177 [==============================] - 0s 76us/step - loss: 0.2917 - accuracy: 0.8814 - val_loss: 1

Epoch 169/1000
177/177 [==============================] - 0s 102us/step - loss: 0.4502 - accuracy: 0.8475 - val_loss: 2.4530 - val_accuracy: 0.4868
Epoch 170/1000
177/177 [==============================] - 0s 93us/step - loss: 0.3661 - accuracy: 0.8814 - val_loss: 2.1418 - val_accuracy: 0.4737
Epoch 171/1000
177/177 [==============================] - 0s 79us/step - loss: 0.3030 - accuracy: 0.8644 - val_loss: 2.1325 - val_accuracy: 0.4868
Epoch 172/1000
177/177 [==============================] - 0s 66us/step - loss: 0.3470 - accuracy: 0.8418 - val_loss: 2.0006 - val_accuracy: 0.5263
Epoch 173/1000
177/177 [==============================] - 0s 74us/step - loss: 0.3026 - accuracy: 0.8701 - val_loss: 2.1336 - val_accuracy: 0.5000
Epoch 174/1000
177/177 [==============================] - 0s 76us/step - loss: 0.3177 - accuracy: 0.8757 - val_loss: 2.1901 - val_accuracy: 0.5132
Epoch 175/1000
177/177 [==============================] - 0s 60us/step - loss: 0.3022 - accuracy: 0.8757 - val_loss: 

Epoch 225/1000
177/177 [==============================] - 0s 294us/step - loss: 0.2912 - accuracy: 0.8870 - val_loss: 2.2416 - val_accuracy: 0.5000
Epoch 226/1000
177/177 [==============================] - 0s 65us/step - loss: 0.3123 - accuracy: 0.8927 - val_loss: 2.1317 - val_accuracy: 0.5000
Epoch 227/1000
177/177 [==============================] - 0s 87us/step - loss: 0.2867 - accuracy: 0.8814 - val_loss: 2.0532 - val_accuracy: 0.5000
Epoch 228/1000
177/177 [==============================] - 0s 130us/step - loss: 0.2999 - accuracy: 0.8644 - val_loss: 2.0949 - val_accuracy: 0.5000
Epoch 229/1000
177/177 [==============================] - 0s 204us/step - loss: 0.2881 - accuracy: 0.8870 - val_loss: 2.2571 - val_accuracy: 0.5132
Epoch 230/1000
177/177 [==============================] - 0s 98us/step - loss: 0.2956 - accuracy: 0.8927 - val_loss: 2.0677 - val_accuracy: 0.5000
Epoch 231/1000
177/177 [==============================] - 0s 284us/step - loss: 0.2786 - accuracy: 0.8870 - val_los

...

Epoch 502/1000
177/177 [==============================] - 0s 99us/step - loss: 0.2535 - accuracy: 0.8927 - val_loss: 2.2057 - val_accuracy: 0.5000
Epoch 503/1000
177/177 [==============================] - 0s 76us/step - loss: 0.2542 - accuracy: 0.8983 - val_loss: 2.2245 - val_accuracy: 0.5132
Epoch 504/1000
177/177 [==============================] - 0s 85us/step - loss: 0.2536 - accuracy: 0.8983 - val_loss: 2.2475 - val_accuracy: 0.5000
Epoch 505/1000
177/177 [==============================] - 0s 97us/step - loss: 0.2551 - accuracy: 0.8927 - val_loss: 2.2263 - val_accuracy: 0.4868
Epoch 506/1000
177/177 [==============================] - 0s 70us/step - loss: 0.2529 - accuracy: 0.9040 - val_loss: 2.2845 - val_accuracy: 0.4868
Epoch 507/1000
177/177 [==============================] - 0s 72us/step - loss: 0.2574 - accuracy: 0.8927 - val_loss: 2.2609 - val_accuracy: 0.4737
Epoch 508/1000
177/177 [==============================] - 0s 88us/step - loss: 0.2534 - accuracy: 0.8927 - val_loss: 2

Epoch 613/1000
177/177 [==============================] - 0s 168us/step - loss: 0.2507 - accuracy: 0.8983 - val_loss: 2.3191 - val_accuracy: 0.4737
Epoch 614/1000
177/177 [==============================] - 0s 131us/step - loss: 0.2586 - accuracy: 0.8927 - val_loss: 2.4487 - val_accuracy: 0.5000
Epoch 615/1000
177/177 [==============================] - 0s 149us/step - loss: 0.2588 - accuracy: 0.8927 - val_loss: 2.3811 - val_accuracy: 0.4737
Epoch 616/1000
177/177 [==============================] - 0s 99us/step - loss: 0.2493 - accuracy: 0.8927 - val_loss: 2.3891 - val_accuracy: 0.4737
Epoch 617/1000
177/177 [==============================] - 0s 146us/step - loss: 0.2684 - accuracy: 0.8927 - val_loss: 2.3814 - val_accuracy: 0.5000
Epoch 618/1000
177/177 [==============================] - 0s 235us/step - loss: 0.2795 - accuracy: 0.8757 - val_loss: 2.5404 - val_accuracy: 0.5000
Epoch 619/1000
177/177 [==============================] - 0s 123us/step - loss: 0.2693 - accuracy: 0.8927 - val_l

Epoch 724/1000
177/177 [==============================] - 0s 141us/step - loss: 0.2662 - accuracy: 0.8870 - val_loss: 2.4524 - val_accuracy: 0.5000
Epoch 725/1000
177/177 [==============================] - 0s 103us/step - loss: 0.2813 - accuracy: 0.8814 - val_loss: 2.3401 - val_accuracy: 0.4868
Epoch 726/1000
177/177 [==============================] - 0s 76us/step - loss: 0.2693 - accuracy: 0.8814 - val_loss: 2.6374 - val_accuracy: 0.4868
Epoch 727/1000
177/177 [==============================] - 0s 87us/step - loss: 0.2874 - accuracy: 0.8927 - val_loss: 2.6034 - val_accuracy: 0.4868
Epoch 728/1000
177/177 [==============================] - 0s 102us/step - loss: 0.2569 - accuracy: 0.8927 - val_loss: 2.4533 - val_accuracy: 0.5000
Epoch 729/1000
177/177 [==============================] - 0s 77us/step - loss: 0.2672 - accuracy: 0.8927 - val_loss: 2.3612 - val_accuracy: 0.5000
Epoch 730/1000
177/177 [==============================] - 0s 125us/step - loss: 0.2507 - accuracy: 0.8870 - val_los

177/177 [==============================] - 0s 93us/step - loss: 0.3075 - accuracy: 0.8870 - val_loss: 2.7283 - val_accuracy: 0.5132
Epoch 835/1000
177/177 [==============================] - 0s 81us/step - loss: 0.2744 - accuracy: 0.8814 - val_loss: 2.4187 - val_accuracy: 0.5132
Epoch 836/1000
177/177 [==============================] - 0s 82us/step - loss: 0.2825 - accuracy: 0.8757 - val_loss: 2.4673 - val_accuracy: 0.4868
Epoch 837/1000
177/177 [==============================] - 0s 117us/step - loss: 0.2441 - accuracy: 0.8870 - val_loss: 2.5840 - val_accuracy: 0.5000
Epoch 838/1000
177/177 [==============================] - 0s 83us/step - loss: 0.2450 - accuracy: 0.8927 - val_loss: 2.6042 - val_accuracy: 0.5132
Epoch 839/1000
177/177 [==============================] - 0s 169us/step - loss: 0.2444 - accuracy: 0.9040 - val_loss: 2.4632 - val_accuracy: 0.5263
Epoch 840/1000
177/177 [==============================] - 0s 70us/step - loss: 0.2529 - accuracy: 0.9040 - val_loss: 2.4130 - val_a

177/177 [==============================] - 0s 88us/step - loss: 0.2356 - accuracy: 0.8983 - val_loss: 2.5934 - val_accuracy: 0.5000
Epoch 945/1000
177/177 [==============================] - 0s 162us/step - loss: 0.2313 - accuracy: 0.8983 - val_loss: 2.5383 - val_accuracy: 0.5000
Epoch 946/1000
177/177 [==============================] - 0s 111us/step - loss: 0.2359 - accuracy: 0.8983 - val_loss: 2.5429 - val_accuracy: 0.5132
Epoch 947/1000
177/177 [==============================] - 0s 314us/step - loss: 0.2346 - accuracy: 0.9040 - val_loss: 2.6584 - val_accuracy: 0.5132
Epoch 948/1000
177/177 [==============================] - 0s 174us/step - loss: 0.2376 - accuracy: 0.9040 - val_loss: 2.6138 - val_accuracy: 0.5000
Epoch 949/1000
177/177 [==============================] - 0s 289us/step - loss: 0.2310 - accuracy: 0.9040 - val_loss: 2.5463 - val_accuracy: 0.5000
Epoch 950/1000
177/177 [==============================] - 0s 291us/step - loss: 0.2328 - accuracy: 0.9040 - val_loss: 2.5421 - v

In [118]:
print('train accuracy: %.2f%%' % (np.mean(hist4.history['accuracy'])*100))

train accuracy: 88.88%


In [49]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [50]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,SR1129,2,2,4.821690e-07,0.000001,9.999983e-01
1,p0017kpresabs_qual,CA105,2,2,2.652370e-03,0.000049,9.972990e-01
2,p0017kpresabs_qual,NRS175,1,1,1.198157e-02,0.988018,5.232074e-09
3,p0017kpresabs_qual,EUH25,2,1,1.388957e-01,0.735954,1.251503e-01
4,p0017kpresabs_qual,NRS070,2,1,7.231966e-03,0.810317,1.824512e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS247,1,2,2.957360e-02,0.385730,5.846961e-01
604,p0040presabsSTCC_qual,NRS215,1,2,3.196635e-03,0.445057,5.517461e-01
605,p0040presabsSTCC_qual,SR4152,2,2,4.319404e-02,0.028984,9.278219e-01
606,p0040presabsSTCC_qual,NRS035,1,1,4.902312e-03,0.680427,3.146706e-01


In [51]:
y_prob4 = df_proba4[df_proba4['phage']=='p0040kpresabsSTCC_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[1.68059540e-01, 7.16124000e-01, 1.15816420e-01],
       [5.09222800e-02, 9.47701450e-01, 1.37626380e-03],
       [6.81654200e-04, 6.34515700e-03, 9.92973200e-01],
       [1.52505260e-01, 9.12068900e-02, 7.56287900e-01],
       [4.15468160e-01, 3.67857370e-02, 5.47746100e-01],
       [8.04308300e-01, 1.31785510e-01, 6.39062750e-02],
       [8.97244500e-02, 1.04419350e-01, 8.05856200e-01],
       [9.98538260e-01, 3.42871440e-04, 1.11897100e-03],
       [6.66289870e-03, 2.27762220e-02, 9.70560850e-01],
       [5.32605300e-03, 7.97545000e-01, 1.97128880e-01],
       [2.52443480e-02, 5.08586350e-01, 4.66169330e-01],
       [1.46642040e-01, 8.73120900e-03, 8.44626700e-01],
       [8.14565700e-01, 7.37342400e-03, 1.78060860e-01],
       [4.69018500e-02, 5.13109150e-02, 9.01787200e-01],
       [9.49106040e-01, 3.88353870e-07, 5.08936830e-02],
       [1.10283960e-02, 9.30608870e-01, 5.83627630e-02],
       [7.83691400e-01, 1.76472220e-01, 3.98363140e-02],
       [9.39512670e-01, 6.34761

In [52]:
ovo4 = rocauc_ovo(y_test, y_prob4, average="macro", multi_class="ovo")
ovo4

0.6147989776465369

In [53]:
ovr4 = rocauc_ovr(y_test, y_prob4, average="macro", multi_class="ovr")
ovr4

0.6147989776465369

In [54]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.6544303174901978

In [55]:
np.std(ovos)

0.02356696605942204

In [56]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.6544303174901978

In [57]:
np.std(ovrs)

0.02356696605942204

In [119]:
accs = [acc_test1, acc_test2, acc_test3, acc_test4]

In [120]:
mean = np.mean(accs)
print('test accuracy mean: %.2f%%' % (mean*100))

test accuracy mean: 52.96%


In [121]:
std = np.std(accs)
print('test accuracy standard deviation:', std)

test accuracy standard deviation: 0.023491534733447825


In [122]:
accs_train = [np.mean(hist1.history['accuracy']), np.mean(hist2.history['accuracy']), np.mean(hist3.history['accuracy']),
             np.mean(hist4.history['accuracy'])]

In [123]:
mean_train = np.mean(accs_train)
print('train accuracy mean: %.2f%%' % (mean_train*100))

train accuracy mean: 88.08%


In [124]:
std_train = np.std(accs_train)
print('train accuracy standard deviation:', std_train)

train accuracy standard deviation: 0.009800938


In [108]:
############ Feature selection using lasso ##########

In [58]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [59]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
selection.fit(X.loc[:, X.columns != 'id'], y)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [60]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [61]:
X_train_features = np.vstack((names, X.loc[:, X.columns != 'id']))
X_train_features = pd.DataFrame(X_train_features)

In [62]:
sel_features = X_train_features.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 182
selected features: 119


In [63]:
cols = sel_features.values
cols.reshape((1, -1))

array([[  0,   1,   3,   4,   7,  12,  13,  15,  16,  17,  18,  19,  20,
         21,  22,  23,  24,  26,  27,  31,  32,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  47,  49,  50,  51,  54,  55,  56,  58,
         59,  61,  62,  63,  64,  65,  66,  67,  68,  70,  71,  72,  75,
         77,  78,  79,  80,  81,  82,  83,  84,  87,  88,  90,  92,  93,
         94,  96,  97,  98,  99, 100, 101, 106, 107, 109, 110, 111, 112,
        113, 114, 116, 117, 121, 124, 125, 126, 128, 130, 131, 132, 133,
        134, 135, 136, 138, 139, 142, 143, 144, 146, 147, 148, 149, 152,
        153, 157, 158, 160, 161, 163, 166, 171, 174, 175, 176, 178, 179,
        180, 181]])

In [64]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT',
       'TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT',
       'TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG',
       'TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC',
       'TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC',
       'TTTCACTGCCTGT',
       'TTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTT',
       'TTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTGA',
       'TTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTT',
       'TTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAG

In [65]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']

In [66]:
df_sel['strain'] = X.iloc[:,0]

In [67]:
df_sel

,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCACTGCCTGT,TTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTT,TTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTGA,TTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTT,TTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCA,...,AGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTT,ACTTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTT,ACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTGATTGT,ACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTT,AACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTT,AACTGCTCATACAGTTAAAACAGCACAAACTGCTCATACAGTTAAAACAGCACAAACTGCTCAAGAACAAAATAAAGTTCAAACACCTGTTAAAGATGTT,ST,CC,pheno,strain
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,5,5,2,107
1,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,8,8,0,109
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,5,5,2,115
3,0,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,5,5,2,120335
4,0,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,5,5,2,120337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,5,5,2,SR4152
249,0,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,3812,5,1,SR4153
250,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,5,5,1,SR4155
251,1,1,1,1,1,0,1,1,1,1,...,1,1,1,1,1,1,5,5,2,SR4156


In [68]:
X_sel = df_sel.loc[:, df_sel.columns != 'pheno']
y_sel = df_sel['pheno']
print(X_sel.shape, y_sel.shape, df_sel.shape)

(253, 120) (253,) (253, 121)


In [69]:
df_sel['pheno'].value_counts()

2    95
1    94
0    64
Name: pheno, dtype: int64

In [70]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_sel)

In [71]:
dat5 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat5['test'] = y_sel_test

In [72]:
dat5

,strain,test
251,SR4156,2
17,BCH-SA-02,0
158,NRS180,2
232,NY417,1
47,CFBREBSa110,2
...,...,...
62,CFBREBSa131,2
138,NRS112,2
21,BCH-SA-06,0
15,834N,2


In [73]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [74]:
#### neural network
model_sel = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [82]:
model_sel.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [83]:
model_sel.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 691us/step - loss: 17.6957 - accuracy: 0.3164 - val_loss: 7.4558 - val_accuracy: 0.2500
Epoch 2/1000
177/177 [==============================] - 0s 87us/step - loss: 15.3592 - accuracy: 0.3164 - val_loss: 6.3558 - val_accuracy: 0.3421
Epoch 3/1000
177/177 [==============================] - 0s 110us/step - loss: 13.2131 - accuracy: 0.4011 - val_loss: 6.2040 - val_accuracy: 0.3816
Epoch 4/1000
177/177 [==============================] - 0s 128us/step - loss: 12.1601 - accuracy: 0.3672 - val_loss: 4.9679 - val_accuracy: 0.3553
Epoch 5/1000
177/177 [==============================] - 0s 121us/step - loss: 10.3705 - accuracy: 0.3107 - val_loss: 4.3355 - val_accuracy: 0.3947
Epoch 6/1000
177/177 [==============================] - 0s 162us/step - loss: 8.9570 - accuracy: 0.3785 - val_loss: 3.7675 - val_accuracy: 0.3421
Epoch 7/1000
177/177 [==============================] - 0s 155us/step - los

177/177 [==============================] - 0s 167us/step - loss: 0.7467 - accuracy: 0.6723 - val_loss: 1.0740 - val_accuracy: 0.4474
Epoch 112/1000
177/177 [==============================] - 0s 148us/step - loss: 0.7884 - accuracy: 0.6780 - val_loss: 1.0745 - val_accuracy: 0.4474
Epoch 113/1000
177/177 [==============================] - 0s 111us/step - loss: 0.7887 - accuracy: 0.6667 - val_loss: 1.0685 - val_accuracy: 0.4474
Epoch 114/1000
177/177 [==============================] - 0s 188us/step - loss: 0.7429 - accuracy: 0.6893 - val_loss: 1.0778 - val_accuracy: 0.4474
Epoch 115/1000
177/177 [==============================] - 0s 178us/step - loss: 0.7429 - accuracy: 0.6723 - val_loss: 1.1072 - val_accuracy: 0.4211
Epoch 116/1000
177/177 [==============================] - 0s 165us/step - loss: 0.7317 - accuracy: 0.6667 - val_loss: 1.1969 - val_accuracy: 0.4211
Epoch 117/1000
177/177 [==============================] - 0s 173us/step - loss: 0.7856 - accuracy: 0.6497 - val_loss: 1.0903 - 

177/177 [==============================] - 0s 174us/step - loss: 0.6018 - accuracy: 0.7797 - val_loss: 1.3257 - val_accuracy: 0.4079
Epoch 222/1000
177/177 [==============================] - 0s 159us/step - loss: 0.7120 - accuracy: 0.7458 - val_loss: 1.1767 - val_accuracy: 0.4079
Epoch 223/1000
177/177 [==============================] - 0s 182us/step - loss: 0.6695 - accuracy: 0.7571 - val_loss: 1.1879 - val_accuracy: 0.3947
Epoch 224/1000
177/177 [==============================] - 0s 148us/step - loss: 0.7280 - accuracy: 0.7571 - val_loss: 1.2460 - val_accuracy: 0.4079
Epoch 225/1000
177/177 [==============================] - 0s 82us/step - loss: 0.5546 - accuracy: 0.7627 - val_loss: 1.1763 - val_accuracy: 0.3947
Epoch 226/1000
177/177 [==============================] - 0s 82us/step - loss: 0.5946 - accuracy: 0.7514 - val_loss: 1.3535 - val_accuracy: 0.4211
Epoch 227/1000
177/177 [==============================] - 0s 88us/step - loss: 0.6590 - accuracy: 0.7514 - val_loss: 1.1863 - val

Epoch 332/1000
177/177 [==============================] - 0s 151us/step - loss: 0.6152 - accuracy: 0.8079 - val_loss: 1.2878 - val_accuracy: 0.4079
Epoch 333/1000
177/177 [==============================] - 0s 152us/step - loss: 0.4854 - accuracy: 0.8475 - val_loss: 1.4698 - val_accuracy: 0.4342
Epoch 334/1000
177/177 [==============================] - 0s 187us/step - loss: 0.5997 - accuracy: 0.8305 - val_loss: 1.2812 - val_accuracy: 0.4079
Epoch 335/1000
177/177 [==============================] - 0s 143us/step - loss: 0.5724 - accuracy: 0.8023 - val_loss: 1.2945 - val_accuracy: 0.4211
Epoch 336/1000
177/177 [==============================] - 0s 103us/step - loss: 0.4971 - accuracy: 0.8305 - val_loss: 1.3376 - val_accuracy: 0.4079
Epoch 337/1000
177/177 [==============================] - 0s 101us/step - loss: 0.5521 - accuracy: 0.8305 - val_loss: 1.4371 - val_accuracy: 0.4079
Epoch 338/1000
177/177 [==============================] - 0s 87us/step - loss: 0.5225 - accuracy: 0.8192 - val_l

177/177 [==============================] - 0s 126us/step - loss: 0.4549 - accuracy: 0.8362 - val_loss: 1.3981 - val_accuracy: 0.4474
Epoch 443/1000
177/177 [==============================] - 0s 123us/step - loss: 0.4441 - accuracy: 0.8305 - val_loss: 1.4206 - val_accuracy: 0.4211
Epoch 444/1000
177/177 [==============================] - 0s 109us/step - loss: 0.4213 - accuracy: 0.8305 - val_loss: 1.4118 - val_accuracy: 0.4211
Epoch 445/1000
177/177 [==============================] - 0s 118us/step - loss: 0.5015 - accuracy: 0.8192 - val_loss: 1.4384 - val_accuracy: 0.3947
Epoch 446/1000
177/177 [==============================] - 0s 261us/step - loss: 0.4511 - accuracy: 0.8362 - val_loss: 1.3914 - val_accuracy: 0.4079
Epoch 447/1000
177/177 [==============================] - 0s 97us/step - loss: 0.4375 - accuracy: 0.8305 - val_loss: 1.5201 - val_accuracy: 0.4342
Epoch 448/1000
177/177 [==============================] - 0s 101us/step - loss: 0.5628 - accuracy: 0.8249 - val_loss: 1.3943 - v

...

Epoch 608/1000
177/177 [==============================] - 0s 107us/step - loss: 0.5325 - accuracy: 0.8531 - val_loss: 1.7516 - val_accuracy: 0.4342
Epoch 609/1000
177/177 [==============================] - ETA: 0s - loss: 0.3897 - accuracy: 0.84 - 0s 93us/step - loss: 0.4289 - accuracy: 0.8588 - val_loss: 1.6256 - val_accuracy: 0.4211
Epoch 610/1000
177/177 [==============================] - 0s 80us/step - loss: 0.7238 - accuracy: 0.8192 - val_loss: 1.5851 - val_accuracy: 0.4342
Epoch 611/1000
177/177 [==============================] - 0s 79us/step - loss: 0.3867 - accuracy: 0.8475 - val_loss: 1.6877 - val_accuracy: 0.4605
Epoch 612/1000
177/177 [==============================] - 0s 112us/step - loss: 0.4608 - accuracy: 0.8418 - val_loss: 1.6490 - val_accuracy: 0.4211
Epoch 613/1000
177/177 [==============================] - 0s 138us/step - loss: 0.4360 - accuracy: 0.8475 - val_loss: 1.5794 - val_accuracy: 0.4342
Epoch 614/1000
177/177 [==============================] - 0s 137us/step -

Epoch 719/1000
177/177 [==============================] - 0s 101us/step - loss: 0.3422 - accuracy: 0.8757 - val_loss: 1.6918 - val_accuracy: 0.4474
Epoch 720/1000
177/177 [==============================] - 0s 99us/step - loss: 0.3742 - accuracy: 0.8531 - val_loss: 1.7283 - val_accuracy: 0.4079
Epoch 721/1000
177/177 [==============================] - 0s 96us/step - loss: 0.3286 - accuracy: 0.8701 - val_loss: 1.7433 - val_accuracy: 0.4211
Epoch 722/1000
177/177 [==============================] - 0s 92us/step - loss: 0.3703 - accuracy: 0.8701 - val_loss: 1.7204 - val_accuracy: 0.4211
Epoch 723/1000
177/177 [==============================] - 0s 86us/step - loss: 0.3331 - accuracy: 0.8588 - val_loss: 1.7293 - val_accuracy: 0.4079
Epoch 724/1000
177/177 [==============================] - 0s 77us/step - loss: 0.3430 - accuracy: 0.8644 - val_loss: 1.7449 - val_accuracy: 0.4079
Epoch 725/1000
177/177 [==============================] - 0s 142us/step - loss: 0.3435 - accuracy: 0.8757 - val_loss:

177/177 [==============================] - 0s 96us/step - loss: 0.3540 - accuracy: 0.8701 - val_loss: 1.8314 - val_accuracy: 0.4079
Epoch 830/1000
177/177 [==============================] - 0s 106us/step - loss: 0.3402 - accuracy: 0.8644 - val_loss: 1.8352 - val_accuracy: 0.4079
Epoch 831/1000
177/177 [==============================] - 0s 147us/step - loss: 0.3126 - accuracy: 0.8757 - val_loss: 1.8668 - val_accuracy: 0.4079
Epoch 832/1000
177/177 [==============================] - 0s 114us/step - loss: 0.3257 - accuracy: 0.8814 - val_loss: 1.8471 - val_accuracy: 0.3947
Epoch 833/1000
177/177 [==============================] - 0s 108us/step - loss: 0.3224 - accuracy: 0.8701 - val_loss: 1.8480 - val_accuracy: 0.3947
Epoch 834/1000
177/177 [==============================] - 0s 104us/step - loss: 0.3138 - accuracy: 0.8757 - val_loss: 1.8285 - val_accuracy: 0.4211
Epoch 835/1000
177/177 [==============================] - 0s 117us/step - loss: 0.3298 - accuracy: 0.8814 - val_loss: 1.8635 - v

177/177 [==============================] - 0s 237us/step - loss: 0.3091 - accuracy: 0.8814 - val_loss: 1.9628 - val_accuracy: 0.4211
Epoch 940/1000
177/177 [==============================] - 0s 73us/step - loss: 0.3249 - accuracy: 0.8870 - val_loss: 1.9747 - val_accuracy: 0.4211
Epoch 941/1000
177/177 [==============================] - 0s 66us/step - loss: 0.2940 - accuracy: 0.8870 - val_loss: 1.9767 - val_accuracy: 0.3947
Epoch 942/1000
177/177 [==============================] - 0s 65us/step - loss: 0.2825 - accuracy: 0.8814 - val_loss: 1.9663 - val_accuracy: 0.4079
Epoch 943/1000
177/177 [==============================] - 0s 99us/step - loss: 0.3257 - accuracy: 0.8870 - val_loss: 1.9900 - val_accuracy: 0.4211
Epoch 944/1000
177/177 [==============================] - 0s 62us/step - loss: 0.3299 - accuracy: 0.8927 - val_loss: 1.9935 - val_accuracy: 0.4079
Epoch 945/1000
177/177 [==============================] - 0s 64us/step - loss: 0.3114 - accuracy: 0.8814 - val_loss: 1.9878 - val_ac

Epoch 995/1000
177/177 [==============================] - 0s 129us/step - loss: 0.2897 - accuracy: 0.8870 - val_loss: 2.0655 - val_accuracy: 0.3947
Epoch 996/1000
177/177 [==============================] - 0s 122us/step - loss: 0.3392 - accuracy: 0.8927 - val_loss: 2.1028 - val_accuracy: 0.3947
Epoch 997/1000
177/177 [==============================] - 0s 128us/step - loss: 0.2857 - accuracy: 0.8983 - val_loss: 2.0295 - val_accuracy: 0.3816
Epoch 998/1000
177/177 [==============================] - 0s 140us/step - loss: 0.4302 - accuracy: 0.8644 - val_loss: 2.0377 - val_accuracy: 0.4211
Epoch 999/1000
177/177 [==============================] - 0s 151us/step - loss: 0.3885 - accuracy: 0.8757 - val_loss: 2.1112 - val_accuracy: 0.3947
Epoch 1000/1000
177/177 [==============================] - 0s 170us/step - loss: 0.2720 - accuracy: 0.8927 - val_loss: 2.0247 - val_accuracy: 0.3816


In [168]:
acc_test_sel = model_sel.evaluate(X_sel_test, y_sel_test)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_sel*100))

76/76 [==============================] - 0s 162us/step
over-sampling test accuracy: 48.68%


In [84]:
pred5 = model_sel.predict_classes(X_sel_test)
pred5

array([2, 1, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 0, 0, 0, 0, 2, 2, 1,
       0, 0, 2, 1, 0, 0, 1, 0, 0, 2, 2, 1, 2, 0, 1, 2, 2, 1, 0, 0, 1, 1,
       2, 2, 2, 0, 1, 1, 1, 2, 1, 2, 2, 0, 1, 2, 2, 2, 0, 2, 1, 1, 2, 0,
       1, 1, 2, 2, 2, 0, 1, 1, 2, 2])

In [85]:
dat5['pred'] = pred5
dat5

,strain,test,pred
251,SR4156,2,2
17,BCH-SA-02,0,1
158,NRS180,2,1
232,NY417,1,2
47,CFBREBSa110,2,2
...,...,...,...
62,CFBREBSa131,2,0
138,NRS112,2,1
21,BCH-SA-06,0,1
15,834N,2,2


In [86]:
proba5 = model_sel.predict_proba(X_sel_test)
dat_proba5 = pd.DataFrame(proba5)

In [87]:
dat_proba5

,0,1,2
0,0.271728,0.000001,0.728271
1,0.332337,0.618436,0.049228
2,0.121654,0.481263,0.397083
3,0.323477,0.336698,0.339825
4,0.262122,0.098647,0.639231
...,...,...,...
71,0.652406,0.011739,0.335855
72,0.033130,0.964271,0.002599
73,0.332337,0.618436,0.049228
74,0.069572,0.005024,0.925404


In [88]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [89]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p40ST.csv", index = False,
         header=None)

In [172]:
hist_sel = model_sel.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 122us/step - loss: 0.3199 - accuracy: 0.8983 - val_loss: 1.8983 - val_accuracy: 0.4868
Epoch 2/1000
177/177 [==============================] - 0s 98us/step - loss: 0.3569 - accuracy: 0.8644 - val_loss: 1.8284 - val_accuracy: 0.4605
Epoch 3/1000
177/177 [==============================] - 0s 166us/step - loss: 0.3077 - accuracy: 0.8757 - val_loss: 1.7959 - val_accuracy: 0.4737
Epoch 4/1000
177/177 [==============================] - 0s 140us/step - loss: 0.3162 - accuracy: 0.8701 - val_loss: 1.8022 - val_accuracy: 0.4868
Epoch 5/1000
177/177 [==============================] - 0s 162us/step - loss: 0.3273 - accuracy: 0.8644 - val_loss: 1.7707 - val_accuracy: 0.5263
Epoch 6/1000
177/177 [==============================] - 0s 134us/step - loss: 0.3317 - accuracy: 0.8701 - val_loss: 1.7946 - val_accuracy: 0.4868
Epoch 7/1000
177/177 [==============================] - 0s 101us/step - loss: 0.

Epoch 112/1000
177/177 [==============================] - 0s 364us/step - loss: 0.3078 - accuracy: 0.8588 - val_loss: 1.9280 - val_accuracy: 0.4737
Epoch 113/1000
177/177 [==============================] - 0s 187us/step - loss: 0.3225 - accuracy: 0.8644 - val_loss: 1.8875 - val_accuracy: 0.4868
Epoch 114/1000
177/177 [==============================] - 0s 218us/step - loss: 0.2996 - accuracy: 0.8701 - val_loss: 1.9400 - val_accuracy: 0.4342
Epoch 115/1000
177/177 [==============================] - 0s 166us/step - loss: 0.3211 - accuracy: 0.8757 - val_loss: 1.8775 - val_accuracy: 0.4868
Epoch 116/1000
177/177 [==============================] - 0s 155us/step - loss: 0.3114 - accuracy: 0.8814 - val_loss: 1.8784 - val_accuracy: 0.4868
Epoch 117/1000
177/177 [==============================] - 0s 115us/step - loss: 0.3976 - accuracy: 0.8588 - val_loss: 1.9090 - val_accuracy: 0.4474
Epoch 118/1000
177/177 [==============================] - 0s 170us/step - loss: 0.2933 - accuracy: 0.8644 - val_

177/177 [==============================] - 0s 189us/step - loss: 0.3438 - accuracy: 0.8814 - val_loss: 1.9660 - val_accuracy: 0.4868
Epoch 223/1000
177/177 [==============================] - 0s 176us/step - loss: 0.2664 - accuracy: 0.8983 - val_loss: 2.0082 - val_accuracy: 0.4474
Epoch 224/1000
177/177 [==============================] - 0s 108us/step - loss: 0.3872 - accuracy: 0.8814 - val_loss: 2.0117 - val_accuracy: 0.4474
Epoch 225/1000
177/177 [==============================] - 0s 98us/step - loss: 0.2833 - accuracy: 0.8814 - val_loss: 2.0029 - val_accuracy: 0.4737
Epoch 226/1000
177/177 [==============================] - 0s 121us/step - loss: 0.3824 - accuracy: 0.8701 - val_loss: 1.9740 - val_accuracy: 0.5263
Epoch 227/1000
177/177 [==============================] - 0s 101us/step - loss: 0.2883 - accuracy: 0.8983 - val_loss: 2.1259 - val_accuracy: 0.4605
Epoch 228/1000
177/177 [==============================] - 0s 111us/step - loss: 0.4511 - accuracy: 0.8757 - val_loss: 2.0185 - v

177/177 [==============================] - 0s 120us/step - loss: 0.3324 - accuracy: 0.8927 - val_loss: 2.1570 - val_accuracy: 0.4474
Epoch 333/1000
177/177 [==============================] - 0s 240us/step - loss: 0.3796 - accuracy: 0.8644 - val_loss: 2.1094 - val_accuracy: 0.4605
Epoch 334/1000
177/177 [==============================] - 0s 136us/step - loss: 0.3403 - accuracy: 0.8814 - val_loss: 2.1713 - val_accuracy: 0.4737
Epoch 335/1000
177/177 [==============================] - 0s 141us/step - loss: 0.3866 - accuracy: 0.8814 - val_loss: 2.2077 - val_accuracy: 0.4868
Epoch 336/1000
177/177 [==============================] - 0s 174us/step - loss: 0.4239 - accuracy: 0.8814 - val_loss: 2.1420 - val_accuracy: 0.4342
Epoch 337/1000
177/177 [==============================] - 0s 125us/step - loss: 0.3659 - accuracy: 0.8588 - val_loss: 2.0918 - val_accuracy: 0.4737
Epoch 338/1000
177/177 [==============================] - 0s 113us/step - loss: 0.3803 - accuracy: 0.8757 - val_loss: 2.1095 - 

177/177 [==============================] - 0s 99us/step - loss: 0.2959 - accuracy: 0.8757 - val_loss: 2.1889 - val_accuracy: 0.4868
Epoch 443/1000
177/177 [==============================] - 0s 110us/step - loss: 0.2905 - accuracy: 0.8644 - val_loss: 2.1788 - val_accuracy: 0.4605
Epoch 444/1000
177/177 [==============================] - 0s 143us/step - loss: 0.3600 - accuracy: 0.8701 - val_loss: 2.2044 - val_accuracy: 0.4605
Epoch 445/1000
177/177 [==============================] - 0s 112us/step - loss: 0.3053 - accuracy: 0.8983 - val_loss: 2.2459 - val_accuracy: 0.4737
Epoch 446/1000
177/177 [==============================] - 0s 227us/step - loss: 0.3140 - accuracy: 0.8927 - val_loss: 2.1718 - val_accuracy: 0.4868
Epoch 447/1000
177/177 [==============================] - 0s 245us/step - loss: 0.2652 - accuracy: 0.8927 - val_loss: 2.1840 - val_accuracy: 0.4605
Epoch 448/1000
177/177 [==============================] - 0s 347us/step - loss: 0.2782 - accuracy: 0.8927 - val_loss: 2.1719 - v

177/177 [==============================] - 0s 204us/step - loss: 0.3185 - accuracy: 0.8814 - val_loss: 2.2815 - val_accuracy: 0.5263
Epoch 553/1000
177/177 [==============================] - 0s 295us/step - loss: 0.3043 - accuracy: 0.8644 - val_loss: 2.2531 - val_accuracy: 0.4868
Epoch 554/1000
177/177 [==============================] - 0s 188us/step - loss: 0.3616 - accuracy: 0.8870 - val_loss: 2.2714 - val_accuracy: 0.4868
Epoch 555/1000
177/177 [==============================] - 0s 368us/step - loss: 0.2526 - accuracy: 0.8870 - val_loss: 2.3362 - val_accuracy: 0.4342
Epoch 556/1000
177/177 [==============================] - 0s 576us/step - loss: 0.3903 - accuracy: 0.8814 - val_loss: 2.3236 - val_accuracy: 0.4737
Epoch 557/1000
177/177 [==============================] - 0s 255us/step - loss: 0.3500 - accuracy: 0.8870 - val_loss: 2.2658 - val_accuracy: 0.4737
Epoch 558/1000
177/177 [==============================] - 0s 291us/step - loss: 0.2682 - accuracy: 0.8757 - val_loss: 2.2891 - 

177/177 [==============================] - 0s 103us/step - loss: 0.3109 - accuracy: 0.8927 - val_loss: 2.3595 - val_accuracy: 0.4737
Epoch 663/1000
177/177 [==============================] - 0s 120us/step - loss: 0.2668 - accuracy: 0.8814 - val_loss: 2.3789 - val_accuracy: 0.4474
Epoch 664/1000
177/177 [==============================] - 0s 126us/step - loss: 0.2797 - accuracy: 0.8814 - val_loss: 2.4307 - val_accuracy: 0.4737
Epoch 665/1000
177/177 [==============================] - 0s 106us/step - loss: 0.2769 - accuracy: 0.8870 - val_loss: 2.3879 - val_accuracy: 0.4868
Epoch 666/1000
177/177 [==============================] - 0s 180us/step - loss: 0.2614 - accuracy: 0.8927 - val_loss: 2.3547 - val_accuracy: 0.4605
Epoch 667/1000
177/177 [==============================] - 0s 114us/step - loss: 0.2635 - accuracy: 0.8927 - val_loss: 2.3837 - val_accuracy: 0.4605
Epoch 668/1000
177/177 [==============================] - 0s 139us/step - loss: 0.2952 - accuracy: 0.8927 - val_loss: 2.4842 - 

177/177 [==============================] - 0s 108us/step - loss: 0.2817 - accuracy: 0.8870 - val_loss: 2.4301 - val_accuracy: 0.4737
Epoch 773/1000
177/177 [==============================] - 0s 134us/step - loss: 0.2692 - accuracy: 0.8927 - val_loss: 2.4565 - val_accuracy: 0.4605
Epoch 774/1000
177/177 [==============================] - 0s 160us/step - loss: 0.3398 - accuracy: 0.8814 - val_loss: 2.4904 - val_accuracy: 0.4342
Epoch 775/1000
177/177 [==============================] - 0s 204us/step - loss: 0.3259 - accuracy: 0.8701 - val_loss: 2.4950 - val_accuracy: 0.4474
Epoch 776/1000
177/177 [==============================] - 0s 146us/step - loss: 0.2955 - accuracy: 0.8644 - val_loss: 2.4699 - val_accuracy: 0.5132
Epoch 777/1000
177/177 [==============================] - 0s 159us/step - loss: 0.2920 - accuracy: 0.8870 - val_loss: 2.5090 - val_accuracy: 0.4737
Epoch 778/1000
177/177 [==============================] - 0s 143us/step - loss: 0.2848 - accuracy: 0.8757 - val_loss: 2.4222 - 

177/177 [==============================] - 0s 119us/step - loss: 0.2679 - accuracy: 0.8870 - val_loss: 2.5371 - val_accuracy: 0.4342
Epoch 883/1000
177/177 [==============================] - 0s 135us/step - loss: 0.2415 - accuracy: 0.8814 - val_loss: 2.5722 - val_accuracy: 0.4474
Epoch 884/1000
177/177 [==============================] - 0s 212us/step - loss: 0.3000 - accuracy: 0.8814 - val_loss: 2.5284 - val_accuracy: 0.4342
Epoch 885/1000
177/177 [==============================] - 0s 147us/step - loss: 0.2790 - accuracy: 0.8870 - val_loss: 2.5137 - val_accuracy: 0.5132
Epoch 886/1000
177/177 [==============================] - 0s 138us/step - loss: 0.2626 - accuracy: 0.8927 - val_loss: 2.5508 - val_accuracy: 0.4605
Epoch 887/1000
177/177 [==============================] - 0s 112us/step - loss: 0.2533 - accuracy: 0.8870 - val_loss: 2.5334 - val_accuracy: 0.4737
Epoch 888/1000
177/177 [==============================] - 0s 106us/step - loss: 0.2405 - accuracy: 0.8983 - val_loss: 2.5047 - 

177/177 [==============================] - 0s 119us/step - loss: 0.3923 - accuracy: 0.8757 - val_loss: 2.6790 - val_accuracy: 0.4474
Epoch 993/1000
177/177 [==============================] - 0s 146us/step - loss: 0.3569 - accuracy: 0.8814 - val_loss: 2.6568 - val_accuracy: 0.4211
Epoch 994/1000
177/177 [==============================] - 0s 167us/step - loss: 0.3129 - accuracy: 0.8644 - val_loss: 2.5993 - val_accuracy: 0.4737
Epoch 995/1000
177/177 [==============================] - 0s 159us/step - loss: 0.2666 - accuracy: 0.8757 - val_loss: 2.5944 - val_accuracy: 0.4737
Epoch 996/1000
177/177 [==============================] - 0s 123us/step - loss: 0.2284 - accuracy: 0.8927 - val_loss: 2.6698 - val_accuracy: 0.4737
Epoch 997/1000
177/177 [==============================] - 0s 150us/step - loss: 0.2933 - accuracy: 0.8870 - val_loss: 2.6047 - val_accuracy: 0.4737
Epoch 998/1000
177/177 [==============================] - 0s 209us/step - loss: 0.2365 - accuracy: 0.8983 - val_loss: 2.6726 - 

In [173]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel.history['accuracy'])*100))

train accuracy: 88.13%


In [75]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [76]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NY360,2,1,3.848032e-03,0.996096,0.000056
1,p0017kpresabs_qual,NRS113,0,0,8.441349e-01,0.000015,0.155850
2,p0017kpresabs_qual,120337,2,2,1.466081e-02,0.005713,0.979626
3,p0017kpresabs_qual,CFBREBSa127,1,0,9.985392e-01,0.000350,0.001111
4,p0017kpresabs_qual,GA27,2,2,1.665667e-08,0.000005,0.999995
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,CFBREBSa131,2,1,6.617341e-03,0.565843,0.427540
604,p0040presabsSTCC_qual,NRS112,2,2,1.055758e-03,0.045611,0.953333
605,p0040presabsSTCC_qual,BCH-SA-06,0,0,8.747076e-01,0.071162,0.054130
606,p0040presabsSTCC_qual,834N,2,2,2.593112e-06,0.000013,0.999985


In [77]:
y_prob5 = df_proba5[df_proba5['phage']=='p0040kpresabsSTCC_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[2.71728130e-01, 1.37542570e-06, 7.28270530e-01],
       [3.32336720e-01, 6.18435740e-01, 4.92275540e-02],
       [1.21654265e-01, 4.81262680e-01, 3.97083000e-01],
       [3.23477100e-01, 3.36698060e-01, 3.39824860e-01],
       [2.62121830e-01, 9.86471000e-02, 6.39231100e-01],
       [2.62121830e-01, 9.86471000e-02, 6.39231100e-01],
       [1.34355890e-03, 8.99865000e-01, 9.87914200e-02],
       [1.90585170e-01, 5.66628880e-05, 8.09358200e-01],
       [1.32375180e-03, 4.98363800e-04, 9.98177900e-01],
       [1.94013150e-01, 8.05982600e-01, 4.18418800e-06],
       [1.20239790e-01, 6.74637260e-02, 8.12296500e-01],
       [2.85753850e-01, 6.95903000e-02, 6.44655800e-01],
       [6.42184800e-02, 6.28409860e-01, 3.07371700e-01],
       [6.43240350e-04, 9.99318600e-01, 3.81064240e-05],
       [1.25677990e-03, 8.94661000e-01, 1.04082270e-01],
       [5.19948900e-01, 5.80648200e-02, 4.21986220e-01],
       [9.37373800e-01, 9.17115900e-03, 5.34549950e-02],
       [9.91059360e-01, 5.01762

In [78]:
ovo5 = rocauc_ovo(y_sel_test, y_prob5, average="macro", multi_class="ovo")
ovo5

0.5674621438972668

In [79]:
ovr5 = rocauc_ovr(y_sel_test, y_prob5, average="macro", multi_class="ovr")
ovr5

0.5674621438972668

In [80]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_sel)

In [81]:
dat6 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat6['test'] = y_sel_test

In [82]:
dat6

,strain,test
152,NRS169,1
210,NRS249,2
218,NRS262,2
238,SR1746,1
113,NRS029,1
...,...,...
96,GA27,2
95,GA231,2
237,SR1287,0
14,506,2


In [83]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [94]:
model_sel2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [95]:
model_sel2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [96]:
model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 779us/step - loss: 7.2492 - accuracy: 0.3220 - val_loss: 6.9562 - val_accuracy: 0.3158
Epoch 2/1000
177/177 [==============================] - 0s 90us/step - loss: 6.1873 - accuracy: 0.3277 - val_loss: 6.3161 - val_accuracy: 0.3158
Epoch 3/1000
177/177 [==============================] - 0s 112us/step - loss: 5.3768 - accuracy: 0.3616 - val_loss: 5.6144 - val_accuracy: 0.3947
Epoch 4/1000
177/177 [==============================] - 0s 138us/step - loss: 4.9851 - accuracy: 0.4124 - val_loss: 4.9740 - val_accuracy: 0.4079
Epoch 5/1000
177/177 [==============================] - 0s 165us/step - loss: 4.3556 - accuracy: 0.4181 - val_loss: 4.3762 - val_accuracy: 0.3947
Epoch 6/1000
177/177 [==============================] - 0s 228us/step - loss: 3.8286 - accuracy: 0.3842 - val_loss: 3.7263 - val_accuracy: 0.3684
Epoch 7/1000
177/177 [==============================] - 0s 202us/step - loss: 3.

Epoch 112/1000
177/177 [==============================] - 0s 131us/step - loss: 0.8538 - accuracy: 0.6497 - val_loss: 1.0738 - val_accuracy: 0.5132
Epoch 113/1000
177/177 [==============================] - 0s 142us/step - loss: 0.7688 - accuracy: 0.6667 - val_loss: 0.9757 - val_accuracy: 0.5132
Epoch 114/1000
177/177 [==============================] - 0s 135us/step - loss: 0.8423 - accuracy: 0.6723 - val_loss: 0.9841 - val_accuracy: 0.4868
Epoch 115/1000
177/177 [==============================] - 0s 128us/step - loss: 0.8229 - accuracy: 0.6497 - val_loss: 1.0213 - val_accuracy: 0.5132
Epoch 116/1000
177/177 [==============================] - 0s 175us/step - loss: 0.7863 - accuracy: 0.6328 - val_loss: 1.0013 - val_accuracy: 0.5526
Epoch 117/1000
177/177 [==============================] - 0s 179us/step - loss: 0.7728 - accuracy: 0.6836 - val_loss: 1.0211 - val_accuracy: 0.5263
Epoch 118/1000
177/177 [==============================] - 0s 201us/step - loss: 0.7608 - accuracy: 0.6667 - val_

177/177 [==============================] - 0s 139us/step - loss: 0.6209 - accuracy: 0.7627 - val_loss: 1.1150 - val_accuracy: 0.5132
Epoch 223/1000
177/177 [==============================] - 0s 157us/step - loss: 0.6230 - accuracy: 0.7797 - val_loss: 1.1162 - val_accuracy: 0.4868
Epoch 224/1000
177/177 [==============================] - 0s 126us/step - loss: 0.6243 - accuracy: 0.7627 - val_loss: 1.1127 - val_accuracy: 0.4868
Epoch 225/1000
177/177 [==============================] - 0s 101us/step - loss: 0.6374 - accuracy: 0.7514 - val_loss: 1.1109 - val_accuracy: 0.5000
Epoch 226/1000
177/177 [==============================] - 0s 136us/step - loss: 0.6390 - accuracy: 0.7458 - val_loss: 1.1084 - val_accuracy: 0.4868
Epoch 227/1000
177/177 [==============================] - 0s 162us/step - loss: 0.6258 - accuracy: 0.7740 - val_loss: 1.1354 - val_accuracy: 0.5132
Epoch 228/1000
177/177 [==============================] - 0s 217us/step - loss: 0.6223 - accuracy: 0.7684 - val_loss: 1.1133 - 

177/177 [==============================] - 0s 110us/step - loss: 0.5545 - accuracy: 0.7684 - val_loss: 1.2438 - val_accuracy: 0.5000
Epoch 333/1000
177/177 [==============================] - 0s 170us/step - loss: 0.6556 - accuracy: 0.7458 - val_loss: 1.2380 - val_accuracy: 0.4868
Epoch 334/1000
177/177 [==============================] - 0s 123us/step - loss: 0.6505 - accuracy: 0.7627 - val_loss: 1.3514 - val_accuracy: 0.4737
Epoch 335/1000
177/177 [==============================] - 0s 125us/step - loss: 0.6128 - accuracy: 0.7684 - val_loss: 1.2514 - val_accuracy: 0.5263
Epoch 336/1000
177/177 [==============================] - 0s 122us/step - loss: 0.5442 - accuracy: 0.7797 - val_loss: 1.2345 - val_accuracy: 0.5000
Epoch 337/1000
177/177 [==============================] - 0s 127us/step - loss: 0.5665 - accuracy: 0.7853 - val_loss: 1.2480 - val_accuracy: 0.5000
Epoch 338/1000
177/177 [==============================] - 0s 106us/step - loss: 0.5354 - accuracy: 0.7853 - val_loss: 1.2508 - 

Epoch 388/1000
177/177 [==============================] - 0s 70us/step - loss: 0.5080 - accuracy: 0.8079 - val_loss: 1.3058 - val_accuracy: 0.5132
Epoch 389/1000
177/177 [==============================] - 0s 64us/step - loss: 0.5296 - accuracy: 0.7853 - val_loss: 1.3262 - val_accuracy: 0.5132
Epoch 390/1000
177/177 [==============================] - 0s 72us/step - loss: 0.5080 - accuracy: 0.7966 - val_loss: 1.3251 - val_accuracy: 0.5132
Epoch 391/1000
177/177 [==============================] - 0s 68us/step - loss: 0.5174 - accuracy: 0.8136 - val_loss: 1.3170 - val_accuracy: 0.5000
Epoch 392/1000
177/177 [==============================] - 0s 75us/step - loss: 0.5178 - accuracy: 0.8079 - val_loss: 1.3089 - val_accuracy: 0.5000
Epoch 393/1000
177/177 [==============================] - 0s 76us/step - loss: 0.4985 - accuracy: 0.7966 - val_loss: 1.3350 - val_accuracy: 0.5263
Epoch 394/1000
177/177 [==============================] - 0s 75us/step - loss: 0.5208 - accuracy: 0.7910 - val_loss: 1

Epoch 499/1000
177/177 [==============================] - 0s 68us/step - loss: 0.4673 - accuracy: 0.8192 - val_loss: 1.4338 - val_accuracy: 0.5132
Epoch 500/1000
177/177 [==============================] - 0s 73us/step - loss: 0.4579 - accuracy: 0.8249 - val_loss: 1.4284 - val_accuracy: 0.5132
Epoch 501/1000
177/177 [==============================] - 0s 76us/step - loss: 0.4851 - accuracy: 0.7966 - val_loss: 1.5182 - val_accuracy: 0.5000
Epoch 502/1000
177/177 [==============================] - 0s 73us/step - loss: 0.4901 - accuracy: 0.8079 - val_loss: 1.4440 - val_accuracy: 0.5263
Epoch 503/1000
177/177 [==============================] - 0s 72us/step - loss: 0.4790 - accuracy: 0.8192 - val_loss: 1.4417 - val_accuracy: 0.5132
Epoch 504/1000
177/177 [==============================] - 0s 72us/step - loss: 0.4649 - accuracy: 0.8305 - val_loss: 1.4500 - val_accuracy: 0.5132
Epoch 505/1000
177/177 [==============================] - 0s 67us/step - loss: 0.4788 - accuracy: 0.8305 - val_loss: 1

177/177 [==============================] - 0s 108us/step - loss: 0.4474 - accuracy: 0.8305 - val_loss: 1.5847 - val_accuracy: 0.5132
Epoch 610/1000
177/177 [==============================] - 0s 94us/step - loss: 0.4606 - accuracy: 0.8192 - val_loss: 1.5552 - val_accuracy: 0.5263
Epoch 611/1000
177/177 [==============================] - 0s 92us/step - loss: 0.4256 - accuracy: 0.8249 - val_loss: 1.5249 - val_accuracy: 0.5132
Epoch 612/1000
177/177 [==============================] - 0s 116us/step - loss: 0.4161 - accuracy: 0.8305 - val_loss: 1.5847 - val_accuracy: 0.5000
Epoch 613/1000
177/177 [==============================] - 0s 117us/step - loss: 0.4382 - accuracy: 0.8192 - val_loss: 1.5751 - val_accuracy: 0.5000
Epoch 614/1000
177/177 [==============================] - 0s 94us/step - loss: 0.4665 - accuracy: 0.8192 - val_loss: 1.5871 - val_accuracy: 0.4868
Epoch 615/1000
177/177 [==============================] - 0s 120us/step - loss: 0.4849 - accuracy: 0.8136 - val_loss: 1.6539 - val

177/177 [==============================] - 0s 98us/step - loss: 0.4971 - accuracy: 0.8249 - val_loss: 1.7389 - val_accuracy: 0.5263
Epoch 720/1000
177/177 [==============================] - 0s 88us/step - loss: 0.4402 - accuracy: 0.8362 - val_loss: 1.6771 - val_accuracy: 0.5132
Epoch 721/1000
177/177 [==============================] - 0s 78us/step - loss: 0.5059 - accuracy: 0.8192 - val_loss: 1.6950 - val_accuracy: 0.5000
Epoch 722/1000
177/177 [==============================] - 0s 96us/step - loss: 0.5163 - accuracy: 0.8249 - val_loss: 1.8456 - val_accuracy: 0.4868
Epoch 723/1000
177/177 [==============================] - 0s 102us/step - loss: 0.5087 - accuracy: 0.8305 - val_loss: 1.7105 - val_accuracy: 0.5263
Epoch 724/1000
177/177 [==============================] - 0s 90us/step - loss: 0.3965 - accuracy: 0.8475 - val_loss: 1.6479 - val_accuracy: 0.5000
Epoch 725/1000
177/177 [==============================] - 0s 83us/step - loss: 0.4388 - accuracy: 0.8418 - val_loss: 1.6505 - val_ac

177/177 [==============================] - 0s 81us/step - loss: 0.4069 - accuracy: 0.8531 - val_loss: 1.9378 - val_accuracy: 0.5000
Epoch 830/1000
177/177 [==============================] - 0s 88us/step - loss: 0.4442 - accuracy: 0.8136 - val_loss: 1.8334 - val_accuracy: 0.5395
Epoch 831/1000
177/177 [==============================] - 0s 95us/step - loss: 0.3985 - accuracy: 0.8531 - val_loss: 1.8377 - val_accuracy: 0.5263
Epoch 832/1000
177/177 [==============================] - 0s 85us/step - loss: 0.3548 - accuracy: 0.8475 - val_loss: 1.9191 - val_accuracy: 0.5263
Epoch 833/1000
177/177 [==============================] - 0s 85us/step - loss: 0.4312 - accuracy: 0.8418 - val_loss: 1.8745 - val_accuracy: 0.5263
Epoch 834/1000
177/177 [==============================] - 0s 78us/step - loss: 0.3512 - accuracy: 0.8644 - val_loss: 1.8235 - val_accuracy: 0.5263
Epoch 835/1000
177/177 [==============================] - 0s 92us/step - loss: 0.4008 - accuracy: 0.8588 - val_loss: 1.8161 - val_acc

177/177 [==============================] - 0s 105us/step - loss: 0.3796 - accuracy: 0.8531 - val_loss: 2.0345 - val_accuracy: 0.5263
Epoch 940/1000
177/177 [==============================] - 0s 118us/step - loss: 0.3630 - accuracy: 0.8814 - val_loss: 1.9825 - val_accuracy: 0.5263
Epoch 941/1000
177/177 [==============================] - 0s 105us/step - loss: 0.3529 - accuracy: 0.8644 - val_loss: 2.0159 - val_accuracy: 0.5395
Epoch 942/1000
177/177 [==============================] - 0s 107us/step - loss: 0.3696 - accuracy: 0.8644 - val_loss: 2.0375 - val_accuracy: 0.5000
Epoch 943/1000
177/177 [==============================] - 0s 207us/step - loss: 0.3400 - accuracy: 0.8588 - val_loss: 1.9720 - val_accuracy: 0.5263
Epoch 944/1000
177/177 [==============================] - 0s 98us/step - loss: 0.3594 - accuracy: 0.8644 - val_loss: 1.9639 - val_accuracy: 0.5263
Epoch 945/1000
177/177 [==============================] - 0s 116us/step - loss: 0.3747 - accuracy: 0.8418 - val_loss: 2.0432 - v

In [185]:
acc_test_sel2 = model_sel2.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel2*100))

76/76 [==============================] - 0s 120us/step
test accuracy: 53.95%


In [97]:
pred6 = model_sel2.predict_classes(X_sel_test)
pred6

array([1, 1, 1, 1, 1, 0, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2,
       1, 1, 2, 0, 2, 1, 2, 2, 1, 1, 0, 2, 2, 2, 1, 2, 0, 2, 0, 1, 1, 2,
       0, 2, 0, 1, 2, 0, 1, 2, 1, 1, 0, 2, 2, 1, 2, 1, 1, 1, 0, 2, 0, 0,
       2, 2, 0, 0, 0, 1, 1, 0, 2, 0])

In [98]:
dat6['pred'] = pred6
dat6

,strain,test,pred
152,NRS169,1,1
210,NRS249,2,1
218,NRS262,2,1
238,SR1746,1,1
113,NRS029,1,1
...,...,...,...
96,GA27,2,1
95,GA231,2,1
237,SR1287,0,0
14,506,2,2


In [99]:
proba6 = model_sel2.predict_proba(X_sel_test)
dat_proba6 = pd.DataFrame(proba6)

In [100]:
dat_proba6

,0,1,2
0,0.000020,0.964723,0.035257
1,0.000003,0.961344,0.038653
2,0.294295,0.370830,0.334875
3,0.004435,0.995246,0.000319
4,0.071749,0.922980,0.005271
...,...,...,...
71,0.058130,0.900391,0.041479
72,0.305791,0.503977,0.190232
73,0.929088,0.007562,0.063350
74,0.217415,0.136907,0.645678


In [101]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [102]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p40ST.csv", index = False,
         header=None)

In [189]:
hist_sel2 = model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 123us/step - loss: 0.4731 - accuracy: 0.8362 - val_loss: 1.5041 - val_accuracy: 0.5395
Epoch 2/1000
177/177 [==============================] - 0s 106us/step - loss: 0.4515 - accuracy: 0.8418 - val_loss: 1.5185 - val_accuracy: 0.5526
Epoch 3/1000
177/177 [==============================] - 0s 201us/step - loss: 0.5591 - accuracy: 0.8305 - val_loss: 1.5917 - val_accuracy: 0.5526
Epoch 4/1000
177/177 [==============================] - 0s 173us/step - loss: 0.4369 - accuracy: 0.8531 - val_loss: 1.4594 - val_accuracy: 0.5921
Epoch 5/1000
177/177 [==============================] - 0s 128us/step - loss: 0.5256 - accuracy: 0.8192 - val_loss: 1.4584 - val_accuracy: 0.5658
Epoch 6/1000
177/177 [==============================] - 0s 137us/step - loss: 0.3870 - accuracy: 0.8475 - val_loss: 1.5343 - val_accuracy: 0.5395
Epoch 7/1000
177/177 [==============================] - 0s 113us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 110us/step - loss: 0.4114 - accuracy: 0.8362 - val_loss: 1.6307 - val_accuracy: 0.5263
Epoch 113/1000
177/177 [==============================] - 0s 122us/step - loss: 0.5025 - accuracy: 0.8418 - val_loss: 1.6249 - val_accuracy: 0.5395
Epoch 114/1000
177/177 [==============================] - 0s 103us/step - loss: 0.5356 - accuracy: 0.8362 - val_loss: 1.5630 - val_accuracy: 0.5395
Epoch 115/1000
177/177 [==============================] - 0s 147us/step - loss: 0.4105 - accuracy: 0.8249 - val_loss: 1.6865 - val_accuracy: 0.5132
Epoch 116/1000
177/177 [==============================] - 0s 150us/step - loss: 0.5730 - accuracy: 0.8136 - val_loss: 1.6742 - val_accuracy: 0.5132
Epoch 117/1000
177/177 [==============================] - 0s 201us/step - loss: 0.3903 - accuracy: 0.8418 - val_loss: 1.5004 - val_accuracy: 0.5658
Epoch 118/1000
177/177 [==============================] - 0s 232us/step - loss: 0.4729 - accuracy: 0.8305 - val_

177/177 [==============================] - 0s 177us/step - loss: 0.4346 - accuracy: 0.8136 - val_loss: 1.5954 - val_accuracy: 0.5132
Epoch 223/1000
177/177 [==============================] - ETA: 0s - loss: 0.2301 - accuracy: 0.93 - 0s 101us/step - loss: 0.6037 - accuracy: 0.8249 - val_loss: 1.5809 - val_accuracy: 0.5263
Epoch 224/1000
177/177 [==============================] - 0s 106us/step - loss: 0.7369 - accuracy: 0.7910 - val_loss: 2.0843 - val_accuracy: 0.4605
Epoch 225/1000
177/177 [==============================] - 0s 88us/step - loss: 0.4755 - accuracy: 0.8136 - val_loss: 1.5764 - val_accuracy: 0.5658
Epoch 226/1000
177/177 [==============================] - 0s 156us/step - loss: 0.7306 - accuracy: 0.7627 - val_loss: 1.7255 - val_accuracy: 0.5263
Epoch 227/1000
177/177 [==============================] - 0s 99us/step - loss: 0.5353 - accuracy: 0.8192 - val_loss: 1.8444 - val_accuracy: 0.5132
Epoch 228/1000
177/177 [==============================] - 0s 114us/step - loss: 0.4937 

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.106287). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


177/177 [==============================] - 0s 673us/step - loss: 0.6075 - accuracy: 0.8136 - val_loss: 1.8902 - val_accuracy: 0.5132
Epoch 308/1000
177/177 [==============================] - 0s 228us/step - loss: 0.5246 - accuracy: 0.8079 - val_loss: 1.6450 - val_accuracy: 0.5395
Epoch 309/1000
177/177 [==============================] - 0s 114us/step - loss: 0.5604 - accuracy: 0.8362 - val_loss: 1.6264 - val_accuracy: 0.5000
Epoch 310/1000
177/177 [==============================] - 0s 119us/step - loss: 0.4995 - accuracy: 0.8249 - val_loss: 1.8710 - val_accuracy: 0.5000
Epoch 311/1000
177/177 [==============================] - 0s 110us/step - loss: 0.6836 - accuracy: 0.7684 - val_loss: 1.7675 - val_accuracy: 0.5000
Epoch 312/1000
177/177 [==============================] - 0s 120us/step - loss: 0.5034 - accuracy: 0.8418 - val_loss: 1.6895 - val_accuracy: 0.5132
Epoch 313/1000
177/177 [==============================] - 0s 122us/step - loss: 0.6512 - accuracy: 0.8249 - val_loss: 1.6586 - 

...

177/177 [==============================] - 0s 124us/step - loss: 0.4368 - accuracy: 0.8531 - val_loss: 1.8329 - val_accuracy: 0.5132
Epoch 528/1000
177/177 [==============================] - 0s 106us/step - loss: 0.3707 - accuracy: 0.8475 - val_loss: 1.7722 - val_accuracy: 0.4868
Epoch 529/1000
177/177 [==============================] - 0s 111us/step - loss: 0.4199 - accuracy: 0.8475 - val_loss: 1.8832 - val_accuracy: 0.4737
Epoch 530/1000
177/177 [==============================] - 0s 168us/step - loss: 0.4818 - accuracy: 0.8362 - val_loss: 1.9171 - val_accuracy: 0.4605
Epoch 531/1000
177/177 [==============================] - 0s 151us/step - loss: 0.4738 - accuracy: 0.8362 - val_loss: 1.7911 - val_accuracy: 0.5000
Epoch 532/1000
177/177 [==============================] - 0s 151us/step - loss: 0.5288 - accuracy: 0.8362 - val_loss: 1.9087 - val_accuracy: 0.4868
Epoch 533/1000
177/177 [==============================] - 0s 136us/step - loss: 0.3500 - accuracy: 0.8362 - val_loss: 1.8213 - 

177/177 [==============================] - 0s 193us/step - loss: 0.4580 - accuracy: 0.8362 - val_loss: 1.9799 - val_accuracy: 0.4737
Epoch 638/1000
177/177 [==============================] - 0s 105us/step - loss: 0.3488 - accuracy: 0.8644 - val_loss: 1.8414 - val_accuracy: 0.5132
Epoch 639/1000
177/177 [==============================] - 0s 108us/step - loss: 0.4895 - accuracy: 0.8531 - val_loss: 1.8749 - val_accuracy: 0.4737
Epoch 640/1000
177/177 [==============================] - 0s 94us/step - loss: 0.3331 - accuracy: 0.8418 - val_loss: 1.9899 - val_accuracy: 0.4474
Epoch 641/1000
177/177 [==============================] - 0s 171us/step - loss: 0.4279 - accuracy: 0.8418 - val_loss: 1.9735 - val_accuracy: 0.4737
Epoch 642/1000
177/177 [==============================] - 0s 98us/step - loss: 0.5244 - accuracy: 0.8362 - val_loss: 1.7711 - val_accuracy: 0.4868
Epoch 643/1000
177/177 [==============================] - 0s 90us/step - loss: 0.4475 - accuracy: 0.8531 - val_loss: 1.9179 - val

177/177 [==============================] - 0s 122us/step - loss: 0.3490 - accuracy: 0.8757 - val_loss: 2.0054 - val_accuracy: 0.4868
Epoch 748/1000
177/177 [==============================] - 0s 154us/step - loss: 0.3934 - accuracy: 0.8644 - val_loss: 1.9711 - val_accuracy: 0.4868
Epoch 749/1000
177/177 [==============================] - 0s 100us/step - loss: 0.3920 - accuracy: 0.8475 - val_loss: 2.0149 - val_accuracy: 0.4737
Epoch 750/1000
177/177 [==============================] - 0s 123us/step - loss: 0.3249 - accuracy: 0.8701 - val_loss: 2.0011 - val_accuracy: 0.5000
Epoch 751/1000
177/177 [==============================] - 0s 111us/step - loss: 0.3780 - accuracy: 0.8814 - val_loss: 2.0042 - val_accuracy: 0.4868
Epoch 752/1000
177/177 [==============================] - 0s 188us/step - loss: 0.4324 - accuracy: 0.8531 - val_loss: 2.0466 - val_accuracy: 0.4605
Epoch 753/1000
177/177 [==============================] - ETA: 0s - loss: 0.3402 - accuracy: 0.84 - 0s 197us/step - loss: 0.314

177/177 [==============================] - 0s 112us/step - loss: 0.6284 - accuracy: 0.8249 - val_loss: 2.1329 - val_accuracy: 0.4868
Epoch 858/1000
177/177 [==============================] - 0s 114us/step - loss: 0.3383 - accuracy: 0.8644 - val_loss: 2.0752 - val_accuracy: 0.4868
Epoch 859/1000
177/177 [==============================] - 0s 110us/step - loss: 0.5515 - accuracy: 0.8588 - val_loss: 2.0258 - val_accuracy: 0.5000
Epoch 860/1000
177/177 [==============================] - 0s 117us/step - loss: 0.5078 - accuracy: 0.8475 - val_loss: 2.2612 - val_accuracy: 0.4605
Epoch 861/1000
177/177 [==============================] - 0s 310us/step - loss: 0.3814 - accuracy: 0.8588 - val_loss: 2.0014 - val_accuracy: 0.4868
Epoch 862/1000
177/177 [==============================] - 0s 321us/step - loss: 0.4914 - accuracy: 0.8475 - val_loss: 2.0084 - val_accuracy: 0.4868
Epoch 863/1000
177/177 [==============================] - 0s 107us/step - loss: 0.3612 - accuracy: 0.8531 - val_loss: 2.1706 - 

177/177 [==============================] - 0s 96us/step - loss: 0.5411 - accuracy: 0.8475 - val_loss: 2.3294 - val_accuracy: 0.4605
Epoch 968/1000
177/177 [==============================] - 0s 151us/step - loss: 0.3296 - accuracy: 0.8870 - val_loss: 2.1284 - val_accuracy: 0.4737
Epoch 969/1000
177/177 [==============================] - 0s 163us/step - loss: 0.4239 - accuracy: 0.8644 - val_loss: 2.1152 - val_accuracy: 0.4605
Epoch 970/1000
177/177 [==============================] - 0s 292us/step - loss: 0.4141 - accuracy: 0.8531 - val_loss: 2.3050 - val_accuracy: 0.4342
Epoch 971/1000
177/177 [==============================] - 0s 173us/step - loss: 0.4352 - accuracy: 0.8079 - val_loss: 2.1876 - val_accuracy: 0.4474
Epoch 972/1000
177/177 [==============================] - 0s 138us/step - loss: 0.4363 - accuracy: 0.8531 - val_loss: 2.1169 - val_accuracy: 0.4474
Epoch 973/1000
177/177 [==============================] - 0s 120us/step - loss: 0.3562 - accuracy: 0.8588 - val_loss: 2.1547 - v

In [190]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel2.history['accuracy'])*100))

train accuracy: 84.22%


In [84]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [85]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,CFBREBSa104,1,2,1.290039e-09,1.567630e-07,9.999999e-01
1,p0017kpresabs_qual,NRS199,2,2,2.856965e-05,2.843749e-03,9.971277e-01
2,p0017kpresabs_qual,NRS233,1,0,9.999446e-01,4.541289e-06,5.090974e-05
3,p0017kpresabs_qual,SR1746,0,2,6.282367e-02,7.075194e-04,9.364688e-01
4,p0017kpresabs_qual,NRS202,2,2,8.229589e-03,2.163908e-05,9.917488e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,GA27,2,1,3.964591e-03,9.959286e-01,1.068284e-04
604,p0040presabsSTCC_qual,GA231,2,1,3.309226e-04,9.996691e-01,6.232397e-10
605,p0040presabsSTCC_qual,SR1287,0,1,8.445997e-06,9.999915e-01,1.182947e-13
606,p0040presabsSTCC_qual,506,2,0,6.516150e-01,1.480882e-02,3.335762e-01


In [86]:
y_prob6 = df_proba6[df_proba6['phage']=='p0040kpresabsSTCC_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[1.98687700e-05, 9.64723100e-01, 3.52570040e-02],
       [3.42582120e-06, 9.61344000e-01, 3.86525730e-02],
       [2.94294950e-01, 3.70830120e-01, 3.34874960e-01],
       [4.43533100e-03, 9.95245640e-01, 3.19084760e-04],
       [7.17489900e-02, 9.22980400e-01, 5.27071530e-03],
       [1.00000000e+00, 1.39304120e-08, 1.25349800e-09],
       [8.87440000e-04, 3.66171520e-03, 9.95450900e-01],
       [1.14780380e-02, 6.91300900e-01, 2.97221000e-01],
       [1.74424290e-01, 6.51607600e-02, 7.60414960e-01],
       [2.76033670e-01, 6.05019030e-01, 1.18947190e-01],
       [4.73191700e-02, 1.84352160e-02, 9.34245600e-01],
       [6.77203100e-03, 6.23574300e-03, 9.86992240e-01],
       [2.30473520e-02, 3.84769800e-02, 9.38475670e-01],
       [3.14404960e-01, 5.19510030e-01, 1.66085000e-01],
       [2.76033670e-01, 6.05019030e-01, 1.18947190e-01],
       [2.64496800e-04, 9.99729200e-01, 6.26693200e-06],
       [5.45805630e-05, 6.14442800e-05, 9.99884000e-01],
       [9.29555950e-03, 7.01120

In [87]:
ovo6 = rocauc_ovo(y_sel_test, y_prob6, average="macro", multi_class="ovo")
ovo6

0.6497607065668444

In [88]:
ovr6 = rocauc_ovr(y_sel_test, y_prob6, average="macro", multi_class="ovr")
ovr6

0.6497607065668444

In [89]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_sel)

In [90]:
dat7 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat7['test'] = y_sel_test

In [91]:
dat7

,strain,test
120,NRS063,1
187,NRS219,2
96,GA27,2
53,CFBREBSa119,1
197,NRS233,2
...,...,...
47,CFBREBSa110,2
68,CFBRSa05,0
56,CFBREBSa123,0
231,NY360,1


In [92]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [93]:
model_sel3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [108]:
model_sel3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [109]:
model_sel3.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 624us/step - loss: 26.0502 - accuracy: 0.2486 - val_loss: 21.3728 - val_accuracy: 0.2500
Epoch 2/1000
177/177 [==============================] - 0s 56us/step - loss: 23.0098 - accuracy: 0.2712 - val_loss: 18.3641 - val_accuracy: 0.2368
Epoch 3/1000
177/177 [==============================] - 0s 65us/step - loss: 20.3182 - accuracy: 0.2712 - val_loss: 15.5648 - val_accuracy: 0.2237
Epoch 4/1000
177/177 [==============================] - 0s 75us/step - loss: 17.4930 - accuracy: 0.3164 - val_loss: 13.0600 - val_accuracy: 0.2368
Epoch 5/1000
177/177 [==============================] - 0s 97us/step - loss: 15.0891 - accuracy: 0.3051 - val_loss: 10.7259 - val_accuracy: 0.2632
Epoch 6/1000
177/177 [==============================] - 0s 73us/step - loss: 12.2056 - accuracy: 0.3164 - val_loss: 8.5503 - val_accuracy: 0.2763
Epoch 7/1000
177/177 [==============================] - 0s 74us/step - lo

177/177 [==============================] - 0s 82us/step - loss: 0.9319 - accuracy: 0.5763 - val_loss: 1.1476 - val_accuracy: 0.4211
Epoch 57/1000
177/177 [==============================] - 0s 130us/step - loss: 0.9355 - accuracy: 0.5763 - val_loss: 1.2096 - val_accuracy: 0.4079
Epoch 58/1000
177/177 [==============================] - 0s 253us/step - loss: 0.9291 - accuracy: 0.5593 - val_loss: 1.1639 - val_accuracy: 0.4079
Epoch 59/1000
177/177 [==============================] - 0s 97us/step - loss: 0.9194 - accuracy: 0.5932 - val_loss: 1.1526 - val_accuracy: 0.4079
Epoch 60/1000
177/177 [==============================] - 0s 122us/step - loss: 0.9170 - accuracy: 0.5932 - val_loss: 1.1860 - val_accuracy: 0.3947
Epoch 61/1000
177/177 [==============================] - 0s 153us/step - loss: 0.9142 - accuracy: 0.5537 - val_loss: 1.1632 - val_accuracy: 0.3947
Epoch 62/1000
177/177 [==============================] - 0s 63us/step - loss: 0.9061 - accuracy: 0.6102 - val_loss: 1.1467 - val_accur

Epoch 112/1000
177/177 [==============================] - 0s 87us/step - loss: 0.8188 - accuracy: 0.6328 - val_loss: 1.1736 - val_accuracy: 0.4211
Epoch 113/1000
177/177 [==============================] - 0s 86us/step - loss: 0.8182 - accuracy: 0.6554 - val_loss: 1.1735 - val_accuracy: 0.4211
Epoch 114/1000
177/177 [==============================] - 0s 98us/step - loss: 0.8189 - accuracy: 0.6497 - val_loss: 1.1719 - val_accuracy: 0.4211
Epoch 115/1000
177/177 [==============================] - 0s 144us/step - loss: 0.8122 - accuracy: 0.6610 - val_loss: 1.1728 - val_accuracy: 0.4211
Epoch 116/1000
177/177 [==============================] - 0s 104us/step - loss: 0.8161 - accuracy: 0.6610 - val_loss: 1.1763 - val_accuracy: 0.4211
Epoch 117/1000
177/177 [==============================] - 0s 90us/step - loss: 0.8125 - accuracy: 0.6667 - val_loss: 1.1781 - val_accuracy: 0.4211
Epoch 118/1000
177/177 [==============================] - 0s 94us/step - loss: 0.8154 - accuracy: 0.6610 - val_loss:

Epoch 223/1000
177/177 [==============================] - 0s 98us/step - loss: 0.6977 - accuracy: 0.7119 - val_loss: 1.2911 - val_accuracy: 0.4474
Epoch 224/1000
177/177 [==============================] - 0s 137us/step - loss: 0.6943 - accuracy: 0.7119 - val_loss: 1.2855 - val_accuracy: 0.4474
Epoch 225/1000
177/177 [==============================] - 0s 155us/step - loss: 0.6924 - accuracy: 0.7232 - val_loss: 1.2958 - val_accuracy: 0.4605
Epoch 226/1000
177/177 [==============================] - 0s 97us/step - loss: 0.6916 - accuracy: 0.7062 - val_loss: 1.2940 - val_accuracy: 0.4605
Epoch 227/1000
177/177 [==============================] - 0s 81us/step - loss: 0.6916 - accuracy: 0.7119 - val_loss: 1.2940 - val_accuracy: 0.4605
Epoch 228/1000
177/177 [==============================] - 0s 173us/step - loss: 0.6919 - accuracy: 0.6836 - val_loss: 1.3114 - val_accuracy: 0.4474
Epoch 229/1000
177/177 [==============================] - 0s 148us/step - loss: 0.6889 - accuracy: 0.7006 - val_los

Epoch 334/1000
177/177 [==============================] - 0s 96us/step - loss: 0.6247 - accuracy: 0.7401 - val_loss: 1.4095 - val_accuracy: 0.4868
Epoch 335/1000
177/177 [==============================] - 0s 61us/step - loss: 0.6194 - accuracy: 0.7401 - val_loss: 1.4122 - val_accuracy: 0.4868
Epoch 336/1000
177/177 [==============================] - 0s 64us/step - loss: 0.6224 - accuracy: 0.7232 - val_loss: 1.3468 - val_accuracy: 0.4474
Epoch 337/1000
177/177 [==============================] - 0s 69us/step - loss: 0.6203 - accuracy: 0.7458 - val_loss: 1.3677 - val_accuracy: 0.4605
Epoch 338/1000
177/177 [==============================] - 0s 61us/step - loss: 0.6153 - accuracy: 0.7288 - val_loss: 1.4073 - val_accuracy: 0.4737
Epoch 339/1000
177/177 [==============================] - 0s 73us/step - loss: 0.6190 - accuracy: 0.7232 - val_loss: 1.4024 - val_accuracy: 0.4605
Epoch 340/1000
177/177 [==============================] - 0s 69us/step - loss: 0.6175 - accuracy: 0.7288 - val_loss: 1

...

Epoch 667/1000
177/177 [==============================] - 0s 69us/step - loss: 0.4878 - accuracy: 0.8249 - val_loss: 1.7193 - val_accuracy: 0.5000
Epoch 668/1000
177/177 [==============================] - 0s 60us/step - loss: 0.4857 - accuracy: 0.8249 - val_loss: 1.7180 - val_accuracy: 0.5000
Epoch 669/1000
177/177 [==============================] - 0s 86us/step - loss: 0.4862 - accuracy: 0.8249 - val_loss: 1.7104 - val_accuracy: 0.5000
Epoch 670/1000
177/177 [==============================] - 0s 65us/step - loss: 0.4855 - accuracy: 0.8249 - val_loss: 1.7184 - val_accuracy: 0.5000
Epoch 671/1000
177/177 [==============================] - 0s 68us/step - loss: 0.4873 - accuracy: 0.8249 - val_loss: 1.6959 - val_accuracy: 0.5000
Epoch 672/1000
177/177 [==============================] - 0s 70us/step - loss: 0.4868 - accuracy: 0.8136 - val_loss: 1.6954 - val_accuracy: 0.5000
Epoch 673/1000
177/177 [==============================] - 0s 56us/step - loss: 0.4880 - accuracy: 0.8136 - val_loss: 1

Epoch 723/1000
177/177 [==============================] - 0s 47us/step - loss: 0.4713 - accuracy: 0.8305 - val_loss: 1.7025 - val_accuracy: 0.5000
Epoch 724/1000
177/177 [==============================] - 0s 49us/step - loss: 0.4690 - accuracy: 0.8192 - val_loss: 1.7253 - val_accuracy: 0.5000
Epoch 725/1000
177/177 [==============================] - 0s 52us/step - loss: 0.4683 - accuracy: 0.8249 - val_loss: 1.7529 - val_accuracy: 0.5000
Epoch 726/1000
177/177 [==============================] - 0s 51us/step - loss: 0.4715 - accuracy: 0.8305 - val_loss: 1.7618 - val_accuracy: 0.5000
Epoch 727/1000
177/177 [==============================] - 0s 48us/step - loss: 0.4738 - accuracy: 0.8305 - val_loss: 1.7704 - val_accuracy: 0.5000
Epoch 728/1000
177/177 [==============================] - 0s 55us/step - loss: 0.4756 - accuracy: 0.8249 - val_loss: 1.6871 - val_accuracy: 0.5000
Epoch 729/1000
177/177 [==============================] - 0s 48us/step - loss: 0.4740 - accuracy: 0.8249 - val_loss: 1

Epoch 779/1000
177/177 [==============================] - 0s 54us/step - loss: 0.4729 - accuracy: 0.8192 - val_loss: 1.7777 - val_accuracy: 0.5000
Epoch 780/1000
177/177 [==============================] - 0s 58us/step - loss: 0.4544 - accuracy: 0.8362 - val_loss: 1.7708 - val_accuracy: 0.5000
Epoch 781/1000
177/177 [==============================] - 0s 58us/step - loss: 0.4593 - accuracy: 0.8305 - val_loss: 1.7470 - val_accuracy: 0.5000
Epoch 782/1000
177/177 [==============================] - 0s 53us/step - loss: 0.4587 - accuracy: 0.8305 - val_loss: 1.7489 - val_accuracy: 0.5000
Epoch 783/1000
177/177 [==============================] - 0s 53us/step - loss: 0.4578 - accuracy: 0.8249 - val_loss: 1.7734 - val_accuracy: 0.5000
Epoch 784/1000
177/177 [==============================] - 0s 60us/step - loss: 0.4531 - accuracy: 0.8305 - val_loss: 1.7882 - val_accuracy: 0.5000
Epoch 785/1000
177/177 [==============================] - 0s 57us/step - loss: 0.4581 - accuracy: 0.8305 - val_loss: 1

Epoch 890/1000
177/177 [==============================] - 0s 81us/step - loss: 0.4288 - accuracy: 0.8362 - val_loss: 1.8325 - val_accuracy: 0.5000
Epoch 891/1000
177/177 [==============================] - 0s 59us/step - loss: 0.4297 - accuracy: 0.8362 - val_loss: 1.8400 - val_accuracy: 0.5000
Epoch 892/1000
177/177 [==============================] - 0s 68us/step - loss: 0.4306 - accuracy: 0.8418 - val_loss: 1.8154 - val_accuracy: 0.5000
Epoch 893/1000
177/177 [==============================] - 0s 57us/step - loss: 0.4301 - accuracy: 0.8362 - val_loss: 1.8362 - val_accuracy: 0.5000
Epoch 894/1000
177/177 [==============================] - 0s 65us/step - loss: 0.4304 - accuracy: 0.8418 - val_loss: 1.8249 - val_accuracy: 0.5000
Epoch 895/1000
177/177 [==============================] - 0s 80us/step - loss: 0.4464 - accuracy: 0.8305 - val_loss: 1.8376 - val_accuracy: 0.4868
Epoch 896/1000
177/177 [==============================] - 0s 87us/step - loss: 0.4362 - accuracy: 0.8249 - val_loss: 1

Epoch 946/1000
177/177 [==============================] - 0s 82us/step - loss: 0.4200 - accuracy: 0.8362 - val_loss: 1.8701 - val_accuracy: 0.5000
Epoch 947/1000
177/177 [==============================] - 0s 67us/step - loss: 0.4216 - accuracy: 0.8305 - val_loss: 1.8926 - val_accuracy: 0.5000
Epoch 948/1000
177/177 [==============================] - 0s 86us/step - loss: 0.4218 - accuracy: 0.8305 - val_loss: 1.8833 - val_accuracy: 0.5000
Epoch 949/1000
177/177 [==============================] - 0s 93us/step - loss: 0.4182 - accuracy: 0.8418 - val_loss: 1.8139 - val_accuracy: 0.5000
Epoch 950/1000
177/177 [==============================] - 0s 84us/step - loss: 0.4243 - accuracy: 0.8305 - val_loss: 1.8124 - val_accuracy: 0.5000
Epoch 951/1000
177/177 [==============================] - 0s 82us/step - loss: 0.4215 - accuracy: 0.8305 - val_loss: 1.8326 - val_accuracy: 0.5000
Epoch 952/1000
177/177 [==============================] - 0s 84us/step - loss: 0.4201 - accuracy: 0.8418 - val_loss: 1

In [210]:
acc_test_sel3 = model_sel3.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel3*100))

76/76 [==============================] - 0s 142us/step
test accuracy: 46.05%


In [110]:
pred7 = model_sel3.predict_classes(X_sel_test)
pred7

array([1, 2, 0, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 1, 1, 0, 0, 0, 1, 2, 2,
       1, 1, 1, 0, 0, 1, 2, 1, 1, 2, 1, 2, 1, 1, 0, 2, 0, 0, 2, 2, 1, 1,
       1, 0, 1, 2, 2, 0, 2, 2, 1, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       1, 1, 0, 1, 0, 2, 1, 0, 1, 1])

In [111]:
dat7['pred'] = pred7
dat7

,strain,test,pred
120,NRS063,1,1
187,NRS219,2,2
96,GA27,2,0
53,CFBREBSa119,1,2
197,NRS233,2,2
...,...,...,...
47,CFBREBSa110,2,2
68,CFBRSa05,0,1
56,CFBREBSa123,0,0
231,NY360,1,1


In [112]:
proba7 = model_sel3.predict_proba(X_sel_test)
dat_proba7 = pd.DataFrame(proba7)

In [113]:
dat_proba7

,0,1,2
0,0.066164,0.921343,0.012493
1,0.032121,0.056917,0.910963
2,0.460097,0.334309,0.205594
3,0.032545,0.000779,0.966675
4,0.484537,0.007996,0.507467
...,...,...,...
71,0.054909,0.042800,0.902291
72,0.382733,0.598350,0.018917
73,0.803941,0.022875,0.173185
74,0.008815,0.565502,0.425683


In [114]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [115]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p40ST.csv", index = False,
         header=None)

In [214]:
hist_sel3 = model_sel3.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 89us/step - loss: 0.3647 - accuracy: 0.8418 - val_loss: 2.5519 - val_accuracy: 0.4605
Epoch 2/1000
177/177 [==============================] - 0s 88us/step - loss: 0.3641 - accuracy: 0.8418 - val_loss: 2.5935 - val_accuracy: 0.4474
Epoch 3/1000
177/177 [==============================] - 0s 85us/step - loss: 0.3627 - accuracy: 0.8418 - val_loss: 2.5527 - val_accuracy: 0.4474
Epoch 4/1000
177/177 [==============================] - 0s 125us/step - loss: 0.3627 - accuracy: 0.8362 - val_loss: 2.5584 - val_accuracy: 0.4474
Epoch 5/1000
177/177 [==============================] - 0s 111us/step - loss: 0.3631 - accuracy: 0.8418 - val_loss: 2.5459 - val_accuracy: 0.4737
Epoch 6/1000
177/177 [==============================] - 0s 105us/step - loss: 0.3644 - accuracy: 0.8418 - val_loss: 2.5866 - val_accuracy: 0.4737
Epoch 7/1000
177/177 [==============================] - 0s 106us/step - loss: 0.36

Epoch 57/1000
177/177 [==============================] - 0s 88us/step - loss: 0.3645 - accuracy: 0.8418 - val_loss: 2.7222 - val_accuracy: 0.4474
Epoch 58/1000
177/177 [==============================] - 0s 114us/step - loss: 0.3571 - accuracy: 0.8475 - val_loss: 2.6516 - val_accuracy: 0.4605
Epoch 59/1000
177/177 [==============================] - 0s 86us/step - loss: 0.3573 - accuracy: 0.8362 - val_loss: 2.5678 - val_accuracy: 0.4605
Epoch 60/1000
177/177 [==============================] - 0s 106us/step - loss: 0.3597 - accuracy: 0.8418 - val_loss: 2.5956 - val_accuracy: 0.4605
Epoch 61/1000
177/177 [==============================] - 0s 81us/step - loss: 0.3571 - accuracy: 0.8475 - val_loss: 2.6631 - val_accuracy: 0.4605
Epoch 62/1000
177/177 [==============================] - 0s 92us/step - loss: 0.3543 - accuracy: 0.8475 - val_loss: 2.6876 - val_accuracy: 0.4605
Epoch 63/1000
177/177 [==============================] - 0s 102us/step - loss: 0.3549 - accuracy: 0.8475 - val_loss: 2.669

Epoch 113/1000
177/177 [==============================] - 0s 97us/step - loss: 0.3663 - accuracy: 0.8418 - val_loss: 2.6076 - val_accuracy: 0.4342
Epoch 114/1000
177/177 [==============================] - 0s 83us/step - loss: 0.3562 - accuracy: 0.8362 - val_loss: 2.7092 - val_accuracy: 0.4474
Epoch 115/1000
177/177 [==============================] - 0s 78us/step - loss: 0.3734 - accuracy: 0.8362 - val_loss: 2.7436 - val_accuracy: 0.4474
Epoch 116/1000
177/177 [==============================] - 0s 73us/step - loss: 0.3574 - accuracy: 0.8418 - val_loss: 2.6362 - val_accuracy: 0.4342
Epoch 117/1000
177/177 [==============================] - 0s 92us/step - loss: 0.3700 - accuracy: 0.8475 - val_loss: 2.6426 - val_accuracy: 0.4474
Epoch 118/1000
177/177 [==============================] - 0s 64us/step - loss: 0.3612 - accuracy: 0.8475 - val_loss: 2.7127 - val_accuracy: 0.4605
Epoch 119/1000
177/177 [==============================] - 0s 78us/step - loss: 0.3554 - accuracy: 0.8475 - val_loss: 2

177/177 [==============================] - 0s 177us/step - loss: 0.3674 - accuracy: 0.8418 - val_loss: 2.7670 - val_accuracy: 0.4474
Epoch 224/1000
177/177 [==============================] - 0s 99us/step - loss: 0.3356 - accuracy: 0.8531 - val_loss: 2.9659 - val_accuracy: 0.4474
Epoch 225/1000
177/177 [==============================] - 0s 89us/step - loss: 0.4408 - accuracy: 0.8531 - val_loss: 2.8452 - val_accuracy: 0.4474
Epoch 226/1000
177/177 [==============================] - 0s 113us/step - loss: 0.3484 - accuracy: 0.8531 - val_loss: 2.6518 - val_accuracy: 0.4211
Epoch 227/1000
177/177 [==============================] - 0s 118us/step - loss: 0.4126 - accuracy: 0.8305 - val_loss: 2.7586 - val_accuracy: 0.4211
Epoch 228/1000
177/177 [==============================] - 0s 75us/step - loss: 0.3842 - accuracy: 0.8475 - val_loss: 2.7278 - val_accuracy: 0.4342
Epoch 229/1000
177/177 [==============================] - 0s 71us/step - loss: 0.3630 - accuracy: 0.8362 - val_loss: 2.8156 - val_

Epoch 279/1000
177/177 [==============================] - 0s 109us/step - loss: 0.3336 - accuracy: 0.8475 - val_loss: 2.8850 - val_accuracy: 0.4474
Epoch 280/1000
177/177 [==============================] - 0s 83us/step - loss: 0.3386 - accuracy: 0.8362 - val_loss: 2.8955 - val_accuracy: 0.4605
Epoch 281/1000
177/177 [==============================] - 0s 80us/step - loss: 0.3432 - accuracy: 0.8531 - val_loss: 2.8911 - val_accuracy: 0.4474
Epoch 282/1000
177/177 [==============================] - 0s 68us/step - loss: 0.3362 - accuracy: 0.8475 - val_loss: 2.8493 - val_accuracy: 0.4474
Epoch 283/1000
177/177 [==============================] - 0s 66us/step - loss: 0.3355 - accuracy: 0.8531 - val_loss: 2.7560 - val_accuracy: 0.4474
Epoch 284/1000
177/177 [==============================] - 0s 60us/step - loss: 0.3325 - accuracy: 0.8531 - val_loss: 2.8115 - val_accuracy: 0.4474
Epoch 285/1000
177/177 [==============================] - 0s 58us/step - loss: 0.3294 - accuracy: 0.8475 - val_loss: 

...

Epoch 612/1000
177/177 [==============================] - 0s 95us/step - loss: 0.3314 - accuracy: 0.8644 - val_loss: 3.1859 - val_accuracy: 0.4605
Epoch 613/1000
177/177 [==============================] - 0s 91us/step - loss: 0.3146 - accuracy: 0.8588 - val_loss: 3.1776 - val_accuracy: 0.4605
Epoch 614/1000
177/177 [==============================] - 0s 248us/step - loss: 0.3244 - accuracy: 0.8644 - val_loss: 3.2298 - val_accuracy: 0.4605
Epoch 615/1000
177/177 [==============================] - 0s 87us/step - loss: 0.3095 - accuracy: 0.8588 - val_loss: 3.2033 - val_accuracy: 0.4605
Epoch 616/1000
177/177 [==============================] - 0s 80us/step - loss: 0.3185 - accuracy: 0.8475 - val_loss: 3.1793 - val_accuracy: 0.4605
Epoch 617/1000
177/177 [==============================] - 0s 138us/step - loss: 0.3071 - accuracy: 0.8475 - val_loss: 3.2936 - val_accuracy: 0.4605
Epoch 618/1000
177/177 [==============================] - 0s 63us/step - loss: 0.3157 - accuracy: 0.8588 - val_loss:

Epoch 723/1000
177/177 [==============================] - 0s 82us/step - loss: 0.2970 - accuracy: 0.8531 - val_loss: 3.3723 - val_accuracy: 0.4605
Epoch 724/1000
177/177 [==============================] - 0s 73us/step - loss: 0.2987 - accuracy: 0.8531 - val_loss: 3.3739 - val_accuracy: 0.4605
Epoch 725/1000
177/177 [==============================] - 0s 71us/step - loss: 0.2990 - accuracy: 0.8644 - val_loss: 3.4173 - val_accuracy: 0.4605
Epoch 726/1000
177/177 [==============================] - 0s 71us/step - loss: 0.2943 - accuracy: 0.8644 - val_loss: 3.3224 - val_accuracy: 0.4605
Epoch 727/1000
177/177 [==============================] - 0s 90us/step - loss: 0.3014 - accuracy: 0.8531 - val_loss: 3.3215 - val_accuracy: 0.4605
Epoch 728/1000
177/177 [==============================] - 0s 179us/step - loss: 0.2985 - accuracy: 0.8531 - val_loss: 3.2958 - val_accuracy: 0.4605
Epoch 729/1000
177/177 [==============================] - 0s 163us/step - loss: 0.2983 - accuracy: 0.8644 - val_loss:

177/177 [==============================] - 0s 90us/step - loss: 0.2885 - accuracy: 0.8588 - val_loss: 3.4827 - val_accuracy: 0.4605
Epoch 834/1000
177/177 [==============================] - 0s 72us/step - loss: 0.2924 - accuracy: 0.8644 - val_loss: 3.4956 - val_accuracy: 0.4605
Epoch 835/1000
177/177 [==============================] - 0s 68us/step - loss: 0.2942 - accuracy: 0.8588 - val_loss: 3.5085 - val_accuracy: 0.4605
Epoch 836/1000
177/177 [==============================] - 0s 76us/step - loss: 0.2911 - accuracy: 0.8531 - val_loss: 3.4848 - val_accuracy: 0.4605
Epoch 837/1000
177/177 [==============================] - 0s 70us/step - loss: 0.2937 - accuracy: 0.8644 - val_loss: 3.5236 - val_accuracy: 0.4605
Epoch 838/1000
177/177 [==============================] - 0s 107us/step - loss: 0.2938 - accuracy: 0.8588 - val_loss: 3.5857 - val_accuracy: 0.4605
Epoch 839/1000
177/177 [==============================] - 0s 91us/step - loss: 0.2907 - accuracy: 0.8644 - val_loss: 3.5613 - val_ac

Epoch 889/1000
177/177 [==============================] - 0s 267us/step - loss: 0.2901 - accuracy: 0.8531 - val_loss: 3.6018 - val_accuracy: 0.4605
Epoch 890/1000
177/177 [==============================] - 0s 98us/step - loss: 0.2857 - accuracy: 0.8644 - val_loss: 3.6019 - val_accuracy: 0.4605
Epoch 891/1000
177/177 [==============================] - 0s 99us/step - loss: 0.2864 - accuracy: 0.8644 - val_loss: 3.6265 - val_accuracy: 0.4605
Epoch 892/1000
177/177 [==============================] - 0s 105us/step - loss: 0.2878 - accuracy: 0.8644 - val_loss: 3.6414 - val_accuracy: 0.4605
Epoch 893/1000
177/177 [==============================] - 0s 85us/step - loss: 0.2880 - accuracy: 0.8644 - val_loss: 3.5536 - val_accuracy: 0.4605
Epoch 894/1000
177/177 [==============================] - 0s 102us/step - loss: 0.2879 - accuracy: 0.8531 - val_loss: 3.5085 - val_accuracy: 0.4605
Epoch 895/1000
177/177 [==============================] - 0s 224us/step - loss: 0.2973 - accuracy: 0.8588 - val_los

Epoch 1000/1000
177/177 [==============================] - 0s 98us/step - loss: 0.2904 - accuracy: 0.8531 - val_loss: 3.7502 - val_accuracy: 0.4605


In [215]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel3.history['accuracy'])*100))

train accuracy: 85.13%


In [94]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [95]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS271,1,0,0.854929,0.144940,0.000130
1,p0017kpresabs_qual,SR2852,2,2,0.000001,0.000075,0.999923
2,p0017kpresabs_qual,CA39,2,0,0.959992,0.013406,0.026602
3,p0017kpresabs_qual,NRS266,1,2,0.000018,0.000003,0.999980
4,p0017kpresabs_qual,NY356,2,2,0.009156,0.411991,0.578852
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,CFBREBSa110,2,2,0.044135,0.010244,0.945621
604,p0040presabsSTCC_qual,CFBRSa05,0,2,0.145032,0.408530,0.446439
605,p0040presabsSTCC_qual,CFBREBSa123,0,0,0.994623,0.004152,0.001224
606,p0040presabsSTCC_qual,NY360,1,1,0.000795,0.948566,0.050639


In [96]:
y_prob7 = df_proba7[df_proba7['phage']=='p0040kpresabsSTCC_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[6.6163880e-02, 9.2134290e-01, 1.2493272e-02],
       [3.2120712e-02, 5.6916554e-02, 9.1096276e-01],
       [4.6009654e-01, 3.3430928e-01, 2.0559421e-01],
       [3.2545436e-02, 7.7917555e-04, 9.6667540e-01],
       [4.8453683e-01, 7.9964560e-03, 5.0746670e-01],
       [4.3814057e-01, 3.6826724e-01, 1.9359213e-01],
       [4.5524746e-01, 2.2191097e-01, 3.2284155e-01],
       [1.3562897e-05, 1.4558930e-02, 9.8542744e-01],
       [4.9082786e-02, 1.7595200e-01, 7.7496520e-01],
       [5.4908630e-02, 4.2800095e-02, 9.0229124e-01],
       [9.6993950e-01, 2.2581594e-02, 7.4789490e-03],
       [1.7308122e-12, 2.3563978e-06, 9.9999760e-01],
       [3.2120712e-02, 5.6916554e-02, 9.1096276e-01],
       [1.4794520e-06, 1.3236107e-01, 8.6763746e-01],
       [3.8582712e-02, 7.3957010e-01, 2.2184724e-01],
       [1.1013061e-02, 8.8465010e-01, 1.0433688e-01],
       [6.8838245e-01, 3.0999678e-01, 1.6207454e-03],
       [8.7510010e-01, 3.8821923e-03, 1.2101774e-01],
       [6.1174270e-01, 3.748

In [97]:
ovo7 = rocauc_ovo(y_sel_test, y_prob7, average="macro", multi_class="ovo")
ovo7

0.6293176692253538

In [98]:
ovr7 = rocauc_ovr(y_sel_test, y_prob7, average="macro", multi_class="ovr")
ovr7

0.6293176692253538

In [99]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_sel)

In [100]:
dat8 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat8['test'] = y_sel_test

In [101]:
dat8

,strain,test
130,NRS103,2
92,EUH25,0
143,NRS148,2
107,NRS001,1
166,NRS192,0
...,...,...
139,NRS113,1
24,BCH-SA-09,2
133,NRS106,2
62,CFBREBSa131,2


In [102]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [103]:
model_sel4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [121]:
model_sel4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [122]:
model_sel4.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 757us/step - loss: 27.7871 - accuracy: 0.3898 - val_loss: 4.6412 - val_accuracy: 0.3816
Epoch 2/1000
177/177 [==============================] - 0s 151us/step - loss: 23.4097 - accuracy: 0.4011 - val_loss: 3.1737 - val_accuracy: 0.3421
Epoch 3/1000
177/177 [==============================] - 0s 162us/step - loss: 19.5339 - accuracy: 0.3955 - val_loss: 2.0779 - val_accuracy: 0.4342
Epoch 4/1000
177/177 [==============================] - 0s 157us/step - loss: 16.7946 - accuracy: 0.4237 - val_loss: 2.5498 - val_accuracy: 0.4079
Epoch 5/1000
177/177 [==============================] - 0s 220us/step - loss: 16.2224 - accuracy: 0.4237 - val_loss: 2.3641 - val_accuracy: 0.4211
Epoch 6/1000
177/177 [==============================] - 0s 167us/step - loss: 14.3084 - accuracy: 0.4633 - val_loss: 1.6344 - val_accuracy: 0.4605
Epoch 7/1000
177/177 [==============================] - 0s 263us/step - l

177/177 [==============================] - 0s 222us/step - loss: 0.8820 - accuracy: 0.6497 - val_loss: 1.2182 - val_accuracy: 0.4737
Epoch 112/1000
177/177 [==============================] - 0s 175us/step - loss: 0.7200 - accuracy: 0.6836 - val_loss: 1.1920 - val_accuracy: 0.5132
Epoch 113/1000
177/177 [==============================] - 0s 326us/step - loss: 0.7946 - accuracy: 0.6667 - val_loss: 1.2024 - val_accuracy: 0.4737
Epoch 114/1000
177/177 [==============================] - 0s 146us/step - loss: 0.8241 - accuracy: 0.6667 - val_loss: 1.2048 - val_accuracy: 0.4605
Epoch 115/1000
177/177 [==============================] - 0s 142us/step - loss: 0.8523 - accuracy: 0.6610 - val_loss: 1.1633 - val_accuracy: 0.4737
Epoch 116/1000
177/177 [==============================] - 0s 100us/step - loss: 0.7645 - accuracy: 0.6554 - val_loss: 1.2286 - val_accuracy: 0.4737
Epoch 117/1000
177/177 [==============================] - 0s 105us/step - loss: 0.7793 - accuracy: 0.6836 - val_loss: 1.1800 - 

177/177 [==============================] - 0s 86us/step - loss: 0.6863 - accuracy: 0.7401 - val_loss: 1.3403 - val_accuracy: 0.4605
Epoch 222/1000
177/177 [==============================] - 0s 76us/step - loss: 0.6082 - accuracy: 0.7627 - val_loss: 1.2725 - val_accuracy: 0.4605
Epoch 223/1000
177/177 [==============================] - 0s 107us/step - loss: 0.5927 - accuracy: 0.7684 - val_loss: 1.3081 - val_accuracy: 0.5000
Epoch 224/1000
177/177 [==============================] - 0s 95us/step - loss: 0.5871 - accuracy: 0.7627 - val_loss: 1.3036 - val_accuracy: 0.4737
Epoch 225/1000
177/177 [==============================] - 0s 79us/step - loss: 0.6505 - accuracy: 0.7458 - val_loss: 1.2687 - val_accuracy: 0.4868
Epoch 226/1000
177/177 [==============================] - 0s 74us/step - loss: 0.7400 - accuracy: 0.7571 - val_loss: 1.4932 - val_accuracy: 0.4342
Epoch 227/1000
177/177 [==============================] - 0s 98us/step - loss: 0.7827 - accuracy: 0.7232 - val_loss: 1.3352 - val_ac

177/177 [==============================] - 0s 116us/step - loss: 0.5495 - accuracy: 0.8023 - val_loss: 1.4260 - val_accuracy: 0.4868
Epoch 332/1000
177/177 [==============================] - 0s 90us/step - loss: 0.5825 - accuracy: 0.7797 - val_loss: 1.4550 - val_accuracy: 0.5000
Epoch 333/1000
177/177 [==============================] - 0s 113us/step - loss: 0.5180 - accuracy: 0.8023 - val_loss: 1.5379 - val_accuracy: 0.4868
Epoch 334/1000
177/177 [==============================] - 0s 135us/step - loss: 0.5779 - accuracy: 0.8023 - val_loss: 1.4295 - val_accuracy: 0.4868
Epoch 335/1000
177/177 [==============================] - 0s 125us/step - loss: 0.6172 - accuracy: 0.8023 - val_loss: 1.4682 - val_accuracy: 0.4868
Epoch 336/1000
177/177 [==============================] - 0s 105us/step - loss: 0.5828 - accuracy: 0.7853 - val_loss: 1.5821 - val_accuracy: 0.4605
Epoch 337/1000
177/177 [==============================] - 0s 108us/step - loss: 0.6112 - accuracy: 0.7910 - val_loss: 1.4381 - v

177/177 [==============================] - 0s 71us/step - loss: 0.5093 - accuracy: 0.7910 - val_loss: 1.5941 - val_accuracy: 0.5132
Epoch 442/1000
177/177 [==============================] - 0s 79us/step - loss: 0.5636 - accuracy: 0.7853 - val_loss: 1.5924 - val_accuracy: 0.5132
Epoch 443/1000
177/177 [==============================] - 0s 74us/step - loss: 0.4760 - accuracy: 0.8192 - val_loss: 1.6789 - val_accuracy: 0.4737
Epoch 444/1000
177/177 [==============================] - 0s 70us/step - loss: 0.4984 - accuracy: 0.8192 - val_loss: 1.5587 - val_accuracy: 0.5000
Epoch 445/1000
177/177 [==============================] - 0s 70us/step - loss: 0.4976 - accuracy: 0.8136 - val_loss: 1.6133 - val_accuracy: 0.5132
Epoch 446/1000
177/177 [==============================] - 0s 67us/step - loss: 0.4922 - accuracy: 0.8136 - val_loss: 1.5843 - val_accuracy: 0.5132
Epoch 447/1000
177/177 [==============================] - 0s 79us/step - loss: 0.4564 - accuracy: 0.8192 - val_loss: 1.5853 - val_acc

Epoch 497/1000
177/177 [==============================] - 0s 81us/step - loss: 0.4238 - accuracy: 0.8418 - val_loss: 1.6231 - val_accuracy: 0.5000
Epoch 498/1000
177/177 [==============================] - 0s 73us/step - loss: 0.5120 - accuracy: 0.8136 - val_loss: 1.8236 - val_accuracy: 0.4605
Epoch 499/1000
177/177 [==============================] - 0s 127us/step - loss: 0.5239 - accuracy: 0.7966 - val_loss: 1.7108 - val_accuracy: 0.4474
Epoch 500/1000
177/177 [==============================] - 0s 120us/step - loss: 0.5271 - accuracy: 0.8249 - val_loss: 1.6282 - val_accuracy: 0.5000
Epoch 501/1000
177/177 [==============================] - 0s 97us/step - loss: 0.5540 - accuracy: 0.8079 - val_loss: 1.8456 - val_accuracy: 0.4342
Epoch 502/1000
177/177 [==============================] - 0s 98us/step - loss: 0.5489 - accuracy: 0.8023 - val_loss: 1.6549 - val_accuracy: 0.5000
Epoch 503/1000
177/177 [==============================] - 0s 111us/step - loss: 0.4357 - accuracy: 0.8305 - val_loss

177/177 [==============================] - 0s 95us/step - loss: 0.4323 - accuracy: 0.8305 - val_loss: 1.8448 - val_accuracy: 0.4737
Epoch 608/1000
177/177 [==============================] - 0s 103us/step - loss: 0.4148 - accuracy: 0.8362 - val_loss: 1.7793 - val_accuracy: 0.5000
Epoch 609/1000
177/177 [==============================] - 0s 102us/step - loss: 0.4380 - accuracy: 0.8362 - val_loss: 1.8476 - val_accuracy: 0.4868
Epoch 610/1000
177/177 [==============================] - 0s 107us/step - loss: 0.5653 - accuracy: 0.8362 - val_loss: 1.9770 - val_accuracy: 0.4605
Epoch 611/1000
177/177 [==============================] - 0s 111us/step - loss: 0.5270 - accuracy: 0.8475 - val_loss: 1.8143 - val_accuracy: 0.4737
Epoch 612/1000
177/177 [==============================] - 0s 128us/step - loss: 0.3911 - accuracy: 0.8475 - val_loss: 1.8436 - val_accuracy: 0.4737
Epoch 613/1000
177/177 [==============================] - 0s 96us/step - loss: 0.4636 - accuracy: 0.8136 - val_loss: 1.8648 - va

177/177 [==============================] - 0s 91us/step - loss: 0.3885 - accuracy: 0.8475 - val_loss: 1.9162 - val_accuracy: 0.4605
Epoch 718/1000
177/177 [==============================] - 0s 81us/step - loss: 0.6241 - accuracy: 0.8079 - val_loss: 2.1286 - val_accuracy: 0.4342
Epoch 719/1000
177/177 [==============================] - 0s 102us/step - loss: 0.4818 - accuracy: 0.8136 - val_loss: 2.0523 - val_accuracy: 0.4605
Epoch 720/1000
177/177 [==============================] - 0s 103us/step - loss: 0.4111 - accuracy: 0.8249 - val_loss: 1.9063 - val_accuracy: 0.4737
Epoch 721/1000
177/177 [==============================] - 0s 91us/step - loss: 0.4041 - accuracy: 0.8418 - val_loss: 2.0539 - val_accuracy: 0.4474
Epoch 722/1000
177/177 [==============================] - 0s 121us/step - loss: 0.4554 - accuracy: 0.8249 - val_loss: 2.0337 - val_accuracy: 0.4342
Epoch 723/1000
177/177 [==============================] - 0s 108us/step - loss: 0.4119 - accuracy: 0.8362 - val_loss: 1.9251 - val

177/177 [==============================] - 0s 102us/step - loss: 0.3658 - accuracy: 0.8757 - val_loss: 2.0676 - val_accuracy: 0.4211
Epoch 828/1000
177/177 [==============================] - 0s 97us/step - loss: 0.3643 - accuracy: 0.8644 - val_loss: 2.0526 - val_accuracy: 0.4342
Epoch 829/1000
177/177 [==============================] - 0s 102us/step - loss: 0.3612 - accuracy: 0.8588 - val_loss: 2.0894 - val_accuracy: 0.4474
Epoch 830/1000
177/177 [==============================] - 0s 107us/step - loss: 0.4042 - accuracy: 0.8475 - val_loss: 2.0987 - val_accuracy: 0.4342
Epoch 831/1000
177/177 [==============================] - 0s 100us/step - loss: 0.4540 - accuracy: 0.8418 - val_loss: 2.0762 - val_accuracy: 0.4474
Epoch 832/1000
177/177 [==============================] - 0s 91us/step - loss: 0.3980 - accuracy: 0.8531 - val_loss: 2.1000 - val_accuracy: 0.4342
Epoch 833/1000
177/177 [==============================] - 0s 108us/step - loss: 0.3518 - accuracy: 0.8531 - val_loss: 2.0883 - va

177/177 [==============================] - 0s 102us/step - loss: 0.3305 - accuracy: 0.8588 - val_loss: 2.1769 - val_accuracy: 0.4342
Epoch 938/1000
177/177 [==============================] - 0s 131us/step - loss: 0.3552 - accuracy: 0.8701 - val_loss: 2.2265 - val_accuracy: 0.4211
Epoch 939/1000
177/177 [==============================] - 0s 115us/step - loss: 0.3611 - accuracy: 0.8701 - val_loss: 2.1696 - val_accuracy: 0.4737
Epoch 940/1000
177/177 [==============================] - 0s 115us/step - loss: 0.3803 - accuracy: 0.8701 - val_loss: 2.2524 - val_accuracy: 0.3947
Epoch 941/1000
177/177 [==============================] - 0s 110us/step - loss: 0.4384 - accuracy: 0.8531 - val_loss: 2.2792 - val_accuracy: 0.4079
Epoch 942/1000
177/177 [==============================] - 0s 235us/step - loss: 0.3831 - accuracy: 0.8588 - val_loss: 2.1795 - val_accuracy: 0.4605
Epoch 943/1000
177/177 [==============================] - 0s 128us/step - loss: 0.4247 - accuracy: 0.8757 - val_loss: 2.2389 - 

In [227]:
acc_test_sel4 = model_sel4.evaluate(X_sel_test, y_sel_test)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_sel4*100))

76/76 [==============================] - 0s 113us/step
over-sampling test accuracy: 52.63%


In [123]:
pred8 = model_sel4.predict_classes(X_sel_test)
pred8

array([1, 0, 2, 0, 2, 2, 0, 0, 1, 2, 2, 1, 1, 0, 2, 1, 2, 1, 1, 0, 2, 0,
       0, 0, 1, 2, 0, 0, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2,
       2, 0, 2, 0, 0, 1, 1, 1, 2, 0, 0, 2, 2, 0, 1, 2, 0, 2, 0, 2, 0, 2,
       1, 2, 2, 1, 1, 2, 1, 2, 2, 0])

In [124]:
dat8['pred'] = pred8
dat8

,strain,test,pred
130,NRS103,2,1
92,EUH25,0,0
143,NRS148,2,2
107,NRS001,1,0
166,NRS192,0,2
...,...,...,...
139,NRS113,1,2
24,BCH-SA-09,2,1
133,NRS106,2,2
62,CFBREBSa131,2,2


In [125]:
proba8 = model_sel4.predict_proba(X_sel_test)
dat_proba8 = pd.DataFrame(proba8)

In [126]:
dat_proba8

,0,1,2
0,7.441123e-09,1.000000,3.137041e-07
1,8.260856e-01,0.030235,1.436798e-01
2,4.092476e-02,0.070835,8.882402e-01
3,9.417498e-01,0.047829,1.042163e-02
4,1.214602e-01,0.024419,8.541209e-01
...,...,...,...
71,7.205188e-02,0.015334,9.126146e-01
72,1.265803e-01,0.777837,9.558301e-02
73,3.180211e-01,0.307720,3.742593e-01
74,3.173670e-01,0.002998,6.796347e-01


In [127]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [128]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p40ST.csv", index = False,
         header=None)

In [231]:
hist_sel4 = model_sel4.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 157us/step - loss: 0.5576 - accuracy: 0.8531 - val_loss: 2.4155 - val_accuracy: 0.4868
Epoch 2/1000
177/177 [==============================] - 0s 146us/step - loss: 0.5358 - accuracy: 0.8475 - val_loss: 2.5646 - val_accuracy: 0.4737
Epoch 3/1000
177/177 [==============================] - 0s 151us/step - loss: 0.4575 - accuracy: 0.8644 - val_loss: 2.3652 - val_accuracy: 0.5132
Epoch 4/1000
177/177 [==============================] - 0s 192us/step - loss: 0.4959 - accuracy: 0.8418 - val_loss: 2.3355 - val_accuracy: 0.5132
Epoch 5/1000
177/177 [==============================] - 0s 131us/step - loss: 0.3945 - accuracy: 0.8475 - val_loss: 2.4211 - val_accuracy: 0.4605
Epoch 6/1000
177/177 [==============================] - 0s 161us/step - loss: 0.4019 - accuracy: 0.8531 - val_loss: 2.3135 - val_accuracy: 0.5000
Epoch 7/1000
177/177 [==============================] - 0s 190us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 100us/step - loss: 0.3601 - accuracy: 0.8814 - val_loss: 2.4821 - val_accuracy: 0.4737
Epoch 113/1000
177/177 [==============================] - 0s 146us/step - loss: 0.3654 - accuracy: 0.8531 - val_loss: 2.5402 - val_accuracy: 0.4474
Epoch 114/1000
177/177 [==============================] - 0s 121us/step - loss: 0.3585 - accuracy: 0.8870 - val_loss: 2.4831 - val_accuracy: 0.5263
Epoch 115/1000
177/177 [==============================] - 0s 117us/step - loss: 0.3436 - accuracy: 0.8870 - val_loss: 2.4727 - val_accuracy: 0.5000
Epoch 116/1000
177/177 [==============================] - 0s 134us/step - loss: 0.3244 - accuracy: 0.8701 - val_loss: 2.5016 - val_accuracy: 0.4868
Epoch 117/1000
177/177 [==============================] - 0s 174us/step - loss: 0.3829 - accuracy: 0.8531 - val_loss: 2.5370 - val_accuracy: 0.4868
Epoch 118/1000
177/177 [==============================] - 0s 167us/step - loss: 0.4973 - accuracy: 0.8531 - val_

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.107537). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


177/177 [==============================] - 0s 112us/step - loss: 0.4623 - accuracy: 0.8701 - val_loss: 2.5753 - val_accuracy: 0.5132
Epoch 184/1000
177/177 [==============================] - 0s 127us/step - loss: 0.3993 - accuracy: 0.8701 - val_loss: 2.5761 - val_accuracy: 0.5000
Epoch 185/1000
177/177 [==============================] - 0s 110us/step - loss: 0.3290 - accuracy: 0.8701 - val_loss: 2.5447 - val_accuracy: 0.5000
Epoch 186/1000
177/177 [==============================] - 0s 623us/step - loss: 0.3189 - accuracy: 0.8814 - val_loss: 2.5800 - val_accuracy: 0.5132
Epoch 187/1000
177/177 [==============================] - 0s 480us/step - loss: 0.3412 - accuracy: 0.8814 - val_loss: 2.5853 - val_accuracy: 0.5132
Epoch 188/1000
177/177 [==============================] - 0s 231us/step - loss: 0.2928 - accuracy: 0.8757 - val_loss: 2.6909 - val_accuracy: 0.4605
Epoch 189/1000
177/177 [==============================] - 0s 132us/step - loss: 0.4052 - accuracy: 0.8418 - val_loss: 2.7068 - 

177/177 [==============================] - 0s 349us/step - loss: 0.3596 - accuracy: 0.8814 - val_loss: 2.6586 - val_accuracy: 0.4868
Epoch 294/1000
177/177 [==============================] - 0s 134us/step - loss: 0.3594 - accuracy: 0.8701 - val_loss: 2.6676 - val_accuracy: 0.4868
Epoch 295/1000
177/177 [==============================] - 0s 109us/step - loss: 0.2856 - accuracy: 0.8757 - val_loss: 2.6817 - val_accuracy: 0.4605
Epoch 296/1000
177/177 [==============================] - 0s 103us/step - loss: 0.2786 - accuracy: 0.8814 - val_loss: 2.7266 - val_accuracy: 0.4868
Epoch 297/1000
177/177 [==============================] - 0s 89us/step - loss: 0.2938 - accuracy: 0.8927 - val_loss: 2.7450 - val_accuracy: 0.5000
Epoch 298/1000
177/177 [==============================] - 0s 108us/step - loss: 0.2912 - accuracy: 0.8927 - val_loss: 2.6958 - val_accuracy: 0.4868
Epoch 299/1000
177/177 [==============================] - 0s 109us/step - loss: 0.2896 - accuracy: 0.8814 - val_loss: 2.6923 - v

...

177/177 [==============================] - 0s 181us/step - loss: 0.3326 - accuracy: 0.8814 - val_loss: 3.0187 - val_accuracy: 0.4737
Epoch 514/1000
177/177 [==============================] - 0s 171us/step - loss: 0.3299 - accuracy: 0.8644 - val_loss: 3.0909 - val_accuracy: 0.4342
Epoch 515/1000
177/177 [==============================] - 0s 114us/step - loss: 0.3366 - accuracy: 0.8814 - val_loss: 3.0049 - val_accuracy: 0.4605
Epoch 516/1000
177/177 [==============================] - 0s 123us/step - loss: 0.2663 - accuracy: 0.8983 - val_loss: 3.0450 - val_accuracy: 0.4605
Epoch 517/1000
177/177 [==============================] - 0s 125us/step - loss: 0.3469 - accuracy: 0.8814 - val_loss: 3.0542 - val_accuracy: 0.4737
Epoch 518/1000
177/177 [==============================] - 0s 131us/step - loss: 0.2564 - accuracy: 0.8870 - val_loss: 3.0572 - val_accuracy: 0.4737
Epoch 519/1000
177/177 [==============================] - 0s 103us/step - loss: 0.2811 - accuracy: 0.8983 - val_loss: 3.0324 - 

177/177 [==============================] - 0s 119us/step - loss: 0.2860 - accuracy: 0.8701 - val_loss: 3.2250 - val_accuracy: 0.5132
Epoch 624/1000
177/177 [==============================] - 0s 115us/step - loss: 0.5666 - accuracy: 0.8701 - val_loss: 3.1898 - val_accuracy: 0.4605
Epoch 625/1000
177/177 [==============================] - 0s 156us/step - loss: 0.3858 - accuracy: 0.8644 - val_loss: 3.2357 - val_accuracy: 0.4211
Epoch 626/1000
177/177 [==============================] - 0s 152us/step - loss: 0.3416 - accuracy: 0.8588 - val_loss: 3.1641 - val_accuracy: 0.4737
Epoch 627/1000
177/177 [==============================] - 0s 124us/step - loss: 0.3096 - accuracy: 0.8983 - val_loss: 3.2126 - val_accuracy: 0.4605
Epoch 628/1000
177/177 [==============================] - 0s 140us/step - loss: 0.3087 - accuracy: 0.8814 - val_loss: 3.2385 - val_accuracy: 0.4605
Epoch 629/1000
177/177 [==============================] - 0s 265us/step - loss: 0.2433 - accuracy: 0.8870 - val_loss: 3.1840 - 

177/177 [==============================] - 0s 119us/step - loss: 0.3518 - accuracy: 0.8701 - val_loss: 3.4457 - val_accuracy: 0.4211
Epoch 734/1000
177/177 [==============================] - 0s 117us/step - loss: 0.3233 - accuracy: 0.8588 - val_loss: 3.3908 - val_accuracy: 0.4342
Epoch 735/1000
177/177 [==============================] - 0s 100us/step - loss: 0.2489 - accuracy: 0.8814 - val_loss: 3.3546 - val_accuracy: 0.4868
Epoch 736/1000
177/177 [==============================] - 0s 100us/step - loss: 0.2994 - accuracy: 0.8814 - val_loss: 3.3598 - val_accuracy: 0.4737
Epoch 737/1000
177/177 [==============================] - 0s 107us/step - loss: 0.3650 - accuracy: 0.8757 - val_loss: 3.5669 - val_accuracy: 0.4605
Epoch 738/1000
177/177 [==============================] - 0s 105us/step - loss: 0.3492 - accuracy: 0.8757 - val_loss: 3.3788 - val_accuracy: 0.4737
Epoch 739/1000
177/177 [==============================] - 0s 100us/step - loss: 0.3119 - accuracy: 0.8870 - val_loss: 3.3559 - 

177/177 [==============================] - 0s 134us/step - loss: 0.2286 - accuracy: 0.8927 - val_loss: 3.5677 - val_accuracy: 0.4737
Epoch 844/1000
177/177 [==============================] - 0s 114us/step - loss: 0.3622 - accuracy: 0.8927 - val_loss: 3.5765 - val_accuracy: 0.4737
Epoch 845/1000
177/177 [==============================] - 0s 159us/step - loss: 0.2322 - accuracy: 0.8870 - val_loss: 3.6488 - val_accuracy: 0.4211
Epoch 846/1000
177/177 [==============================] - 0s 122us/step - loss: 0.3419 - accuracy: 0.8701 - val_loss: 3.5779 - val_accuracy: 0.4737
Epoch 847/1000
177/177 [==============================] - 0s 123us/step - loss: 0.3305 - accuracy: 0.8983 - val_loss: 3.5545 - val_accuracy: 0.4605
Epoch 848/1000
177/177 [==============================] - 0s 147us/step - loss: 0.2104 - accuracy: 0.8983 - val_loss: 3.6533 - val_accuracy: 0.4211
Epoch 849/1000
177/177 [==============================] - 0s 184us/step - loss: 0.4378 - accuracy: 0.8531 - val_loss: 3.6733 - 

177/177 [==============================] - 0s 261us/step - loss: 0.2638 - accuracy: 0.8927 - val_loss: 3.7161 - val_accuracy: 0.4342
Epoch 954/1000
177/177 [==============================] - 0s 125us/step - loss: 0.3223 - accuracy: 0.8983 - val_loss: 3.9360 - val_accuracy: 0.4211
Epoch 955/1000
177/177 [==============================] - 0s 128us/step - loss: 0.3123 - accuracy: 0.8701 - val_loss: 3.7337 - val_accuracy: 0.4605
Epoch 956/1000
177/177 [==============================] - 0s 116us/step - loss: 0.3209 - accuracy: 0.8870 - val_loss: 3.7103 - val_accuracy: 0.4474
Epoch 957/1000
177/177 [==============================] - 0s 602us/step - loss: 0.3167 - accuracy: 0.8983 - val_loss: 3.7479 - val_accuracy: 0.4342
Epoch 958/1000
177/177 [==============================] - 0s 139us/step - loss: 0.2502 - accuracy: 0.8814 - val_loss: 3.8261 - val_accuracy: 0.4342
Epoch 959/1000
177/177 [==============================] - 0s 124us/step - loss: 0.4149 - accuracy: 0.8814 - val_loss: 3.7537 - 

In [232]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel4.history['accuracy'])*100))

train accuracy: 87.92%


In [104]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [105]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS104,0,1,8.210638e-02,0.917809,8.477923e-05
1,p0017kpresabs_qual,NRS071,0,2,1.015229e-02,0.255327,7.345203e-01
2,p0017kpresabs_qual,NRS072,1,2,3.642946e-03,0.000002,9.963547e-01
3,p0017kpresabs_qual,BCH-SA-12,0,2,2.809318e-03,0.099188,8.980029e-01
4,p0017kpresabs_qual,CFBREBSa125,2,0,4.875667e-01,0.088657,4.237761e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS113,1,2,2.686909e-07,0.093121,9.068785e-01
604,p0040presabsSTCC_qual,BCH-SA-09,2,1,5.529492e-07,0.999990,9.559324e-06
605,p0040presabsSTCC_qual,NRS106,2,1,1.115902e-03,0.998816,6.852559e-05
606,p0040presabsSTCC_qual,CFBREBSa131,2,2,2.708040e-07,0.156768,8.432316e-01


In [106]:
y_prob8 = df_proba8[df_proba8['phage']=='p0040kpresabsSTCC_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[7.44112300e-09, 9.99999640e-01, 3.13704080e-07],
       [8.26085570e-01, 3.02346000e-02, 1.43679800e-01],
       [4.09247580e-02, 7.08350240e-02, 8.88240200e-01],
       [9.41749800e-01, 4.78286400e-02, 1.04216300e-02],
       [1.21460244e-01, 2.44188680e-02, 8.54120850e-01],
       [9.40836740e-02, 1.35315870e-01, 7.70600400e-01],
       [8.03121100e-01, 5.93025570e-02, 1.37576360e-01],
       [9.60081100e-01, 3.12673450e-02, 8.65150200e-03],
       [5.96103070e-02, 6.01115350e-01, 3.39274380e-01],
       [3.73587200e-01, 8.47830600e-03, 6.17934500e-01],
       [9.17856250e-02, 2.74709280e-01, 6.33505050e-01],
       [1.10934090e-03, 9.98890700e-01, 1.23283940e-08],
       [9.22625600e-03, 9.55500540e-01, 3.52731940e-02],
       [7.27837560e-01, 6.12933100e-02, 2.10869150e-01],
       [9.17856250e-02, 2.74709280e-01, 6.33505050e-01],
       [2.00453880e-01, 5.16905840e-01, 2.82640250e-01],
       [9.28353200e-02, 4.50314800e-01, 4.56849930e-01],
       [1.12440550e-04, 6.40546

In [107]:
ovo8 = rocauc_ovo(y_sel_test, y_prob8, average="macro", multi_class="ovo")
ovo8

0.6223127895523811

In [108]:
ovr8 = rocauc_ovr(y_sel_test, y_prob8, average="macro", multi_class="ovr")
ovr8

0.6223127895523811

In [109]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.6172133273104615

In [110]:
np.std(ovos2)

0.03044269927183505

In [111]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.6172133273104615

In [112]:
np.std(ovrs2)

0.03044269927183505

In [233]:
accs_l= [acc_test_sel, acc_test_sel2, acc_test_sel3, acc_test_sel4]

In [234]:
mean_l = np.mean(accs_l)
print('test accuracy mean after lasso: %.2f%%' % (mean_l*100))

test accuracy mean after lasso: 50.33%


In [235]:
std_l = np.std(accs_l)
print('test accuracy standard deviation after lasso:', std_l)

test accuracy standard deviation after lasso: 0.03137959128497353


In [236]:
accs_train_l = [np.mean(hist_sel.history['accuracy']), np.mean(hist_sel2.history['accuracy']), np.mean(hist_sel3.history['accuracy']),
             np.mean(hist_sel4.history['accuracy'])]

In [237]:
mean_train_l = np.mean(accs_train_l)
print('train accuracy mean after lasso: %.2f%%' % (mean_train_l*100))

train accuracy mean after lasso: 86.35%


In [238]:
std_train_l = np.std(accs_train_l)
print('train accuracy standard deviation after lasso:', std_train_l)

train accuracy standard deviation after lasso: 0.01707402
